<a href="https://colab.research.google.com/github/bemarina/text-clf-deep-learning/blob/main/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Filings classification**

In [37]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd


In [19]:
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [27]:

def preprocess_text(text):
  """
  Function to clean the text, remove special characters, stop words and digits.
  Args:
    text: String to be preprocessed.
  Returns cleaned text.
  """
  text=text.lower()
  # remove new lines
  text=text.replace('\n', ' ')
  # remove digits
  text=re.sub(r'\d+', ' ', text)
  text=re.sub(r'[^\w\s]',' ',text)
  text=text.replace('\t', ' ')
  # tokenize text
  tokens=nltk.word_tokenize(text)
  # remove stopwords
  stop_words=stopwords.words('english')
  tokens=[word for word in tokens if word not in stop_words]
  # remove empty tokens
  tokens=[word for word in tokens if word]
  # remove single character tokens
  tokens=[word for word in tokens if len(word) > 1]
  # keep unique words only
  tokens = list(set(tokens))
  tokens=' '.join(tokens)
  return tokens

In [34]:
import os

# access file from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [45]:
# list file contents Google Drive
#!ls /content/drive/My\ Drive/MLProjects

In [ ]:
# import os

# # Create a folder in the root directory
# # !mkdir -p "/content/drive/My Drive/My Folder"

# # Write a sample file to Google Drive
# with open('/content/drive/My Drive/MLProjects/sample.txt', 'w') as f:
#   f.write('Hello, World!')


In [ ]:
# load file from Google Drive
#data = open('/content/drive/My Drive/MLProjects/sample.txt').read()

In [42]:
# load csv file from Google Drive
data = pd.read_csv('/content/drive/My Drive/MLProjects/data-another-try-v2.csv')

In [44]:
data.head()

,number,filing_date,report_date,url,filing_text,explanatory_note,category
0,1,2000-05-01,2000-04-01,NaN,reports nj standard data prepaid oz investing ...,NaN,NaN
1,2,2000-04-14,1999-11-30,NaN,reports illinois property similar standard rel...,NaN,NaN
2,3,2000-04-14,1999-08-31,NaN,reports illinois property similar standard rel...,NaN,NaN
3,4,1996-09-27,1996-02-29,NaN,NaN,NaN,NaN
4,5,1997-12-22,1997-08-31,NaN,NaN,NaN,NaN


In [43]:
data['category'].unique()

array([nan, 'Regulatory Response', 'Disclosures', 'Errors or Mistakes',
       'Restatements'], dtype=object)

In [ ]:
print(data.shape)
data.groupby('category').count()

(369, 5)


,filing_date,report_date,url,explanatory_note
category,,,,
Disclosures,78,78,78,78
Errors or Mistakes,10,10,10,10
Regulatory Response,6,6,6,6
Restatements,25,25,25,25


In [ ]:
print(data['url'].isna().sum())
print(data['category'].isna().sum())
print(data['category'].notna().sum())
print(data['explanatory_note'].isna().sum())


0
250
119
235


In [ ]:
https://www.sec.gov/Archives/edgar/data/1800/0...

In [ ]:
data.head(n=10)

,filing_date,report_date,url,explanatory_note,category
0,2000-05-01,2000-04-01,https://www.sec.gov/Archives/edgar/data/20/000...,NaN,NaN
1,2000-04-14,1999-11-30,https://www.sec.gov/Archives/edgar/data/1750/0...,NaN,NaN
2,2000-04-14,1999-08-31,https://www.sec.gov/Archives/edgar/data/1750/0...,NaN,NaN
3,1996-09-27,1996-02-29,https://www.sec.gov/Archives/edgar/data/1761/0...,NaN,NaN
4,1997-12-22,1997-08-31,https://www.sec.gov/Archives/edgar/data/1761/0...,NaN,NaN
5,2013-03-27,2012-12-31,https://www.sec.gov/Archives/edgar/data/1800/0...,This Form 10-K/A is being filed to correct a d...,Regulatory Response
6,1998-09-02,1998-04-30,https://www.sec.gov/Archives/edgar/data/1923/0...,NaN,NaN
7,1999-08-30,1999-04-30,https://www.sec.gov/Archives/edgar/data/1923/0...,NaN,NaN
8,2007-03-14,2006-10-31,https://www.sec.gov/Archives/edgar/data/1923/0...,This Amendment No. 1 to the Quarterly Report o...,Disclosures
9,2008-09-16,2008-04-30,https://www.sec.gov/Archives/edgar/data/1923/0...,This Amendment No. 1 on Form 10-K/A hereby ame...,Errors or Mistakes


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=data)

https://docs.google.com/spreadsheets/d/1YAb__Ec3v1EoeHDThMwQHT1T2rzoqrSQjx5XgJM4MVI#gid=0


/usr/local/lib/python3.10/dist-packages/google/colab/sheets.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return frame.applymap(_clean_val).replace({np.nan: None})


In [ ]:
text_sample = """10-K/A 1 a2213952z10-ka.htm 10-K/A


UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
WASHINGTON, D. C. 20549



FORM 10-K/A
Amendment No. 1

(MARK ONE)
ý	 	ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF
THE SECURITIES EXCHANGE ACT OF 1934
OR
o	 	TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF
THE SECURITIES EXCHANGE ACT OF 1934


For the fiscal year ended December 31, 2012	 	Commission file number 1-2189
Abbott Laboratories

An Illinois Corporation	 	36-0698440
100 Abbott Park Road
Abbott Park, Illinois 60064-6400	 	(I.R.S. employer identification number)
(847) 937-6100
(telephone number)
Securities Registered Pursuant to Section 12(b) of the Act:


Title of Each Class
 	Name of Each Exchange on Which Registered

Common Shares, Without Par Value
  	 	New York Stock Exchange
Chicago Stock Exchange

Indicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act.

Yes ý            No o

Indicate by check mark if the registrant is not required to file reports pursuant to Section 13 or 15(d) of the Act.

Yes o            No ý

Indicate by check mark whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 during the preceding 12 months (or for such shorter period that the registrant was required to file such reports), and (2) has been subject to such filing requirements for the past 90 days.

Yes ý            No o

Indicate by check mark whether the registrant has submitted electronically and posted on its corporate Web site, if any, every Interactive Data File required to be submitted and posted pursuant to Rule 405 of Regulation S-T during the preceding 12 months (or for such shorter period that the registrant was required to submit and post such files).

Yes ý            No o

Indicate by check mark if disclosure of delinquent filers pursuant to Item 405 of Regulation S-K is not contained herein, and will not be contained, to the best of registrant's knowledge, in definitive proxy or information statements incorporated by reference in Part III of this Form 10-K or any amendment to this Form 10-K. o

Indicate by check mark whether the registrant is a large accelerated filer, an accelerated filer, a non-accelerated filer, or a smaller reporting company. See the definitions of "large accelerated filer," "accelerated filer" and "smaller reporting company" in Rule 12b-2 of the Exchange Act.

Large Accelerated Filer ý	 	Accelerated Filer o	 	Non-accelerated Filer o	 	Smaller Reporting Company o
Indicate by check mark whether the registrant is a shell company (as defined in Rule 12b-2 of the Act).

Yes o            No ý

The aggregate market value of the 1,520,159,513 shares of voting stock held by nonaffiliates of the registrant, computed by reference to the closing price as reported on the New York Stock Exchange, as of the last business day of Abbott Laboratories' most recently completed second fiscal quarter (June 30, 2012), was $98,004,683,803. Abbott has no non-voting common equity.

Number of common shares outstanding as of January 31, 2013: 1,570,677,029

DOCUMENTS INCORPORATED BY REFERENCE

Portions of the 2013 Abbott Laboratories Proxy Statement are incorporated by reference into Part III. The Proxy Statement will be filed on or about March 15, 2013.




Explanatory Note

        This Form 10-K/A is being filed to correct a drafting error made by Deloitte & Touche LLP (Deloitte) in its Report of Independent Registered Public Accounting Firm which previously referred to "auditing standards" in the report on the audits of the consolidated financial statements. Deloitte has revised its report to appropriately state that their audits were conducted in accordance with "the standards" of the Public Company Accounting Oversight Board (United States).

        For the convenience of the reader, this Form 10-K/A sets forth the entire 2012 Form 10-K. However, this Form 10-K/A amends and restates only the Report of Independent Registered Public Accounting Firm in Item 8 of the 2012 Form 10-K to give effect to the change discussed above. The aforementioned changes have no effect on the other parts of the 2012 Form 10-K, the financial position of the Company as of December 31, 2012, 2011, and 2010, or the results of its operations and its cash flows for the years then ended.


PART I

ITEM 1.    BUSINESS

GENERAL DEVELOPMENT OF BUSINESS

        Abbott Laboratories is an Illinois corporation, incorporated in 1900. Abbott's* principal business is the discovery, development, manufacture, and sale of a broad and diversified line of health care products.


FINANCIAL INFORMATION RELATING TO INDUSTRY SEGMENTS, GEOGRAPHIC AREAS, AND CLASSES OF SIMILAR PRODUCTS

        Incorporated herein by reference is Note 6 entitled "Segment and Geographic Area Information" of the Notes to Consolidated Financial Statements included under Item 8, "Financial Statements and Supplementary Data" and the sales information related to HUMIRA® included in "Financial Review."


NARRATIVE DESCRIPTION OF BUSINESS

        Through December 31, 2012, Abbott had five reportable revenue segments: Proprietary Pharmaceutical Products, Established Pharmaceutical Products, Diagnostic Products, Nutritional Products, and Vascular Products.

        On January 1, 2013, Abbott completed the separation of its research-based pharmaceuticals business through the distribution of the issued and outstanding common stock of AbbVie Inc. (AbbVie) to Abbott's shareholders. AbbVie was formed to hold Abbott's research-based pharmaceuticals business and, as a result of the distribution, is now an independent public company trading under the symbol "ABBV" on the New York Stock Exchange.



*
As used throughout the text of this report on Form 10-K, the term "Abbott" refers to Abbott Laboratories, an Illinois corporation, or Abbott Laboratories and its consolidated subsidiaries, as the context requires.
1


Proprietary Pharmaceutical Products

        These products include a broad line of adult and pediatric pharmaceuticals manufactured, marketed, and sold worldwide (except as noted) and are generally sold directly to wholesalers, distributors, government agencies, health care facilities, specialty pharmacies, and independent retailers from distribution centers and public warehouses. Outside the United States, sales are made either directly to customers or through distributors, depending on the market served. Certain products are co-marketed or co-promoted with other companies. As a result of the separation of Abbott's research-based pharmaceuticals business, beginning in 2013, Abbott will no longer have a Proprietary Pharmaceutical Products segment.

        The principal products included in the Proprietary Pharmaceutical Products segment are:

•
HUMIRA®, for the treatment of rheumatoid arthritis, psoriatic arthritis, ankylosing spondylitis, psoriasis, juvenile idiopathic arthritis, and Crohn's disease as well as ulcerative colitis in the United States and European Union and axial spondyloarthritis and pediatric Crohn's disease in the European Union;

•
Kaletra®, also marketed as Aluvia®, and Norvir® for the treatment of HIV infection;

•
Lupron®, also marketed as Lucrin®, used for the palliative treatment of advanced prostate cancer, treatment of endometriosis and central precocious puberty, and for the preoperative treatment of patients with anemia caused by uterine fibroids;

•
Synagis®, for the prevention of respiratory syncytial virus (RSV);

•
AndroGel®, for the treatment of adult males who have low testosterone (marketed and sold in the United States);

•
the anesthesia product sevoflurane (sold under the trademarks Ultane® and Sevorane®);

•
Zemplar®, for the prevention and treatment of secondary hyperparathyroidism associated with Stage 3, 4, or 5 chronic kidney disease;

•
Synthroid®, for the treatment of hypothyroidism (marketed and sold in the United States);

•
Creon®, for the treatment of pancreatic exocrine insufficiency associated with several underlying conditions, including cystic fibrosis and chronic pancreatitis (marketed and sold in the United States); and

•
TriCor®, Trilipix®, Simcor®, and Niaspan®, for the treatment of dyslipidemia (marketed and sold in the United States).
        The Proprietary Pharmaceutical Products segment directs its primary marketing efforts toward securing the prescription, or recommendation, of its pharmaceutical products by physicians. Managed care providers, market access organizations (for example, health maintenance organizations and pharmacy benefit managers) and national and regional governments and agencies (for example, the United States Department of Veterans Affairs and the United States Department of Defense) are also important customers.

        Competition in the Proprietary Pharmaceutical Products segment is generally from other health care and pharmaceutical companies. The search for technological innovations in pharmaceutical products is a significant aspect of competition in this segment. The introduction of new products by competitors and changes in medical practices and procedures can result in product obsolescence in the Proprietary Pharmaceutical Products segment. Price can also be a factor. In addition, the substitution of generic drugs for the brand prescribed has increased competitive pressures on pharmaceutical products that do not have patent protection.

2

Established Pharmaceutical Products

        These products include a broad line of branded generic pharmaceuticals manufactured worldwide and marketed and sold outside the United States, and are generally sold directly to wholesalers, distributors, government agencies, health care facilities, specialty pharmacies, and independent retailers from Abbott-owned distribution centers and public warehouses, depending on the market served. Certain products are co-marketed or co-promoted with other companies.

        The principal products included in the Established Pharmaceutical Products segment are:

•
Creon®, for the treatment of pancreatic exocrine insufficiency associated with several underlying conditions, including cystic fibrosis and chronic pancreatitis (marketed and sold outside the United States);

•
the anti-infective clarithromycin (sold under the trademarks Biaxin®, Klacid®, and Klaricid®);

•
Influvac®, an influenza vaccine available during flu season;

•
Serc®, for the treatment of Ménière's disease and vestibular vertigo;

•
Brufen®, for the treatment of pain, fever and inflammation;

•
Synthroid®, for the treatment of hypothyroidism (marketed and sold outside the United States);

•
Duspatal® and Dicetel®, for the treatment of irritable bowel syndrome or biliary spasm;

•
Duphaston®, for the treatment of many different gynecological disorders;

•
Adomet®, Heptral®, Transmetil®, Samyr®, and Donamet®, for the treatment of intrahepatic cholestasis (associated with liver disease) or depressive symptoms;

•
Duphalac®, for regulation of the physiological rhythm of the colon;

•
Lipanthyl® and TriCor®, for the treatment of dyslipidemia (marketed and sold outside the United States); and

•
Teveten® and Teveten® Plus, for the treatment of essential hypertension, and Physiotens®, for the treatment of hypertension.
        The Established Pharmaceutical Products segment directs its primary marketing efforts toward securing the prescription, or recommendation, of Abbott's brand of products by physicians both in the primary care and secondary (hospital) care environment. Government agencies are also important customers.

        Competition in the Established Pharmaceutical Products segment is generally from other health care and pharmaceutical companies. Changes to government tenders and reimbursement schemes are significant factors with respect to pricing. In addition, the substitution of generic drugs for the brand prescribed and introduction of additional forms of already marketed established products by generic or branded competitors have increased competitive pressures.

Diagnostic Products

        These products include a broad line of diagnostic systems and tests manufactured, marketed, and sold worldwide to blood banks, hospitals, commercial laboratories, clinics, physicians' offices, government agencies, alternate-care testing sites, and plasma protein therapeutic companies. The segment's products are generally marketed and sold directly from Abbott-owned distribution centers, public warehouses and third-party distributors. Outside the United States, sales are made either directly to customers or through distributors, depending on the market served.

3

        The principal products included in the Diagnostic Products segment are:

•
immunoassay and clinical chemistry systems, including ARCHITECT® and ABBOTT PRISM®;

•
assays used for screening and/or diagnosis for drugs of abuse, cancer, therapeutic drug monitoring, fertility, physiological diseases, and infectious diseases such as hepatitis and HIV;

•
the m2000™, an instrument that automates the extraction, purification, and preparation of DNA and RNA from patient samples, and detects and measures infectious agents including HIV, HBV, HCV, HPV, and CT/NG;

•
the Vysis® product line of genomic-based tests, including the PathVysion® HER-2 DNA probe kit; the UroVysion® bladder cancer recurrence kit; and the Vysis ALK Break Apart FISH Probe Kit, the only FDA-approved companion diagnostic to Pfizer's approved non-small-cell lung cancer therapy XALKORI®;

•
informatics and automation solutions for use in the laboratory;

•
a full line of hematology systems and reagents known as the Cell-Dyn® series; and

•
the i-STAT® point-of-care diagnostic systems and tests for blood analysis.
        In addition, under a distribution agreement with Celera Group, the Diagnostic Products segment exclusively distributes certain Celera molecular diagnostic products, including the ViroSeq® HIV genotyping system and products used for the detection of mutations in the CFTR gene, which causes cystic fibrosis.

        The Diagnostic Products segment's products are subject to competition in technological innovation, price, convenience of use, service, instrument warranty provisions, product performance, long-term supply contracts, and product potential for overall cost-effectiveness and productivity gains. Some products in this segment can be subject to rapid product obsolescence or regulatory changes. Although Abbott has benefited from technological advantages of certain of its current products, these advantages may be reduced or eliminated as competitors introduce new products.

Nutritional Products

        These products include a broad line of pediatric and adult nutritional products manufactured, marketed, and sold worldwide. These products are generally marketed and sold directly to customers and to institutions, wholesalers, retailers, health care facilities, government agencies, and third-party distributors from Abbott-owned distribution centers or third-party distributors.

        The principal products included in the Nutritional Products segment are:

•
various forms of prepared infant formula and follow-on formula, including Similac®Advance®, Similac® Advance® with EarlyShield®, Similac®, Similac® with Iron, Similac Sensitive®, Similac Sensitive® RS, Similac Go&Grow®, Similac® NeoSure®, Similac® Organic, Similac Special Care®, Similac® Total Comfort®, Isomil® Advance®, Isomil®, Alimentum®, Gain®, and Grow®;

•
adult and other pediatric nutritional products, including Ensure®, Ensure Plus®, Ensure® Muscle Health, Ensure® (with Nutrivigor®), Glucerna®, Glucerna® Hunger Smart®, ProSure®, PediaSure®, PediaSure Sidekicks®, EleCare®, Juven®, Abound®, and Pedialyte®;

•
nutritional products used in enteral feeding in health care institutions, including Jevity®, Glucerna® 1.2 Cal, Glucerna® 1.5 Cal, Osmolite®, Oxepa®, Freego (Enteral Pump) and Freego® sets, and Nepro®; and

•
Zone Perfect® bars and the EAS® family of nutritional brands, including Myoplex® and AdvantEdge®.
4

        Primary marketing efforts for nutritional products are directed toward securing the recommendation of Abbott's brand of products by physicians or other health care professionals. In addition, certain nutritional products sold as Gain™, Grow™, PediaSure®, PediaSure Sidekicks®, Pedialyte®, Ensure®, Zone Perfect®, EAS®/Myoplex®, and Glucerna® are also promoted directly to the public by consumer marketing efforts in select markets.

        Competition for nutritional products in the segment is generally from other diversified consumer and health care manufacturers. Competitive factors include consumer advertising, formulation, packaging, scientific innovation, intellectual property, price, and availability of product forms. A significant aspect of competition is the search for ingredient innovations. The introduction of new products by competitors, changes in medical practices and procedures, and regulatory changes can result in product obsolescence. In addition, private label and local manufacturers' products may increase competitive pressure.

Vascular Products

        These products include a broad line of coronary, endovascular, vessel closure, and structural heart devices for the treatment of vascular disease manufactured, marketed and sold worldwide. The segment's products are generally marketed and sold directly to hospitals from Abbott-owned distribution centers and public warehouses. Outside the United States, sales are made either directly to customers or through distributors, depending on the market served.

        The principal products included in the Vascular Products segment are:

•
Xience Xpedition®, Xience Prime®, Xience nano™, and Xience V®, drug-eluting coronary stent systems developed on the Multi-Link Vision® platform;

•
Absorb®, a drug-eluting coronary bioresorbable vascular scaffold;

•
Multi-Link 8®, Multi-Link Vision® and Multi-Link Mini Vision®, coronary metallic stents;

•
TREK® and Voyager®, coronary balloon dilatation products;

•
Hi-Torque Balance Middleweight Elite® and ASAHI® coronary guidewires (licensed from Asahi Intecc Co., Ltd.);

•
StarClose® and Perclose® vessel closure devices;

•
Acculink®/Accunet® and Xact®/Emboshield NAV6®, carotid stent systems;

•
Armada® and Absolute Pro Peripheral® balloon dilatation products;

•
Herculink Elite Renal® and Omnilink Elite Iliac® stent systems; and

•
MitraClip®, a percutaneous valve repair system.
        The Vascular Products segment's products are subject to competition in technological innovation, price, convenience of use, service, product performance, long-term supply contracts, and product potential for overall cost-effectiveness and productivity gains. Some products in this segment can be subject to rapid product obsolescence or regulatory changes. Although Abbott has benefited from technological advantages of certain of its current products, these advantages may be reduced or eliminated as competitors introduce new products.

Other Products

        The principal products in Abbott's other businesses include blood glucose monitoring meters, test strips, data management software and accessories for people with diabetes, including the FreeStyle® product line, and medical devices for the eye, including cataract surgery, LASIK surgery, contact lens care products, and dry eye products. These products are mostly marketed worldwide and generally sold directly to wholesalers, government agencies, health care facilities, mail order pharmacies, and independent

5

retailers from Abbott-owned distribution centers and public warehouses. Some of these products are marketed and distributed through distributors. Blood glucose monitoring meters, contact lens care products, and dry eye products are also marketed and sold over-the-counter to consumers. These products are subject to competition in technological innovation, price, convenience of use, service, and product performance. Medical devices for the eye also can be subject to rapid product obsolescence or regulatory changes.


INFORMATION WITH RESPECT TO ABBOTT'S BUSINESS IN GENERAL

Sources and Availability of Raw Materials

        Abbott purchases, in the ordinary course of business, raw materials and supplies essential to Abbott's operations from numerous suppliers in the United States and abroad. There have been no recent significant availability problems or supply shortages.

Patents, Trademarks, and Licenses

        Abbott is aware of the desirability for patent and trademark protection for its products. Accordingly, where possible, patents and trademarks are sought and obtained for Abbott's products in the United States and all countries of major marketing interest to Abbott. Abbott owns and is licensed under a substantial number of patents and patent applications. Principal trademarks and the products they cover are discussed in the Narrative Description of Business on pages 1 through 5. These, and various patents which expire during the period 2013 to 2032, in the aggregate, are believed to be of material importance in the operation of Abbott's business. Abbott believes that, after the separation of AbbVie, no single patent, license, or trademark is material in relation to Abbott's business as a whole. In connection with the separation and distribution of AbbVie, Abbott contributed certain pharmaceutical related patents, licenses, and trademarks to AbbVie. Patent-related litigation is discussed in Legal Proceedings on pages 18 through 20.

Seasonal Aspects, Customers, Backlog, and Renegotiation

        There are no significant seasonal aspects to Abbott's business. Abbott has no single customer that, if the customer were lost, would have a material adverse effect on Abbott. Orders for Abbott's products are generally filled on a current basis, and order backlog is not material to Abbott's business. No material portion of Abbott's business is subject to renegotiation of profits or termination of contracts at the election of the government.

Research and Development

        Abbott spent approximately $4.3 billion in 2012, $4.1 billion in 2011, and $3.7 billion in 2010, on research to discover and develop new products and processes and to improve existing products and processes. The majority of research and development expenditures was concentrated on proprietary pharmaceutical products.

Environmental Matters

        Abbott believes that its operations comply in all material respects with applicable laws and regulations concerning environmental protection. Regulations under federal and state environmental laws impose stringent limitations on emissions and discharges to the environment from various manufacturing operations. Abbott's capital and operating expenditures for pollution control in 2012 were approximately $12 million and $63 million, respectively. After the separation of AbbVie, capital and operating expenditures for pollution control in 2013 are estimated to be $10 million and $53 million, respectively.

        Abbott has been identified as one of many potentially responsible parties in investigations and/or remediations at several locations in the United States, including Puerto Rico, under the Comprehensive

6

Environmental Response, Compensation, and Liability Act, commonly known as Superfund. Abbott is also engaged in remediation at several other sites, some of which are owned by Abbott, in cooperation with the Environmental Protection Agency (EPA) or similar agencies. While it is not feasible to predict with certainty the final costs related to those investigations and remediation activities, Abbott believes that such costs, together with other expenditures to maintain compliance with applicable laws and regulations concerning environmental protection, should not have a material adverse effect on Abbott's financial position, cash flows, or results of operations.

Employees

        Abbott employed approximately 91,000 persons as of December 31, 2012. Approximately 21,000 persons were transferred to AbbVie in connection with the separation.

Regulation

        The development, manufacture, marketing, sale, promotion, and distribution of Abbott's products are subject to comprehensive government regulation by the U.S. Food and Drug Administration and similar international regulatory agencies. Government regulation by various international, supranational, federal and state agencies, both domestic and international, addresses (among other matters) the development and approval to market Abbott's products, as well as the inspection of, and controls over, research and laboratory procedures, clinical investigations, product approvals and manufacturing, labeling, packaging, supply chains, marketing and promotion, pricing and reimbursement, sampling, distribution, quality control, post-market surveillance, record keeping, storage, and disposal practices. Abbott's international operations are also affected by trade regulations in many countries that limit the import of raw materials and finished products and by local and international laws and regulations that seek to prevent corruption and bribery in the marketplace (including the United States Foreign Corrupt Practices Act and the United Kingdom Bribery Act which provide among other things, guidance on corporate interactions with government officials). In addition, Abbott is subject to laws and regulations pertaining to health care fraud and abuse, including state and federal anti-kickback and false claims laws in the United States. Prescription drug, nutrition, and medical device manufacturers such as Abbott are also subject to taxes, as well as application, product, user, establishment, and other fees. Governmental agencies can also invalidate intellectual property rights and control the entrance of multi-source drugs for small molecule and generic biologic medicines.

        Compliance with these laws and regulations is costly and materially affects Abbott's business. Among other effects, health care regulations substantially increase the time, difficulty, and costs incurred in obtaining and maintaining approval to market newly developed and existing products. Abbott expects this regulatory environment will continue to require significant technical expertise and capital investment to ensure compliance. Failure to comply can delay the release of a new product or result in regulatory and enforcement actions, the seizure or recall of a product, the suspension or revocation of the authority necessary for a product's production and sale, and other civil or criminal sanctions, including fines and penalties.

        In addition to regulatory initiatives, Abbott's business can be affected by ongoing studies of the utilization, safety, efficacy, and outcomes of health care products and their components that are regularly conducted by industry participants, government agencies, and others. These studies can call into question the utilization, safety, and efficacy of previously marketed products. In some cases, these studies have resulted, and may in the future result, in the discontinuance of marketing of such products domestically or globally, and may give rise to claims for damages from persons who believe they have been injured as a result of their use.

        Access to human health care products continues to be a subject of investigation and action by governmental agencies, legislative bodies, and private organizations in the United States and other countries. A major focus is cost containment. Efforts to reduce health care costs are also being made in the

7

private sector, notably by health care payors and providers, which have instituted various cost reduction and containment measures. Abbott expects insurers and providers to continue attempts to reduce the cost of health care products. Many countries control the price of health care products directly or indirectly, through reimbursement, payment, pricing, coverage limitations, or compulsory licensing, and are adopting laws and rules to govern the introduction of biosimilar products. Domestic and foreign budgetary pressures may also heighten the scope and severity of pricing pressures on Abbott's products for the foreseeable future.

        Specifically, U.S. federal laws requiring pharmaceutical manufacturers to pay certain statutorily-prescribed rebates to state Medicaid programs on prescription drugs reimbursed under state Medicaid plans, and the efforts by states to seek additional rebates, affect Abbott's proprietary pharmaceutical business. Similarly, the Veterans Health Care Act of 1992 requires manufacturers to extend additional discounts on pharmaceutical products to various federal agencies, including the Department of Veterans Affairs, Department of Defense, Public Health Service entities and institutions, as well as certain other covered entities. The Veterans Health Care Act also established the 340B drug discount program, which requires pharmaceutical manufacturers to provide products at reduced prices to designated health care facilities.

        In the United States, most states also have generic substitution legislation requiring or permitting a dispensing pharmacist to substitute a different manufacturer's version of a pharmaceutical product for the one prescribed. In addition, the federal government follows a diagnosis-related group (DRG) payment system for certain institutional services provided under Medicare or Medicaid and has implemented a prospective payment system (PPS) for services delivered in hospital outpatient, nursing home, and home health settings. DRG and PPS entitle a health care facility to a fixed reimbursement based on the diagnosis and/or procedure rather than actual costs incurred in patient treatment, thereby increasing the incentive for the facility to limit or control expenditures for many health care products. End stage renal disease treatment is covered through a bundled payment that likewise creates incentives for providers to control costs. Medicare enters into contracts with private plans to negotiate prices for medicine delivered under Part D.

        Under the Patient Protection and Affordable Care Act and the Health Care and Education Reconciliation Act (together, the Affordable Care Act), Abbott pays a fee related to its pharmaceutical sales to government programs. In 2011, Abbott began providing a discount of 50 percent for branded prescription drugs sold to patients who fall into the Medicare Part D coverage gap, or "donut hole." The Affordable Care Act has also implemented the Medicare Shared Savings Program, which created accountable care organizations. The program provides incentives for health care providers to potentially reduce utilization of health care products by allowing accountable care organizations to share in Medicare savings. In 2013, as a result of the Affordable Care Act, manufacturers will begin paying an excise tax on sales of certain medical devices. Medicare is also implementing a competitive bidding system for durable medical equipment (including diabetes products), enteral nutrition products, and supplies.

        The Affordable Care Act also includes provisions known as the Physician Payments Sunshine Act, which require manufacturers of drugs, biologics, devices and medical supplies covered under Medicare and Medicaid starting in 2013 to record any transfers of value to physicians and teaching hospitals and to report this data beginning in 2014 to the Centers for Medicare and Medicaid Services for subsequent public disclosure. Similar reporting requirements have also been enacted on the state level domestically, and an increasing number of countries worldwide either have adopted or are considering similar laws requiring transparency of interactions with health care professionals. Failure to report appropriate data may result in civil or criminal fines and/or penalties.

        With the separation of AbbVie at the beginning of 2013, Abbott no longer sells pharmaceutical products in the U.S. and therefore is no longer subject to certain fees, rebates, and discounts applicable to U.S. pharmaceutical manufacturers.

8

        In the United States, governmental cost containment efforts also affect Abbott's nutrition business. Under regulations governing the federally funded Special Supplemental Nutrition Program for Women, Infants, and Children (WIC), all states must have a cost containment program for infant formula. As a result, through competitive bidding states obtain rebates from manufacturers of infant formula whose products are used in the program.

        Abbott expects debate to continue during 2013 at all government levels worldwide over the marketing, availability, method of delivery, and payment for health care products and services. Abbott believes that future legislation and regulation in the markets it serves could affect access to health care products and services, increase rebates, reduce prices or reimbursements or the rate of price increases for health care products and services, change health care delivery systems, create new fees and obligations for the pharmaceutical, nutrition, diagnostic, and medical device industries, or require additional reporting and disclosure. It is not possible to predict the extent to which Abbott or the health care industry in general might be affected by the matters discussed above.

        On April 19, 2012, the U.S. District Court for the Northern District of Illinois vacated a consent decree to which Abbott was previously a party. The consent decree required Abbott to ensure its diagnostics manufacturing processes in Lake County, Illinois conform to the U.S. Food and Drug Administration's (FDA) Quality System Regulation and restricted the sale in the United States of certain products in the Diagnostic Products segment. In 2003, the FDA concluded that those operations were in substantial conformity with the Quality System Regulation.


INTERNATIONAL OPERATIONS

        Abbott markets products worldwide through affiliates and distributors. Most of the products discussed in the preceding sections of this report are also sold outside the United States. In addition, certain products of a local nature and variations of product lines to meet local regulatory requirements and marketing preferences are manufactured and marketed to customers outside the United States. International operations are subject to certain additional risks inherent in conducting business outside the United States, including price and currency exchange controls, changes in currency exchange rates, limitations on foreign participation in local enterprises, expropriation, nationalization, and other governmental action.


INTERNET INFORMATION

        Copies of Abbott's Annual Report on Form 10-K, Quarterly Reports on Form 10-Q, Current Reports on Form 8-K, and amendments to those reports filed or furnished pursuant to Section 13(a) or 15(d) of the Securities Exchange Act of 1934 are available free of charge through Abbott's investor relations website (www.abbottinvestor.com) as soon as reasonably practicable after Abbott electronically files the material with, or furnishes it to, the Securities and Exchange Commission.

        Abbott's corporate governance guidelines, outline of directorship qualifications, code of business conduct and the charters of Abbott's audit committee, compensation committee, nominations and governance committee, and public policy committee are all available on Abbott's investor relations website (www.abbottinvestor.com).

9


ITEM 1A.    RISK FACTORS

        In addition to the other information in this report, the following risk factors should be considered before deciding to invest in any of Abbott's securities. Additional risks and uncertainties not presently known to Abbott, or risks Abbott currently considers immaterial, could also affect Abbott's actual results. Abbott's business, financial condition, results of operations, or prospects could be materially adversely affected by any of these risks.

Abbott may acquire other businesses, license rights to technologies or products, form alliances, or dispose of or spin-off businesses, which could cause it to incur significant expenses and could negatively affect profitability.

        Abbott may pursue acquisitions, technology licensing arrangements, and strategic alliances, or dispose of or spin-off some of its businesses, as part of its business strategy. Abbott may not complete these transactions in a timely manner, on a cost-effective basis, or at all, and may not realize the expected benefits. If Abbott is successful in making an acquisition, the products and technologies that are acquired may not be successful or may require significantly greater resources and investments than originally anticipated. Abbott may not be able to integrate acquisitions successfully into its existing business and could incur or assume significant debt and unknown or contingent liabilities. Abbott could also experience negative effects on its reported results of operations from acquisition or disposition-related charges, amortization of expenses related to intangibles and charges for impairment of long-term assets. These effects could cause a deterioration of Abbott's credit rating and result in increased borrowing costs and interest expense.

The expiration or loss of patent protection and licenses may affect Abbott's future revenues and operating income.

        Many of Abbott's businesses rely on patent and trademark and other intellectual property protection. Although most of the challenges to Abbott's intellectual property have come from other businesses, governments may also challenge intellectual property protections. To the extent Abbott's intellectual property is successfully challenged, invalidated, or circumvented or to the extent it does not allow Abbott to compete effectively, Abbott's business will suffer. To the extent that countries do not enforce Abbott's intellectual property rights or to the extent that countries require compulsory licensing of its intellectual property, Abbott's future revenues and operating income will be reduced. Abbott's patents and trademarks are described in greater detail in the section captioned "Patents, Trademarks, and Licenses," and litigation regarding these patents is described in the section captioned "Legal Proceedings."

Competitors' intellectual property may prevent Abbott from selling its products or have a material adverse effect on Abbott's future profitability and financial condition.

        Competitors may claim that an Abbott product infringes upon their intellectual property. Resolving an intellectual property infringement claim can be costly and time consuming and may require Abbott to enter into license agreements. Abbott cannot guarantee that it would be able to obtain license agreements on commercially reasonable terms. A successful claim of patent or other intellectual property infringement could subject Abbott to significant damages or an injunction preventing the manufacture, sale or use of affected Abbott products. Any of these events could have a material adverse effect on Abbott's profitability and financial condition.

Abbott is subject to cost containment efforts that could cause a reduction in future revenues and operating income.

        In the United States and other countries, Abbott's businesses have experienced downward pressure on product pricing. Cost containment efforts by governments and private organizations are described in

10

greater detail in the section captioned "Regulation." To the extent these cost containment efforts are not offset by greater patient access to health care or other factors, Abbott's future revenues and operating income will be reduced.

Abbott is subject to numerous governmental regulations and it can be costly to comply with these regulations and to develop compliant products and processes.

        Abbott's products are subject to rigorous regulation by the U.S. Food and Drug Administration, and numerous international, supranational, federal, and state authorities. The process of obtaining regulatory approvals to market a drug or medical device can be costly and time-consuming, and approvals might not be granted for future products, or additional indications or uses of existing products, on a timely basis, if at all. Delays in the receipt of, or failure to obtain approvals for, future products, or new indications and uses, could result in delayed realization of product revenues, reduction in revenues, and in substantial additional costs.

        In addition, no assurance can be given that Abbott will remain in compliance with applicable FDA and other regulatory requirements once clearance or approval has been obtained for a product. These requirements include, among other things, regulations regarding manufacturing practices, product labeling, and advertising and postmarketing reporting, including adverse event reports and field alerts due to manufacturing quality concerns. Many of Abbott's facilities and procedures and those of Abbott's suppliers are subject to ongoing regulation, including periodic inspection by the FDA and other regulatory authorities. Abbott must incur expense and spend time and effort to ensure compliance with these complex regulations. Possible regulatory actions for non-compliance could include warning letters, fines, damages, injunctions, civil penalties, recalls, seizures of Abbott's products, and criminal prosecution. These actions could result in, among other things, substantial modifications to Abbott's business practices and operations; refunds, recalls, or seizures of Abbott's products; a total or partial shutdown of production in one or more of Abbott's facilities while Abbott or Abbott's suppliers remedy the alleged violation; the inability to obtain future pre-market clearances or approvals; and withdrawals or suspensions of current products from the market. Any of these events could disrupt Abbott's business and have a material adverse effect on Abbott's revenues, profitability and financial condition.

Laws and regulations affecting government benefit programs could impose new obligations on Abbott, require Abbott to change its business practices, and restrict its operations in the future.

        Abbott's industry is also subject to various federal, state, and international laws and regulations pertaining to government benefit program reimbursement, price reporting and regulation, and health care fraud and abuse, including anti-kickback and false claims laws, and international and individual state laws relating to pricing and sales and marketing practices. Violations of these laws may be punishable by criminal and/or civil sanctions, including, in some instances, substantial fines, imprisonment, and exclusion from participation in federal and state health care programs, including Medicare, Medicaid, and Veterans Administration health programs. These laws and regulations are broad in scope and they are subject to evolving interpretations, which could require Abbott to incur substantial costs associated with compliance or to alter one or more of its sales or marketing practices. In addition, violations of these laws, or allegations of such violations, could disrupt Abbott's business and result in a material adverse effect on Abbott's revenues, profitability, and financial condition.

Changes in the health care regulatory environment may adversely affect Abbott's business.

        A number of the provisions of the Patient Protection and Affordable Care Act and the Health Care and Education Reconciliation Act of 2010 require further rulemaking action by governmental agencies to implement. The laws change access to health care products and services and create new fees for the pharmaceutical and medical device industries. Future rulemaking could increase rebates, reduce prices or

11

the rate of price increases for health care products and services, or require additional reporting and disclosure. Abbott cannot predict the timing or impact of any future rulemaking.

Abbott's research and development efforts may not succeed in developing commercially successful products and technologies, which may cause Abbott's revenue and profitability to decline.

        To remain competitive, Abbott must continue to launch new products and technologies. To accomplish this, Abbott commits substantial efforts, funds, and other resources to research and development. A high rate of failure is inherent in the research and development of new products and technologies. Abbott must make ongoing substantial expenditures without any assurance that its efforts will be commercially successful. Failure can occur at any point in the process, including after significant funds have been invested.

        Promising new product candidates may fail to reach the market or may only have limited commercial success because of efficacy or safety concerns, failure to achieve positive clinical outcomes, inability to obtain necessary regulatory approvals, limited scope of approved uses, excessive costs to manufacture, the failure to establish or maintain intellectual property rights, or infringement of the intellectual property rights of others. Even if Abbott successfully develops new products or enhancements or new generations of Abbott's existing products, they may be quickly rendered obsolete by changing customer preferences, changing industry standards, or competitors' innovations. Innovations may not be accepted quickly in the marketplace because of, among other things, entrenched patterns of clinical practice or uncertainty over third-party reimbursement. Abbott cannot state with certainty when or whether any of its products under development will be launched, whether it will be able to develop, license, or otherwise acquire compounds or products, or whether any products will be commercially successful. Failure to launch successful new products or new indications for existing products may cause Abbott's products to become obsolete, causing Abbott's revenues and operating results to suffer.

New products and technological advances by Abbott's competitors may negatively affect Abbott's results of operations.

        Abbott's products face intense competition from its competitors' products. Competitors' products may be safer, more effective, more effectively marketed or sold, or have lower prices or superior performance features than Abbott's products. Abbott cannot predict with certainty the timing or impact of the introduction of competitors' products.

The manufacture of many of Abbott's products is a highly exacting and complex process, and if Abbott or one of its suppliers encounters problems manufacturing products, Abbott's business could suffer.

        The manufacture of many of Abbott's products is a highly exacting and complex process, due in part to strict regulatory requirements. Problems may arise during manufacturing for a variety of reasons, including equipment malfunction, failure to follow specific protocols and procedures, problems with raw materials, natural disasters, and environmental factors. In addition, single suppliers are currently used for certain products and materials. If problems arise during the production of a batch of product, that batch of product may have to be discarded. This could, among other things, lead to increased costs, lost revenue, damage to customer relations, time and expense spent investigating the cause and, depending on the cause, similar losses with respect to other batches or products. If problems are not discovered before the product is released to the market, recall and product liability costs may also be incurred. To the extent Abbott or one of its suppliers experiences significant manufacturing problems, this could have a material adverse effect on Abbott's revenues and profitability.

12

Significant safety issues could arise for Abbott's products, which could have a material adverse effect on Abbott's revenues and financial condition.

        Health care products typically receive regulatory approval based on data obtained in controlled clinical trials of limited duration. Following regulatory approval, these products will be used over longer periods of time in many patients. Investigators may also conduct additional, and perhaps more extensive, studies. If new safety issues are reported, Abbott may be required to amend the conditions of use for a product. For example, Abbott may be required to provide additional warnings on a product's label or narrow its approved intended use, either of which could reduce the product's market acceptance. If serious safety issues arise with an Abbott product, sales of the product could be halted by Abbott or by regulatory authorities. Safety issues affecting suppliers' or competitors' products also may reduce the market acceptance of Abbott's products.

        In addition, in the ordinary course of business, Abbott is the subject of product liability claims and lawsuits alleging that its products or the products of other companies that Abbott promotes have resulted or could result in an unsafe condition for or injury to patients. Product liability claims and lawsuits and safety alerts or product recalls, regardless of their validity or ultimate outcome, may have a material adverse effect on Abbott's business and reputation and on Abbott's ability to attract and retain customers. Consequences may also include additional costs, a decrease in market share for the products, lower income or exposure to other claims. Product liability losses are self-insured. Product liability claims could have a material adverse effect on Abbott's profitability and financial condition.

Further deterioration in the economic position and credit quality of certain European countries may negatively affect Abbott's results of operations.

        If economic conditions in certain European countries, including Greece, Portugal, Italy, and Spain, continue to worsen, the time it takes to collect outstanding trade receivables may increase. Financial instability and fiscal deficits in these countries may result in additional austerity measures to reduce costs, including health care. At the same time, ongoing sovereign debt issues, including the impact of credit downgrades, could increase Abbott's collection risk given that a significant amount of Abbott's receivables in these countries are with governmental health care systems.

Abbott depends on sophisticated information technology systems to operate its business and a cyber attack or other breach of these systems could have a material adverse effect on Abbott's results of operations.

        Similar to other large multi-national companies, the size and complexity of Abbott's information technology systems makes them vulnerable to a cyber attack, malicious intrusion, breakdown, destruction, loss of data privacy, or other significant disruption. Abbott's systems have been and are expected to continue to be the target of malware and other cyber attacks. Abbott has invested in its systems and the protection of its data to reduce the risk of an invasion or interruption and monitors its systems on an ongoing basis for any current or potential threats. There can be no assurance that these measures and efforts will prevent future interruptions or breakdowns that could have a significant effect on Abbott's business.

Abbott may incur operational difficulties or be exposed to claims and liabilities as a result of the separation.

        AbbVie and Abbott entered into a separation and distribution agreement and various other agreements to govern the separation of AbbVie from Abbott and the relationship between the two companies going forward. Certain of these agreements provide for the performance of services by each company for the benefit of the other for a period of time. If AbbVie is unable to satisfy its obligations under these agreements, including its indemnification obligations, Abbott could incur operational difficulties or losses. These arrangements could also lead to disputes between Abbott and AbbVie over

13

Abbott's rights to certain shared property and rights and over the allocation of costs and revenues for products and operations.

        The separation and distribution agreement also provides for, among other things, indemnification obligations designed to make AbbVie financially responsible for substantially all liabilities that may exist relating to its business activities, whether incurred prior to or after AbbVie's separation from Abbott, as well as those obligations of Abbott assumed by AbbVie pursuant to the separation and distribution agreement. It is possible that a court would disregard the allocation agreed to between Abbott and AbbVie and require Abbott to assume responsibility for obligations allocated to AbbVie. Third parties could also seek to hold Abbott responsible for any of these liabilities or obligations. The indemnity rights Abbott has under the separation agreement may not be sufficient to protect Abbott. Even if Abbott is successful in obtaining indemnification, Abbott may have to bear losses temporarily. In addition, Abbott's indemnity obligations to AbbVie may be significant. These risks could negatively affect Abbott's results of operations.

There could be significant liability if the distribution of AbbVie common stock to Abbott shareholders is determined to be a taxable transaction.

        Abbott received a private letter ruling from the Internal Revenue Service (IRS) to the effect that, among other things, the separation and the distribution of AbbVie qualifies as a transaction that is tax-free for U.S. federal income tax purposes under Sections 355 and 368(a)(1)(D) of the Internal Revenue Code (the Code). In addition, Abbott received an opinion from outside tax counsel to the effect that the separation and distribution qualifies as a transaction that is described in Sections 355(a) and 368(a)(1)(D) of the Code. The ruling and the opinion rely on certain facts, assumptions, representations and undertakings from Abbott and AbbVie regarding the past and future conduct of the companies' respective businesses and other matters. If any of these facts, assumptions, representations or undertakings are incorrect or not satisfied, Abbott and its shareholders may not be able to rely on the ruling or the opinion of tax counsel and could be subject to significant tax liabilities. Notwithstanding the receipt by Abbott of the private letter ruling from the IRS and opinion of tax counsel, the IRS could determine on audit that the separation is taxable if it determines that any of these facts, assumptions, representations or undertakings are not correct or have been violated or if it disagrees with the conclusions in the opinion that are not covered by the private letter ruling, or for other reasons, including as a result of certain significant changes in the share ownership of Abbott or AbbVie after the separation. If the separation is determined to be taxable for U.S. federal income tax purposes, Abbott and its shareholders that are subject to U.S. federal income tax could incur significant U.S. federal income tax liabilities.

The international nature of Abbott's business subjects it to additional business risks that may cause its revenue and profitability to decline.

        Abbott's business is subject to risks associated with doing business internationally. Following the separation of AbbVie, sales outside of the United States are expected to make up approximately 70 percent of Abbott's net sales. The risks associated with Abbott's operations outside the United States include:

•
fluctuations in currency exchange rates;

•
changes in medical reimbursement policies and programs;

•
multiple regulatory requirements that are subject to change and that could restrict Abbott's ability to manufacture, market, and sell its products;

•
differing local product preferences and product requirements;

•
trade protection measures and import or export licensing requirements;

•
difficulty in establishing, staffing, and managing operations;
14

•
differing labor regulations;

•
potentially negative consequences from changes in or interpretations of tax laws;

•
political and economic instability, including sovereign debt issues;

•
price and currency exchange controls, limitations on participation in local enterprises, expropriation, nationalization, and other governmental action;

•
inflation, recession and fluctuations in interest rates;

•
compulsory licensing or diminished protection of intellectual property; and

•
potential penalties or other adverse consequences for violations of anti-corruption, anti-bribery and other similar laws and regulations, including the Foreign Corrupt Practices Act and the U.K. Bribery Act.
Events contemplated by these risks may, individually or in the aggregate, have a material adverse effect on Abbott's revenues and profitability.

Other factors can have a material adverse effect on Abbott's future profitability and financial condition.

        Many other factors can affect Abbott's profitability and its financial condition, including:

•
changes in or interpretations of laws and regulations, including changes in accounting standards, taxation requirements, product marketing application standards, product labeling, source, and use laws, and environmental laws;

•
differences between the fair value measurement of assets and liabilities and their actual value, particularly for pensions, retiree health care, stock compensation, intangibles, and goodwill; and for contingent liabilities such as litigation, the absence of a recorded amount, or an amount recorded at the minimum, compared to the actual amount;

•
changes in the rate of inflation (including the cost of raw materials, commodities, and supplies), interest rates, market value of Abbott's equity investments, and the performance of investments held by Abbott or Abbott's employee benefit trusts;

•
changes in the creditworthiness of counterparties that transact business with or provide services to Abbott or Abbott's employee benefit trusts;

•
changes in business, economic, and political conditions, including: war, political instability, terrorist attacks, the threat of future terrorist activity and related military action; natural disasters; the cost and availability of insurance due to any of the foregoing events; labor disputes, strikes, slow-downs, or other forms of labor or union activity; and pressure from third-party interest groups;

•
changes in Abbott's business units and investments and changes in the relative and absolute contribution of each to earnings and cash flow resulting from evolving business strategies, changing product mix, changes in tax laws or tax rates both in the U.S. and abroad and opportunities existing now or in the future;

•
changes in the buying patterns of a major distributor, retailer, or wholesale customer resulting from buyer purchasing decisions, pricing, seasonality, or other factors, or other problems with licensors, suppliers, distributors, and business partners;

•
changes in credit markets impacting Abbott's ability to obtain financing for its business operations; and
15

•
legal difficulties, any of which could preclude or delay commercialization of products or adversely affect profitability, including claims asserting statutory or regulatory violations, and adverse litigation decisions.

CAUTIONARY STATEMENT REGARDING FORWARD-LOOKING STATEMENTS

        This Form 10-K contains forward-looking statements that are based on management's current expectations, estimates, and projections. Words such as "expects," "anticipates," "intends," "plans," "believes," "seeks," "estimates," "forecasts," variations of these words, and similar expressions are intended to identify these forward-looking statements. Certain factors, including but not limited to those identified under "Item 1A. Risk Factors" of this Form 10-K, may cause actual results to differ materially from current expectations, estimates, projections, forecasts, and from past results. No assurance can be made that any expectation, estimate, or projection contained in a forward-looking statement will be achieved or will not be affected by the factors cited above or other future events. Abbott undertakes no obligation to release publicly any revisions to forward-looking statements as the result of subsequent events or developments, except as required by law.

ITEM 1B.    UNRESOLVED STAFF COMMENTS

        None.

16


ITEM 2.    PROPERTIES

        Abbott's corporate offices are located at 100 Abbott Park Road, Abbott Park, Illinois 60064. The locations of Abbott's principal plants, as of December 31, 2012, are listed below.

Location	 	Segments of Products Produced
Abbott Park, Illinois	 	Proprietary Pharmaceutical and Diagnostic Products
Alajuela, Costa Rica	 	Vascular Products
Alcobendas, Spain	 	Non-Reportable
Altavista, Virginia	 	Nutritional Products
Anasco, Puerto Rico*	 	Non-Reportable
Baddi, India	 	Established Pharmaceutical Products
Barceloneta, Puerto Rico**	 	Established Pharmaceutical, Proprietary Pharmaceutical, Diagnostic and Vascular Products
Branch Beringen, Switzerland*	 	Vascular Products
Brockville, Canada	 	Nutritional Products
Campoverde di Aprilia, Italy**	 	Established Pharmaceutical and Proprietary Pharmaceutical Products
Casa Grande, Arizona	 	Nutritional Products
Chatillon, France	 	Established Pharmaceutical Products
Clonmel, Ireland	 	Vascular Products
Columbus, Ohio	 	Nutritional Products
Cootehill, Ireland	 	Nutritional Products
Cork, Ireland**	 	Proprietary Pharmaceutical Products
Des Plaines, Illinois	 	Diagnostic Products
Donegal, Ireland	 	Non-Reportable
Fairfield, California*	 	Nutritional Products
Granada, Spain	 	Nutritional Products
Hangzhou, China	 	Non-Reportable
Irving, Texas	 	Diagnostic Products
Jayuya, Puerto Rico**	 	Proprietary Pharmaceutical Products
Karachi, Pakistan	 	Established Pharmaceutical Products
Katsuyama, Japan	 	Established Pharmaceutical Products
Longford, Ireland	 	Diagnostic Products
Ludwigshafen, Germany**	 	Established Pharmaceutical and Proprietary Pharmaceutical Products
Menlo Park, California*	 	Vascular Products
Milpitas, California*	 	Non-Reportable
Murrieta, California	 	Vascular Products
Neustadt, Germany	 	Established Pharmaceutical Products
North Chicago, Illinois**	 	Proprietary Pharmaceutical Products
Olst, the Netherlands	 	Established Pharmaceutical Products
Ottawa, Ontario, Canada*	 	Diagnostic Products
Princeton, New Jersey*	 	Diagnostic Products
Redwood City, California*	 	Vascular Products
Rio de Janeiro, Brazil	 	Established Pharmaceutical Products
Santa Clara, California	 	Diagnostic Products
Singapore	 	Nutritional Products
Sligo, Ireland**	 	Proprietary Pharmaceutical, Nutritional and Diagnostic Products
Sturgis, Michigan	 	Nutritional Products
Temecula, California	 	Vascular Products
Tlalpan, Mexico	 	Established Pharmaceutical Products
Uppsala, Sweden	 	Non-Reportable
Weesp, the Netherlands	 	Established Pharmaceutical Products
Wiesbaden, Delkenheim, Germany	 	Diagnostic Products
Witney, Oxon, England	 	Non-Reportable
Worcester, Massachusetts**	 	Proprietary Pharmaceutical Products
Zwolle, the Netherlands	 	Nutritional Products
*
Leased property

**
Contributed to AbbVie in connection with the separation on January 1, 2013.
17

        In addition to the above, as of December 31, 2012, Abbott had manufacturing facilities in seven other locations in the United States, including Puerto Rico, one of which was contributed to AbbVie in connection with the separation. Outside the United States, Abbott had manufacturing facilities located in two other countries. Abbott's facilities are deemed suitable and provide adequate productive capacity.

        After the separation of AbbVie, Abbott's research and development facilities in the United States are primarily located in California, Illinois, New Jersey, and Ohio. Abbott also has research and development facilities in various other countries including Germany, Netherlands, Singapore, Spain, and the United Kingdom. Prior to the separation of AbbVie, research and development related to the Proprietary Pharmaceutical Products segment was carried out in various locations including California, Illinois and Massachusetts in the United States, Germany, and Japan.

        Except as noted, the corporate offices, and those principal plants in the United States listed above, are owned by Abbott or subsidiaries of Abbott. The remaining manufacturing plants and all other facilities are owned or leased by Abbott or subsidiaries of Abbott. There are no material encumbrances on the properties.

ITEM 3.    LEGAL PROCEEDINGS

        Abbott is involved in various claims, legal proceedings and investigations, including (as of January 31, 2013, except where noted below) those described below. While it is not feasible to predict the outcome of such pending claims, proceedings and investigations with certainty, management is of the opinion that their ultimate resolution should not have a material adverse effect on Abbott's financial position, cash flows, or results of operations.

        Seven shareholder derivative lawsuits are pending in a consolidated proceeding, In Re Abbott-Depakote Shareholder Derivative Litigation, in the United States District Court for the Northern District of Illinois against certain of Abbott's current and former directors and members of senior management. The lawsuits allege a breach of fiduciary duty in relation to certain business practices regarding the sales and marketing of Depakote. In each case, the plaintiffs request damages nominally on behalf of Abbott, attorneys' fees, and other forms of relief. In Re Abbott-Depakote Shareholder Derivative Litigation includes: Chester County Employees' Retirement Fund, Warren Pinchuck and Roy Sapir, and Jacksonville Police & Fire Pension Fund, all filed in November 2011; Louisiana Municipal Police Employees' Retirement System and Pipefitters Local Union 537 Pension Fund, both filed in December 2011; Public School Retirement System of the School District of Kansas City, Missouri, filed in January 2012; and Pipefitters Local Union No. 120 Pension Fund, filed in April 2012. Two lawsuits were previously pending in the Circuit Court for the Nineteenth Judicial Circuit, Lake County, Illinois: Patricia Goodman, filed in November 2011, and William Bojan, filed in December 2011. These two cases were dismissed in March 2012.

        In January 2008, Cordis Corporation and Wyeth filed suit against Abbott in the United States District Court for the District of New Jersey alleging that the Xience V stent infringes three patents and seeking an injunction, damages, and a determination of willful infringement. In January 2012, the court issued an order invalidating the plaintiff's patents and dismissing the case against Abbott. Cordis and Wyeth withdrew one patent from the case and in June 2012 appealed the court's order invalidating the remaining two patents. In a separate action, in September 2009, Wyeth, Cordis Corporation, and Cordis LLC filed suit against Abbott in the United States District Court for the District of New Jersey alleging the Xience V (and later the Xience Prime) stent systems infringe an additional patent, and in August 2010 the plaintiffs amended their lawsuit to add a second related patent to this case. The plaintiffs in this case seek an injunction and damages. Abbott denies all substantive allegations in this case. In February 2012, the court stayed the litigation pending the completion of inter partes reexamination of the two patents at issue by the United States Patent and Trademark Office and any resulting appeals.

        In December 2008, Medinol Limited (Medinol) sued Abbott in the District Court of The Hague, the Netherlands, and the Regional Court in Dusseldorf, Germany asserting that Abbott's Vision and Xience V

18

stents infringe one of Medinol's European stent design patents. Medinol subsequently accused Abbott's Multi-Link 8 and Xience Prime stents of infringement of the same European stent design patent. In December 2009, the Dutch court found that Abbott's stents do not infringe Medinol's European patent, but did not rule on the patent's validity. Medinol appealed the Dutch court's finding, and in October 2012, the Dutch appeals court affirmed the lower court finding of noninfringement. Also in December 2008, Medinol asserted in the Dusseldorf court that Abbott's Vision, Xience V, Penta, Xience Prime, Multi-Link 8, and Zeta stents infringe two Medinol German stent design patents and one Medinol German stent design utility model. Abbott initiated an action in the German Federal Patent Court seeking a declaration that Medinol's patents are invalid. In March 2010, the Dusseldorf court, which does not assess the validity of patents, found that Abbott's stents do not infringe Medinol's European patent, but that they do infringe Medinol's two German stent design patents. Medinol has appealed the Dusseldorf court's non-infringement decision with respect to the European patent and Abbott has appealed the infringement decisions with respect to the two German patents. In November and December 2010, the German Federal Patent Court held two invalidity hearings on the European patent and two German patents being asserted by Medinol in Germany, and in January 2011, found all such Medinol patents invalid. However, after allowing Medinol to modify the claims for one of its German patents, the German Federal Patent Court concluded that the modified claims of that patent were valid. The question of validity of these patents has been appealed to the Federal Supreme Court. In June 2011, Medinol asserted a second, related European patent against Abbott in the Dusseldorf court. The question of validity of this second European patent, and of the German stent design utility model, is subject to continued lower court proceedings. The infringement proceedings for all patents at issue are stayed, pending further determinations relating to the question of validity. Medinol seeks damages and injunctions in The Netherlands and seeks damages in Germany. Abbott denies all substantive allegations in each case.

        The United States Department of Justice, through the United States Attorney's Office for the Eastern District of Tennessee, and the Texas State Attorney General are investigating the sales and marketing activities of Abbott's biliary stent products. Investigations are also ongoing relating to the sales and marketing activities for Abbott's carotid and coronary stents and stent related products by the United States Attorney's Office for the Eastern District of Tennessee, as previously mentioned, and the United States Attorney's Office for the District of Maryland. The government is seeking to determine whether any of these activities violated civil and/or criminal laws, including the Federal False Claims Act, the Food and Drug Cosmetic Act, and the Anti-Kickback Statute in connection with Medicare and/or Medicaid reimbursement paid to third parties.

        As previously reported, Abbott was seeking to enforce its patent rights relating to fenofibrate tablets (a drug Abbott sells under the trademark TriCor®). In a case filed in the United States District Court for the District of New Jersey in August 2011, Abbott and the patent owner, Laboratoires Fournier, S.A. (Fournier), alleged infringement of three patents and sought injunctive relief against Mylan Pharmaceuticals Inc. and Mylan, Inc. (Mylan). In a related case where Abbott was involved as a result of its acquisition of Fournier Laboratories Ireland Ltd. (Fournier Ireland), Abbott sought to enforce additional rights relating to fenofibrate tablets. In a case filed in the United States District Court for the District of New Jersey in August 2011, Abbott's subsidiary, Fournier Ireland, and joint patent owner, Alkermes Pharma Ireland Limited, alleged infringement of two jointly-owned patents and sought injunctive relief against Mylan. In the fourth quarter of 2012, this case was settled and dismissed without prejudice.

        On January 1, 2013, Abbott completed the separation and distribution of AbbVie. As previously reported, Abbott was a party to a Corporate Integrity Agreement (CIA) with the Office of Inspector General for the U.S. Department of Health and Human Services relating to Abbott's United States pharmaceuticals business. The CIA required enhancements to certain compliance procedures and contained numerous reporting and monitoring obligations. Abbott also submitted to a term of probation that was initially set at 5 years, and was shortened to 3 years upon the separation of AbbVie. The

19

conditions of probation included certain reporting requirements, maintenance of certain compliance measures, certifications of the CEO and board of directors, and other conditions. The obligations under the CIA and the conditions of probation became effective in October 2012 and transferred to and became fully binding on AbbVie upon the separation. Abbott is no longer a party to the CIA or subject to probation.

        Pursuant to the separation and distribution agreement, AbbVie is responsible for certain investigations, claims and litigation matters relating to Abbott's former research-based pharmaceuticals business. In some cases, AbbVie has been substituted for Abbott, and Abbott is no longer a party to such investigations, claims and litigation matters. In addition, AbbVie has assumed the liabilities associated with, and has agreed to indemnify Abbott for any damages incurred in connection with, certain investigations, claims and litigation matters, including the following:

        Several cases are pending that generally allege that Abbott and numerous other pharmaceutical companies reported false pricing information in connection with certain drugs that are reimbursable under Medicare and Medicaid. These cases, brought by state Attorneys General, generally seek monetary damages and/or injunctive relief and attorneys' fees. The following cases are pending in state courts: Commonwealth of Kentucky, filed in September 2003 in the Circuit Court of Franklin County, Kentucky; State of Wisconsin, filed in June 2004 in the Circuit Court of Dane County, Wisconsin; State of Illinois, filed in February 2005 in the Circuit Court of Cook County, Illinois; and State of Louisiana, filed in October 2010 in the Nineteenth Judicial District, Parish of Baton Rouge, Louisiana. The previously reported federal court cases consolidated under the Multi District Litigation Rules as In Re Pharmaceutical Industry Average Wholesale Price Litigation, MDL 1456, have been settled, and the following previously reported cases have been settled in principle: State of Alaska, filed in October 2006 in the Superior Court for the Third Judicial District in Anchorage, Alaska; State of Idaho, filed in January 2007 in the District Court of the Fourth District in Ada County, Idaho; and State of Utah, filed in November 2007 in the Third Judicial District in Salt Lake County, Utah.

        In a case filed in the United States District Court for the District of Delaware in April 2012, Abbott is seeking to enforce its patent rights relating to ritonavir tablets (a drug Abbott sells under the trademark Norvir®). Abbott alleges that Roxane Laboratories, Inc.'s (Roxane) proposed generic product infringes five Abbott patents and seeks declaratory and injunctive relief. Also in April 2012, Roxane filed a declaratory judgment action in the United States District Court for the Southern District of Ohio alleging that two Abbott patents are invalid and not infringed by Roxane's proposed generic product. In November 2012, Roxane filed an amended declaratory judgment complaint in the United States District Court for the Southern District of Ohio, naming AbbVie as a defendant along with Abbott.

ITEM 4.    MINE SAFETY DISCLOSURES

        Not applicable.

20



EXECUTIVE OFFICERS OF THE REGISTRANT

        Executive officers of Abbott are elected annually by the board of directors. All other officers are elected by the board or appointed by the chairman of the board. All officers are either elected at the first meeting of the board of directors held after the annual shareholder meeting or appointed by the chairman after that board meeting. Each officer holds office until a successor has been duly elected or appointed and qualified or until the officer's death, resignation, or removal. Vacancies may be filled at any time by the board. Any officer may be removed by the board of directors when, in its judgment, removal would serve the best interests of Abbott. Any officer appointed by the chairman of the board may be removed by the chairman whenever, in the chairman's judgment, removal would serve the best interests of Abbott. A vacancy in any office appointed by the chairman of the board may be filled by the chairman.

        Abbott's executive officers, their ages as of February 15, 2013, and the dates of their first election as officers of Abbott are listed below. The executive officers' principal occupations and employment for the past five years and the year of appointment to the earliest reported office are also shown. Unless otherwise stated, employment was by Abbott. There are no family relationships between any corporate officers or directors.

Miles D. White, 57

1999 to present — Chairman of the Board and Chief Executive Officer, and Director.

Elected Corporate Officer — 1993.

Hubert L. Allen, 47

2013 to present — Executive Vice President, General Counsel and Secretary.

2010 to 2012 — Divisional Vice President and Associate General Counsel, Established Pharmaceuticals.

2009 to 2010 — Divisional Vice President and Associate General Counsel, Proprietary Pharmaceuticals.

2007 to 2009 — Section Head, Legal, Abbott Nutrition.

Elected Corporate Officer — 2012.

Richard W. Ashley, 69

2004 to present — Executive Vice President, Corporate Development.

Elected Corporate Officer — 2004.

Brian J. Blaser, 48

2012 to present — Executive Vice President, Diagnostics Products.

2010 to 2012 — Senior Vice President, Diagnostics.

2008 to 2010 — Vice President, Diagnostics, Operations.

2008 — Divisional Vice President, Global Operations.

2007 to 2008 — Divisional Vice President, Manufacturing.

Elected Corporate Officer — 2008.

John M. Capek, 51

2007 to present — Executive Vice President, Medical Devices.

21

Elected Corporate Officer — 2006.

Thomas C. Freyman, 58

2004 to present — Executive Vice President, Finance and Chief Financial Officer.

Elected Corporate Officer — 1991.

John C. Landgraf, 60*

2011 to present — Executive Vice President, Nutritional Products.

2008 to 2010 — Senior Vice President, Pharmaceuticals, Manufacturing and Supply.

2004 to 2008 — Senior Vice President, Global Pharmaceutical Manufacturing and Supply.

Elected Corporate Officer — 2000.

Michael J. Warmuth, 50

2012 to present — Executive Vice President, Established Pharmaceuticals.

2010 to 2012 — Senior Vice President, Established Products, Pharmaceutical Products Group.

2008 to 2010 — Senior Vice President, Diagnostics.

2008 — Vice President, Hematology Diagnostics.

2007 to 2008 — Vice President, Global Engineering Services.

Elected Corporate Officer — 2007.

A. David Forrest, 51

2010 to present — Senior Vice President, International Nutrition.

2007 to 2010 — Divisional Vice President, Europe and Canada.

Elected Corporate Officer — 2010.

Stephen R. Fussell, 55

2005 to present — Senior Vice President, Human Resources.

Elected Corporate Officer — 1999.

Paul K. Magill, 51

2012 to present — Senior Vice President, Chief Marketing Officer.

2009 to 2012 — Principal, Senior Expert, McKinsey and Company (a management consulting firm).

2008 to 2009 — Senior Partner, Monitor Group (a management consulting firm).

Elected Corporate Officer — 2012.

Heather L. Mason, 52

2008 to present — Senior Vice President, Diabetes Care.

2007 to 2008 — Vice President, Latin America Pharmaceuticals.

Elected Corporate Officer — 2001.

22

Murthy Simhambhatla, 47

2013 to present — Senior Vice President, Abbott Medical Optics.

2012 — Divisional Vice President and General Manager, Abbott Medical Optics.

2011 to 2012 — Divisional Vice President and General Manager, Ibis.

2008 to 2011 — General Manager, Australia and New Zealand.

2006 to 2008 — Divisional Vice President and General Manager, Drug Eluting Stents.

Elected Corporate Officer — 2012.

J. Scott White, 44

2010 to present — Senior Vice President, U.S. Nutrition.

2007 to 2009 — Division Vice President and Regional Director for Latin America, Abbott Nutrition International.

Elected Corporate Officer — 2009.

Greg W. Linder, 56**

2001 to present — Vice President and Controller.

Elected Corporate Officer — 1999.

*
Mr. Landgraf has announced that he will retire from Abbott, effective February 28, 2013.

**
Mr. Linder has announced that he will retire from Abbott, effective February 28, 2013.
23



PART II

ITEM 5.    MARKET FOR REGISTRANT'S COMMON EQUITY, RELATED STOCKHOLDER MATTERS AND ISSUER PURCHASES OF EQUITY SECURITIES

Principal Market

        The principal market for Abbott's common shares is the New York Stock Exchange. Shares are also listed on the Chicago Stock Exchange and traded on various regional and electronic exchanges. Outside the United States, Abbott's shares are listed on the London Stock Exchange and the SIX Swiss Exchange.


 	Market Price Per Share

 	2012	 	2011

 	high	 	low	 	high	 	low
First Quarter

 	$	61.49	 	$	53.96	 	$	49.45	 	$	45.07
Second Quarter

 	 	64.47	 	 	59.04	 	 	54.24	 	 	49.05
Third Quarter

 	 	70.41	 	 	63.51	 	 	53.60	 	 	46.29
Fourth Quarter

 	 	72.47	 	 	62.62	 	 	56.44	 	 	48.96
Shareholders

        There were 60,476 shareholders of record of Abbott common shares as of December 31, 2012.

Dividends

        Abbott declared quarterly dividends of $.51 per share on common shares in the first, second and third quarters of 2012. In the fourth quarter of 2012, Abbott declared a quarterly dividend of $.14 per share on common shares, reflecting the impact of the separation of AbbVie. On January 4, 2013, AbbVie reported that its board of directors had declared a quarterly dividend of $.40 per share of AbbVie common stock.

        Abbott declared quarterly dividends of $.48 per share on common shares in 2011.

        On January 1, 2013, Abbott completed the distribution of the issued and outstanding common stock of AbbVie to Abbott's shareholders. Abbott's shareholders of record as of the close of business on December 12, 2012, the record date for the distribution, received one share of AbbVie common stock for each Abbott common share held as of the record date. Abbott shareholders received cash in lieu of any fractional shares of AbbVie common stock.

        Abbott Laboratories is an Illinois High Impact Business (HIB) and is located in a federal Foreign Trade Sub-Zone (Sub-Zone 22F). Dividends may be eligible for a subtraction from base income for Illinois income tax purposes. If you have questions, please contact your tax advisor.

24

Issuer Purchases of Equity Securities

Period
 	(a) Total Number
of Shares
(or Units)
Purchased	 	(b) Average Price
Paid per Share
(or Unit)	 	(c) Total Number of
Shares (or Units)
Purchased as Part of
Publicly Announced
Plans or Programs	 	(d) Maximum Number (or
Approximate Dollar Value) of
Shares (or Units) that May
Yet Be Purchased Under the
Plans or Programs
October 1, 2012 — October 31, 2012

 	 	3,790,158	1	$	65.791	 	 	3,650,000	 	$	1,552,719,103	2
November 1, 2012 — November 30, 2012

 	 	6,217,576	1	$	64.860	 	 	6,173,386	 	$	1,152,271,705	2
December 1, 2012 — December 31, 2012

 	 	66,285	1	$	65.202	 	 	0	 	$	1,152,271,705	2
Total

 	 	10,074,019	1	$	65.213	 	 	9,823,386	 	$	1,152,271,705	2
1.
These shares include:

(i)
the shares deemed surrendered to Abbott to pay the exercise price in connection with the exercise of employee stock options — 140,158 in October, 34,190 in November, and 66,285 in December; and

(ii)
the shares purchased on the open market for the benefit of participants in the Abbott Laboratories, Limited Employee Stock Purchase Plan — 0 in October, 10,000 in November, and 0 in December.
These shares do not include the shares surrendered to Abbott to satisfy tax withholding obligations in connection with the vesting of restricted stock or restricted stock units.

2.
On October 13, 2008, Abbott announced that its board of directors approved the purchase of up to $5 billion of its common shares, from time to time.
25

ITEM 6.    SELECTED FINANCIAL DATA


 	Year ended December 31

 	2012	 	2011	 	2010	 	2009	 	2008

 	(dollars in millions, except per share data)

Net sales

 	$	39,873.9	 	$	38,851.3	 	$	35,166.7	 	$	30,764.7	 	$	29,527.6
Earnings from continuing operations

 	 	5,962.9	 	 	4,728.4	 	 	4,626.2	 	 	5,745.8	 	 	4,734.2
Net earnings

 	 	5,962.9	 	 	4,728.4	 	 	4,626.2	 	 	5,745.8	 	 	4,880.7
Basic earnings per common share from continuing operations

 	 	3.76	 	 	3.03	 	 	2.98	 	 	3.71	 	 	3.06
Basic earnings per common share

 	 	3.76	 	 	3.03	 	 	2.98	 	 	3.71	 	 	3.16
Diluted earnings per common share from continuing operations

 	 	3.72	 	 	3.01	 	 	2.96	 	 	3.69	 	 	3.03
Diluted earnings per common share

 	 	3.72	 	 	3.01	 	 	2.96	 	 	3.69	 	 	3.12
Total assets

 	 	67,234.9	 	 	60,276.9	 	 	60,573.9	 	 	52,581.6	 	 	42,419.2
Long-term debt

 	 	18,085.3	 	 	12,039.8	 	 	12,523.5	 	 	11,266.3	 	 	8,713.3
Cash dividends declared per common share

 	 	1.67	1	 	1.92	 	 	1.76	 	 	1.60	 	 	1.44
1.
The $1.67 dividend for 2012 reflects a quarterly dividend of $.14 per share in the fourth quarter of 2012, which contemplated the impact of the separation of AbbVie. On January 4, 2013, AbbVie reported that its board of directors had declared a quarterly dividend of $.40 per share of AbbVie common stock.
26


ITEM 7.    MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS

Financial Review

        Abbott's revenues are derived primarily from the sale of a broad line of health care products under short-term receivable arrangements. Patent protection and licenses, technological and performance features, and inclusion of Abbott's products under a contract or by a pharmacy benefit manager most impact which products are sold; price controls, competition and rebates most impact the net selling prices of products; and foreign currency translation impacts the measurement of net sales and costs. Abbott's primary products are nutritional products, prescription pharmaceuticals, diagnostic testing products and vascular products.

        In October 2011, Abbott announced a plan to separate into two publicly traded companies, one in diversified medical products and the other in research-based pharmaceuticals. To accomplish the separation, Abbott created a new company, AbbVie Inc. ("AbbVie") for its research-based pharmaceuticals business which consists primarily of Abbott's Proprietary Pharmaceutical Products segment. On January 1, 2013, Abbott distributed all of the outstanding shares of AbbVie to Abbott's shareholders. As a result of the distribution, AbbVie is now an independent company trading under the symbol "ABBV". Beginning in the first quarter of 2013, the historical results of the research-based pharmaceuticals business will be reflected in Abbott's consolidated financial statements as discontinued operations.

        Prior to the separation of AbbVie, sales in international markets were approximately 60 percent of consolidated net sales. Post-separation, sales outside the U.S. are expected to comprise approximately 70 percent of net sales.

        Continued robust growth of HUMIRA in a broad range of indications, the acquisitions of Solvay's pharmaceuticals business (Solvay Pharmaceuticals) and Piramal Healthcare Limited's Healthcare Solutions business, sales growth and margin improvement in the nutritional and diagnostics businesses, a government investigation of Abbott's sales and marketing activities related to Depakote, and the challenging economic and fiscal environment in many countries around the world have impacted Abbott's sales, costs and financial position over the last three years.

        Pharmaceutical research and development was focused over the last three years on therapeutic areas that included immunology, oncology, neuroscience, pain management, hepatitis C (HCV), chronic kidney disease and women's health. In addition, Abbott acquired the rights to various in-process pharmaceutical research and development projects including the development of second-generation oral antioxidant inflammation modulators and an oral, next-generation JAK1 inhibitor with the potential to treat multiple autoimmune diseases.

        In 2003, Abbott began the worldwide launch of HUMIRA for rheumatoid arthritis, followed by launches for six additional indications in the U.S. and eight additional indications in the European Union. HUMIRA's worldwide sales increased to $9.3 billion in 2012 compared to $7.9 billion in 2011, and $6.5 billion in 2010. Generic competition for TriCor began in the fourth quarter of 2012. Austerity measures implemented by several European countries reduced healthcare spending and affected pharmaceutical pricing over the last three years. The U.S. proprietary pharmaceuticals business was negatively affected by the 2010 U.S. health care reform legislation which resulted in rebate changes beginning in 2010 and the payment of an annual fee beginning in 2011.

        In February 2010, Abbott acquired Solvay Pharmaceuticals which provided Abbott with a large and complementary portfolio of pharmaceutical products and expanded Abbott's presence in key global emerging markets. The acquisition added approximately $3.1 billion to Abbott's 2010 net sales, primarily outside the U.S. In September 2010, Abbott completed the acquisition of Piramal's Healthcare Solutions business, propelling Abbott to market leadership in the Indian pharmaceutical market and further

27

accelerating the company's growth in emerging markets. Abbott recorded expense of approximately $262 million in 2012, $345 million in 2011 and $710 million in 2010 related to the integration of the Solvay business and restructuring plans to streamline operations, improve efficiencies and reduce costs primarily in certain Solvay sites and functions.

        In May 2012, Abbott reached resolution of all Depakote-related federal claims, Medicaid-related claims with 49 states and the District of Columbia, and consumer protection claims with 45 states and the District of Columbia. Abbott recorded charges related to this matter of $1.5 billion in 2011 and $100 million in 2012, of which approximately $1.6 billion was paid in 2012.

        In Abbott's worldwide nutritional products business, sales were positively impacted by demographics such as an aging population and an increasing rate of chronic disease in developed markets and the rise of a middle class in many emerging markets, as well as by numerous new product introductions that leveraged Abbott's strong brands. At the same time, manufacturing and distribution process changes and other cost reductions drove margin improvements across the business.

        In Abbott's worldwide diagnostics business, margin improvement continued to be a key focus and operating margins increased from 14.7 percent of sales in 2010 to 18.7 percent in 2012.

        Over the last three years, Abbott continued to build its Xience drug-eluting stent franchise with the receipt of approval to market Xience Xpedition in various countries, including U.S. approval in the fourth quarter of 2012 as well as the launches of Xience nano and Xience PRIME in the U.S. in 2011, and Xience PRIME and Xience V in Japan in April 2012 and January 2010, respectively. Xience, which includes Xience V, PRIME, nano and Xpedition, ended 2012 as the market-leading drug eluting stent globally. In 2011, the third party distributor of the Promus product began transitioning away from the product and that supply agreement ended in 2012. The effect of the winding down of the agreement will continue into the first quarter of 2013.

        In 2010, the U.S. government passed health care reform legislation which included an increase in Medicaid rebate rates and the extension of the rebate to drugs provided through Medicaid managed care organizations beginning in 2010. The legislation also imposed annual fees which pharmaceutical manufacturers began paying in 2011 and medical device companies will begin paying in 2013, as well as additional rebates related to the Medicare Part D "donut hole" beginning in 2011. In addition to a 2010 one-time charge of approximately $60 million to reduce deferred tax assets associated with retiree health care liabilities related to the Medicare Part D retiree drug subsidy, the legislation's negative impact on Abbott's performance grew from more than $200 million in 2010 to approximately $400 million per year in 2011 and 2012. The $400 million annual impact included approximately $100 million for the annual pharmaceutical manufacturer fee. This fee is not tax-deductible and is included in selling, general and administrative expenses. With the separation of AbbVie at the beginning of 2013, Abbott no longer sells pharmaceutical products in the U.S. and therefore is no longer subject to the annual pharmaceutical fee or the additional rebates. Beginning in 2013, Abbott will begin paying the 2.3 percent medical device tax under U.S. health care reform legislation. This tax will be included in selling, general and administrative expenses and the amount of the tax is not expected to be material.

        In the fourth quarter of 2012, Abbott extinguished $7.7 billion of long-term debt and incurred a charge of $1.35 billion related to the early repayment, net of gains from the unwinding of interest rate swaps related to the debt. Abbott's short- and long-term debt totaled $20.5 billion at December 31, 2012. This balance includes $1 billion of short-borrowings and $14.7 billion of long-term debt that was issued by AbbVie Inc. in 2012. After the separation of AbbVie on January 1, 2013, Abbott has no remaining obligations related to this $15.7 billion of debt. At December 31, 2012, Abbott's long-term debt rating was A+ by Standard and Poor's Corporation and A1 by Moody's Investors Service. Operating cash flows in excess of capital expenditures and cash dividends have partially funded acquisitions over the last three years.

28

        In 2013, Abbott will focus on several key initiatives. In the nutritional business, Abbott will continue to build its product portfolio with the introduction of new science-based products, expand in high-growth emerging markets and implement additional margin improvement initiatives. In the established pharmaceuticals business, which includes international sales of branded generic products, Abbott will continue to focus on obtaining additional product approvals across numerous countries and expanding its presence in emerging markets. In the diagnostics business, Abbott will focus on the development of next-generation instruments and other advanced technologies, expansion in emerging markets, and further improvements in the segment's operating margin. In the vascular business, Abbott will continue to focus on marketing products in the Xience and endovascular franchises, increasing international MitraClip sales, and obtaining regulatory review of the MitraClip device in the U.S. as well as further clinical development of ABSORB, its bioresorbable vascular scaffold (BVS) device and a further roll-out of ABSORB in numerous countries. In Abbott's other segments, Abbott will focus on developing differentiated technologies in higher growth markets.

Critical Accounting Policies

        Sales Rebates —  In 2012, approximately 56 percent of Abbott's consolidated gross revenues were subject to various forms of rebates and allowances that Abbott recorded as reductions of revenues at the time of sale. Most of these rebates and allowances are in the Proprietary Pharmaceutical Products segment and the Nutritional Products segment. Abbott provides rebates to pharmacy benefit management companies, state agencies that administer the federal Medicaid program, insurance companies that administer Medicare drug plans, state agencies that administer the Special Supplemental Nutrition Program for Women, Infants, and Children (WIC), wholesalers, group purchasing organizations, and other government agencies and private entities. Rebate amounts are usually based upon the volume of purchases using contractual or statutory prices for a product. Factors used in the rebate calculations include the identification of which products have been sold subject to a rebate, which customer or government agency price terms apply, and the estimated lag time between sale and payment of a rebate. Using historical trends, adjusted for current changes, Abbott estimates the amount of the rebate that will be paid, and records the liability as a reduction of gross sales when Abbott records its sale of the product. Settlement of the rebate generally occurs from two to eight months after sale. Abbott regularly analyzes the historical rebate trends and makes adjustments to reserves for changes in trends and terms of rebate programs. Rebates and chargebacks charged against gross sales in 2012, 2011 and 2010 amounted to approximately $6.2 billion, $5.5 billion and $4.9 billion, respectively, or 22.9 percent, 22.2 percent and 23.1 percent, respectively, based on gross sales of approximately $26.9 billion, $24.8 billion and $21.1 billion, respectively, subject to rebate. A one-percentage point increase in the percentage of rebates to related gross sales would decrease net sales by approximately $269 million in 2012. Abbott considers a one-percentage point increase to be a reasonably likely increase in the percentage of rebates to related gross sales. Other allowances charged against gross sales were approximately $542 million, $409 million and $415 million for cash discounts in 2012, 2011 and 2010, respectively, and $365 million, $490 million and $537 million for returns in 2012, 2011 and 2010, respectively. Cash discounts are known within 15 to 30 days of sale, and therefore can be reliably estimated. Returns can be reliably estimated because Abbott's historical returns are low, and because sales returns terms and other sales terms have remained relatively unchanged for several periods.

        Management analyzes the adequacy of ending rebate accrual balances each quarter. In the domestic nutritional business, management uses both internal and external data available to estimate the level of inventory in the distribution channel. Management has access to several large customers' inventory management data, and for other customers, utilizes data from a third party that measures time on the retail shelf. These sources allow management to make reliable estimates of inventory in the distribution channel. Except for a transition period before or after a change in the supplier for the WIC business in a state, inventory in the distribution channel does not vary substantially. Management also estimates the states' processing lag time based on claims data. In addition, internal processing time is a factor in

29

estimating the accrual. In the WIC business, the state where the sale is made, which is the determining factor for the applicable price, is reliably determinable. Estimates are required for the amount of WIC sales within each state where Abbott has the WIC business. External data sources utilized for that estimate are participant data from the U.S. Department of Agriculture (USDA), which administers the WIC program, participant data from some of the states, and internally administered market research. The USDA has been making its data available for many years. Internal data includes historical redemption rates and pricing data. At December 31, 2012, Abbott had WIC business in 22 states.

        In the domestic proprietary pharmaceutical business, the most significant charges against gross sales are for Medicaid and Medicare Rebates, Pharmacy Benefit Manager Rebates and Wholesaler Chargebacks. In order to evaluate the adequacy of the ending accrual balances, management uses both internal and external data to estimate the level of inventory in the distribution channel and the rebate claims processing lag time. External data sources used to estimate the inventory in the distribution channel include inventory levels periodically reported by wholesalers and third party market data purchased by Abbott. Management estimates the processing lag time based on periodic sampling of claims data. To estimate the price rebate percentage, systems and calculations are used to track sales by product by customer and to estimate the contractual or statutory price. Abbott's systems and calculations have developed over time as rebates have become more significant, and Abbott believes they are reliable.

        The following table is an analysis of the four largest rebate accruals, which comprise approximately 70 percent of the consolidated rebate provisions charged against revenues in 2012. Remaining rebate provisions charged against gross sales are not significant in the determination of operating earnings. (dollars in millions)



 	Domestic Proprietary Pharmaceutical Products

 	Domestic
Nutritionals
WIC Rebates	 	Medicaid and
Medicare Rebates	 	Pharmacy Benefit
Manager Rebates	 	Wholesaler
Chargebacks
Balance at January 1, 2010

 	$	153	 	$	352	 	$	239	 	$	160
Provisions

 	 	616	 	 	899	 	 	841	 	 	1,162
Payments

 	 	(640	)	 	(617	)	 	(670	)	 	(1,163	)

Balance at December 31, 2010

 	 	129	 	 	634	 	 	410	 	 	159
Provisions

 	 	575	 	 	985	 	 	831	 	 	1,361
Payments

 	 	(568	)	 	(899	)	 	(735	)	 	(1,349	)

Balance at December 31, 2011

 	 	136	 	 	720	 	 	506	 	 	171
Provisions

 	 	657	 	 	1,077	 	 	830	 	 	1,645
Payments

 	 	(670	)	 	(990	)	 	(840	)	 	(1,592	)

Balance at December 31, 2012

 	$	123	 	$	807	 	$	496	 	$	224

        Historically, adjustments to prior years' rebate accruals have not been material to net income. Abbott employs various techniques to verify the accuracy of claims submitted to it, and where possible, works with the organizations submitting claims to gain insight into changes that might affect the rebate amounts. For Medicaid, Medicare and other government agency programs, the calculation of a rebate involves interpretations of relevant regulations, which are subject to challenge or change in interpretation.

        Income Taxes —  Abbott operates in numerous countries where its income tax returns are subject to audits and adjustments. Because Abbott operates globally, the nature of the audit items are often very complex, and the objectives of the government auditors can result in a tax on the same income in more than one country. Abbott employs internal and external tax professionals to minimize audit adjustment amounts where possible. In accordance with the accounting rules relating to the measurement of tax contingencies, in order to recognize an uncertain tax benefit, the taxpayer must be more likely than not of sustaining the position, and the measurement of the benefit is calculated as the largest amount that is more

30

than 50 percent likely to be realized upon resolution of the benefit. Application of these rules requires a significant amount of judgment. In the U.S., Abbott's federal income tax returns through 2009 are settled except for one item, and the income tax returns for years after 2009 are open. Abbott does not record deferred income taxes on earnings reinvested indefinitely in foreign subsidiaries.

        Pension and Post-Employment Benefits —  Abbott offers pension benefits and post-employment health care to many of its employees. Abbott engages outside actuaries to assist in the determination of the obligations and costs under these programs. Abbott must develop long-term assumptions, the most significant of which are the health care cost trend rates, discount rates and the expected return on plan assets. The discount rates used to measure liabilities were determined based on high-quality fixed income securities that match the duration of the expected retiree benefits. The health care cost trend rates represent Abbott's expected annual rates of change in the cost of health care benefits and is a forward projection of health care costs as of the measurement date. A difference between the assumed rates and the actual rates, which will not be known for decades, can be significant in relation to the obligations and the annual cost recorded for these programs. Low asset returns due to poor market conditions and low interest rates have significantly increased actuarial losses for these plans. At December 31, 2012, pretax net actuarial losses and prior service costs and (credits) recognized in Accumulated other comprehensive income (loss) for Abbott's defined benefit plans and medical and dental plans were losses of $4.8 billion and $379 million, respectively. Actuarial losses and gains are amortized over the remaining service attribution periods of the employees under the corridor method, in accordance with the rules for accounting for post-employment benefits. Differences between the expected long-term return on plan assets and the actual annual return are amortized over a five-year period. Note 4 to the consolidated financial statements describes the impact of a one-percentage point change in the health care cost trend rate; however, there can be no certainty that a change would be limited to only one percentage point.

        Valuation of Intangible Assets —  Abbott has acquired and continues to acquire significant intangible assets that Abbott records at fair value. Transactions involving the purchase or sale of intangible assets occur with some frequency between companies in the health care field and valuations are usually based on a discounted cash flow analysis. The discounted cash flow model requires assumptions about the timing and amount of future net cash flows, risk, the cost of capital, terminal values and market participants. Each of these factors can significantly affect the value of the intangible asset. Abbott engages independent valuation experts who review Abbott's critical assumptions and calculations for acquisitions of significant intangibles. Abbott reviews definite-lived intangible assets for impairment each quarter using an undiscounted net cash flows approach. If the undiscounted cash flows of an intangible asset are less than the carrying value of an intangible asset, the intangible asset is written down to its fair value, which is usually the discounted cash flow amount. Where cash flows cannot be identified for an individual asset, the review is applied at the lowest group level for which cash flows are identifiable. Goodwill and indefinite-lived intangible assets, which relate to in-process research and development acquired in a business combination, are reviewed for impairment annually or when an event that could result in an impairment occurs. At December 31, 2012, goodwill and intangibles amounted to $15.8 billion and $8.6 billion, respectively, and amortization expense for intangible assets amounted to $1.4 billion in 2012. There were no impairments of goodwill in 2012, 2011 or 2010. In 2012 and 2011, Abbott recorded impairment charges of $82 million and $174 million, respectively, for certain research and development assets due to changes in the projected development and regulatory timelines for the projects.

        Litigation —  Abbott accounts for litigation losses in accordance with FASB Accounting Standards Codification No. 450, "Contingencies." Under ASC No. 450, loss contingency provisions are recorded for probable losses at management's best estimate of a loss, or when a best estimate cannot be made, a minimum loss contingency amount is recorded. These estimates are often initially developed substantially earlier than the ultimate loss is known, and the estimates are refined each accounting period as additional information becomes known. Accordingly, Abbott is often initially unable to develop a best estimate of loss, and therefore the minimum amount, which could be zero, is recorded. As information becomes

31

known, either the minimum loss amount is increased, resulting in additional loss provisions, or a best estimate can be made, also resulting in additional loss provisions. Occasionally, a best estimate amount is changed to a lower amount when events result in an expectation of a more favorable outcome than previously expected. Abbott estimates the range of possible loss to be from approximately $70 million to $100 million for its legal proceedings and environmental exposures. Accruals of approximately $80 million have been recorded at December 31, 2012 for these proceedings and exposures. These accruals represent management's best estimate of probable loss, as defined by FASB ASC No. 450, "Contingencies."

32


Results of Operations

Sales

        The following table details the components of sales growth by reportable segment for the last three years:



 	Components of Change %

 	Total
% Change

 	Price	 	Volume	 	Exchange
Total Net Sales


2012 vs. 2011


2.6
1.7
3.8
(2.9
)
2011 vs. 2010

 	 	10.5	 	 	1.2	 	 	6.5	 	 	2.8
2010 vs. 2009

 	 	14.3	 	 	(0.1	)	 	13.2	 	 	1.2
Total U.S.


2012 vs. 2011


4.8
5.6
(0.8
)
—
2011 vs. 2010

 	 	5.4	 	 	4.4	 	 	1.0	 	 	—
2010 vs. 2009

 	 	6.8	 	 	0.7	 	 	6.1	 	 	—
Total International


2012 vs. 2011


1.1
(1.0
)
7.1
(5.0
)
2011 vs. 2010

 	 	14.3	 	 	(1.2	)	 	10.6	 	 	4.9
2010 vs. 2009

 	 	20.7	 	 	(0.8	)	 	19.3	 	 	2.2
Proprietary Pharmaceutical Products Segment


2012 vs. 2011


5.5
4.5
3.7
(2.7
)
2011 vs. 2010

 	 	11.0	 	 	3.5	 	 	5.2	 	 	2.3
2010 vs. 2009

 	 	13.6	 	 	0.3	 	 	12.7	 	 	0.6
Established Pharmaceutical Products Segment


2012 vs. 2011


(4.4
)
(1.3
)
3.4
(6.5
)
2011 vs. 2010

 	 	20.0	 	 	(1.7	)	 	17.4	 	 	4.3
2010 vs. 2009

 	 	51.7	 	 	(0.3	)	 	49.1	 	 	2.9
Nutritional Products Segment


2012 vs. 2011


7.7
4.5
4.2
(1.0
)
2011 vs. 2010

 	 	8.6	 	 	3.0	 	 	3.6	 	 	2.0
2010 vs. 2009

 	 	4.7	 	 	1.7	 	 	1.2	 	 	1.8
Diagnostic Products Segment


2012 vs. 2011


4.0
(1.4
)
8.7
(3.3
)
2011 vs. 2010

 	 	8.8	 	 	(1.1	)	 	6.5	 	 	3.4
2010 vs. 2009

 	 	6.0	 	 	0.1	 	 	4.3	 	 	1.6
Vascular Products Segment


2012 vs. 2011


(7.9
)
(5.2
)
(0.4
)
(2.3
)
2011 vs. 2010

 	 	4.4	 	 	(4.3	)	 	5.5	 	 	3.2
2010 vs. 2009

 	 	18.6	 	 	(4.7	)	 	22.3	 	 	1.0
        Total Net Sales in 2012 reflect unit growth, partially offset by the impact of unfavorable foreign exchange. The decrease in 2012 Vascular Products sales is partially due to the winding down of royalty and supply agreements related to certain third-party products, including Promus. Excluding this royalty and supply agreement revenue in both periods and the unfavorable effect of exchange, Vascular Products sales

33

increased 3.4 percent in 2012. In 2011 and 2010, Total Net, Total U.S., Total International, Proprietary Pharmaceutical Products segment and Established Pharmaceutical Products segment sales reflect the acquisition of Solvay's pharmaceuticals business on February 15, 2010 and unit growth, while the relatively weaker U.S. dollar favorably impacted international sales across all segments. Total Net, Total International and Established Pharmaceutical Products segment sales growth in 2011 also reflects the acquisition of Piramal Healthcare Limited's Healthcare Solution business in September 2010.

        A comparison of significant product and product group sales is as follows. Percent changes are versus the prior year and are based on unrounded numbers.


 	2012	 	Percent
Change	 	2011	 	Percent
Change	 	2010	 	Percent
Change

 	(dollars in millions)

Proprietary Pharmaceuticals —


Total U.S. Proprietary sales


$
10,158
7
$
9,455
8
$
8,744
12
HUMIRA

 	 	4,376	 	 	28	 	 	3,427	 	 	19	 	 	2,872	 	 	14
TRILIPIX/TriCor

 	 	1,098	 	 	(20	)	 	1,372	 	 	1	 	 	1,355	 	 	1
Niaspan

 	 	911	 	 	(7	)	 	976	 	 	5	 	 	927	 	 	8
AndroGel

 	 	1,152	 	 	32	 	 	874	 	 	35	 	 	649	 	 	n/m
Lupron

 	 	569	 	 	5	 	 	540	 	 	12	 	 	483	 	 	(11	)
Synthroid

 	 	551	 	 	6	 	 	522	 	 	16	 	 	451	 	 	9
Creon

 	 	353	 	 	7	 	 	332	 	 	35	 	 	246	 	 	n/m
Kaletra

 	 	280	 	 	(14	)	 	326	 	 	(10	)	 	363	 	 	(19	)
Total International Proprietary sales


7,854
3
7,625
15
6,645
16
HUMIRA

 	 	4,889	 	 	9	 	 	4,505	 	 	23	 	 	3,676	 	 	24
Kaletra

 	 	733	 	 	(13	)	 	844	 	 	(5	)	 	892	 	 	(3	)
Lupron

 	 	231	 	 	(14	)	 	270	 	 	2	 	 	265	 	 	2
Total Established Pharmaceuticals —


5,121
(4
)
5,355
20
4,461
52
Clarithromycin

 	 	501	 	 	(9	)	 	551	 	 	6	 	 	521	 	 	(11	)
TriCor and Lipanthyl (fenofibrate)

 	 	292	 	 	(5	)	 	308	 	 	24	 	 	248	 	 	n/m
Creon

 	 	306	 	 	4	 	 	296	 	 	58	 	 	187	 	 	n/m
Serc

 	 	205	 	 	(12	)	 	233	 	 	30	 	 	180	 	 	n/m
Duphaston

 	 	259	 	 	16	 	 	223	 	 	64	 	 	136	 	 	n/m
Synthroid

 	 	105	 	 	1	 	 	103	 	 	9	 	 	95	 	 	19
Nutritionals —


U.S. Pediatric Nutritionals


1,445
14
1,268
5
1,208
(7
)
International Pediatric Nutritionals

 	 	2,080	 	 	8	 	 	1,926	 	 	15	 	 	1,676	 	 	9
U.S. Adult Nutritionals

 	 	1,452	 	 	6	 	 	1,368	 	 	2	 	 	1,345	 	 	6
International Adult Nutritionals

 	 	1,484	 	 	4	 	 	1,427	 	 	13	 	 	1,268	 	 	15
Diagnostics —


Immunochemistry


3,279
4
3,150
8
2,904
4
Vascular Products (1) —


Xience

 	 	1,599	 	 	3	 	 	1,558	 	 	14	 	 	1,370	 	 	40
Other Coronary Products

 	 	598	 	 	(1	)	 	605	 	 	9	 	 	555	 	 	5
Endovascular

 	 	452	 	 	1	 	 	449	 	 	9	 	 	414	 	 	5
n/m — Percent change is not meaningful

(1)
Other Coronary Products include primarily guidewires and balloon catheters. Endovascular includes vessel closure, carotid stents and other peripheral products.
34

        Excluding the negative effect of exchange, Total International Proprietary sales increased 8.9 percent in 2012. In Proprietary Pharmaceuticals, a generic version of TriCor entered the U.S. market in the fourth quarter of 2012. Total Established Pharmaceutical Products sales decreased in 2012 due to the negative effect of exchange and decreased sales of Clarithromycin and Serc due to, in part, pricing pressures in Europe, partially offset by growth in emerging markets. Excluding the effect of exchange, Total Established Pharmaceutical Products sales increased 2.1 percent. U.S. Pediatric Nutritional sales in 2012 reflect market share gains for Similac and unit growth for PediaSure. The increase in 2012 U.S. Adult Nutritional sales reflects unit growth for the Ensure and Glucerna products. International Pediatric and Adult Nutritionals sales increases over the three years were due primarily to volume growth in developing countries and were negatively impacted in 2012 by the effect of the relatively stronger U.S. dollar.

        The increases in U.S. Proprietary product sales in 2011 and 2010 are primarily due to increased sales of HUMIRA and the acquisition of Solvay Pharmaceuticals in February 2010, partially offset by decreased sales of Depakote, Zemplar, and Kaletra. The increases in Established Pharmaceutical sales in 2011 and 2010 are primarily due to the acquisitions of Solvay Pharmaceuticals and Piramal and growth in emerging markets. U.S. Pediatric Nutritionals sales in 2011 and 2010 were affected by the voluntary recall of certain Similac-brand powder infant formulas in September 2010 and the subsequent recovery in market share in 2011. International Proprietary Pharmaceuticals, International Adult Nutritionals and Immunochemistry sales were positively impacted by the effect of the relatively weaker U.S. dollar in 2011 and 2010. Abbott has periodically sold product rights to non-strategic products and has recorded the related gains in net sales in accordance with Abbott's revenue recognition policies as discussed in Note 1 to the consolidated financial statements. Related net sales were approximately $58 million in 2010, while there were no significant sales in 2012 and 2011.

        The expiration of licenses and patent protection and generic competition can affect the future revenues and operating income of Abbott. There are currently no significant patent or license expirations in the next three years that are expected to affect Abbott after the separation of AbbVie.

Operating Earnings

        Gross profit margins were 62.1 percent of net sales in 2012, 60.0 percent in 2011 and 58.3 percent in 2010. The increase in the gross profit margin in 2012 was impacted by improved gross margins across all reportable segments as a result of cost reduction initiatives, the impact of exchange and favorable product mix. The increase in the gross profit margin in 2011 was due, in part, to improved margins in the established pharmaceutical, diagnostics and diabetes businesses and was partially offset by the unfavorable effect of exchange on the profit margin ratio. The increase in the gross profit margin in 2010 was due, in part, to improved margins in the established pharmaceutical, vascular, diabetes, diagnostics and nutritional businesses and the favorable effect of exchange on the gross profit margin ratio.

        In the U.S., states receive price rebates from manufacturers of infant formula under the federally subsidized Special Supplemental Nutrition Program for Women, Infants, and Children. There are also rebate programs for pharmaceutical products in numerous countries. These rebate programs continue to have a negative effect on the gross profit margins of the Nutritional, Proprietary Pharmaceutical and Established Pharmaceutical Products segments.

        Research and development expense was $4.322 billion in 2012, $4.129 billion in 2011 and $3.724 billion in 2010 and represented increases of 4.7 percent in 2012, 10.9 percent in 2011 and 35.7 percent in 2010. Excluding charges related to the Solvay restructurings announced in September 2010, research and development expense increased 6.2 percent in 2011 and 29.4 percent in 2010. The 2010 increase, exclusive of the effects of the restructuring charges, reflects the acquisitions of Solvay's pharmaceuticals business in February 2010 and Facet Biotech in April 2010. The increases in 2012, 2011 and 2010 also reflect continued pipeline spending, including programs in biologics, hepatitis C and diagnostics. The majority of research and development expenditures over the three years were

35

concentrated on pharmaceutical products. $2.9 billion of Abbott's 2012 research and development expenses related to Abbott's pharmaceutical products, of which $2.2 billion was directly allocated to the Proprietary Pharmaceutical Products segment. In 2012, research and development expenditures totaled $367 million for the Vascular Products segment, $382 million for the Diagnostics Products segment, $275 million for the Established Pharmaceutical Products segment and $186 million for the Nutritional Products segment.

        Selling, general and administrative expenses decreased 5.5 percent in 2012 and increased 22.9 percent in 2011 and 23.4 percent in 2010. 2012 includes approximately $405 million related to the separation of AbbVie from Abbott and a $100 million litigation charge related to the government investigation related to Depakote while 2011 includes a litigation charge of $1.5 billion related to the Depakote investigation. Excluding separation costs, litigation charges and Solvay-related restructuring and integration costs, selling, general and administrative expenses increased 4.6 percent in 2012 and 6.7 percent in 2011. Excluding charges related to Solvay restructuring and integration projects, selling, general and administrative expenses in 2010 increased 18.2 percent. This increase, exclusive of the effects of the restructuring and integration charges, reflects the acquisitions of Solvay's pharmaceuticals business in 2010 and Advanced Medical Optics, Inc. in 2009 and higher provisions for litigation in 2010. The remaining increases in selling, general and administrative expenses over the three year period were due primarily to increased selling and marketing support for new and existing products, including continued spending for HUMIRA, inflation, and in 2011, the impact of the pharmaceutical fee imposed by U.S. healthcare reform legislation.

Restructurings

        In 2012, Abbott management approved plans to streamline various commercial operations in order to reduce costs and improve efficiencies in Abbott's core diagnostics, established pharmaceutical and nutritionals businesses. Abbott recorded employee related severance charges of approximately $167 million in 2012. Additional charges of approximately $22 million were also recorded in 2012, primarily for asset impairments. Approximately $70 million is recorded in Cost of products sold and approximately $119 million as Selling, general and administrative expense. As of December 31, 2012, no significant cash payments have been made relating to these actions.

        In 2011 and prior years, Abbott management approved plans to realign its worldwide pharmaceutical and vascular manufacturing operations and selected domestic and international commercial and research and development operations in order to reduce costs. In 2011 and 2010, Abbott recorded charges of approximately $194 million and $56 million, respectively, reflecting the impairment of manufacturing facilities and other assets, employee severance and other related charges. Approximately $76 million in 2011 is classified as Cost of products sold, $69 million as Research and development and $49 million as Selling, general and administrative. Approximately $56 million in 2010 is classified as Cost of products sold. The following summarizes the activity for these restructurings: (dollars in millions)

Accrued balance at January 1, 2010

 	$	145
2010 restructuring charges

 	 	56
Payments, impairments and other adjustments

 	 	(124	)

Accrued balance at December 31, 2010

 	 	77
2011 restructuring charges

 	 	194
Payments, impairments and other adjustments

 	 	(94	)

Accrued balance at December 31, 2011

 	 	177
Payments and other adjustments

 	 	(48	)

Accrued balance at December 31, 2012

 	$	129

36

        An additional $110 million, $25 million and $13 million were recorded in 2012, 2011 and 2010, respectively, relating to these restructurings, primarily for accelerated depreciation.

        In 2012 and 2010, Abbott management approved restructuring plans primarily related to the acquisition of Solvay's pharmaceuticals business. These plans streamline operations, improve efficiencies and reduce costs in certain Solvay sites and functions as well as in certain Abbott and Solvay commercial organizations in various countries. In 2012, Abbott recorded a charge of approximately $150 million for employee severance and contractual obligations, primarily related to the exit from a research and development facility. Approximately $142 million is recorded as Research and development and $8 million as Selling, general and administrative. In 2010, Abbott recorded charges to Cost of products sold, Research and development and Selling, general and administrative of approximately $99 million, $152 million and $272 million, respectively. The following summarizes the activity for these restructurings: (dollars in millions)

2010 restructuring charge

 	$	523
Payments, impairments and other adjustments

 	 	(113	)

Accrued balance at December 31, 2010

 	 	410
Payments and other adjustments

 	 	(302	)

Accrued balance at December 31, 2011

 	 	108
Restructuring charges

 	 	150
Payments and other adjustments

 	 	(143	)

Accrued balance at December 31, 2012

 	$	115

        An additional $38 million, $102 million and $12 million were recorded in 2012, 2011 and 2010, respectively, relating to these restructurings, primarily for additional employee severance and accelerated depreciation.

        In 2011 and 2008, Abbott management approved plans to streamline global manufacturing operations, reduce overall costs, and improve efficiencies in Abbott's core diagnostic business. In 2011, a charge of $28 million was recorded in Cost of products sold. The following summarizes the activity for these restructurings: (dollars in millions)

Accrued balance at January 1, 2010

 	$	98
Payments and other adjustments

 	 	(10	)

Accrued balance at December 31, 2010

 	 	88
2011 restructuring charge

 	 	28
Payments and other adjustments

 	 	(37	)

Accrued balance at December 31, 2011

 	 	79
Payments and other adjustments

 	 	(23	)

Accrued balance at December 31, 2012

 	$	56

        In addition, charges of approximately $16 million, $42 million and $60 million were recorded in 2012, 2011 and 2010, primarily for accelerated depreciation and product transfer costs.

Interest expense and Interest (income)

        In 2012, interest expense increased primarily due to bridge facility fees related to the separation of AbbVie from Abbott. In 2011, interest expense decreased due to lower debt levels. Interest income in 2012 and 2011 decreased as a result of lower rates. In 2010, interest expense increased due primarily to increased debt levels. Interest income decreased in 2010 due to lower investment balances.

37

Change in Accounting Principle and Other (income) expense, net

        Prior to January 1, 2011, the accounts of foreign subsidiaries were consolidated based on a fiscal year ended November 30 due to the time needed to consolidate these subsidiaries. Effective January 1, 2011, the one month lag in the consolidation of the accounts of foreign subsidiaries was eliminated and the year-end of foreign subsidiaries was changed to December 31. Abbott believes that the change in accounting principle related to the elimination of the one month reporting lag is preferable because it results in more contemporaneous reporting of the results of foreign subsidiaries. In accordance with applicable accounting literature, a change in subsidiaries' year-end is treated as a change in accounting principle and requires retrospective application. The cumulative effect of the change was an increase in retained earnings of $289 million as of January 1, 2009 and a corresponding decrease in other long-term liabilities. The impact of the change was not material to the results of operations for the previously reported annual and interim periods after January 1, 2009, and thus, those results have not been revised. A charge of $137 million was recorded to Other (income) expense, net in 2011 to recognize the cumulative immaterial impacts to 2009 and 2010. Had the financial statements been revised, net sales, operating earnings and net earnings in 2010 would have decreased by $21 million, $195 million and $175 million, respectively.

        Other (income) expense, net, for 2012 includes income of approximately $60 million from the resolution of a contractual agreement and a loss of approximately $62 million for the impairment of certain equity securities. Other (income) expense, net, for 2011 includes $56 million of fair value adjustments and accretion in the contingent consideration related to the acquisition of Solvay's pharmaceutical business. Other (income) expense, net, for 2012, 2011 and 2010 also includes ongoing contractual payments from Takeda associated with the conclusion of the TAP joint venture.

Net Loss on Extinguishment of Debt

        In the fourth quarter of 2012, Abbott extinguished $7.7 billion of long-term debt. Abbott incurred a cost of $1.35 billion to extinguish this debt, net of gains from the unwinding of interest rate swaps related to the debt.

Taxes on Earnings

        The income tax rates on earnings were 4.8 percent in 2012, 9.0 percent in 2011 and 19.0 percent in 2010. Taxes on earnings in 2012 reflect the $493 million effect of the tax rate applied to Abbott's net debt extinguishment loss as well as the recognition of $408 million of tax benefits as a result of the favorable resolution of various tax positions pertaining to a prior year, which also decreased the gross amount of unrecognized tax benefits by approximately $560 million. Taxes on earnings in 2011 reflect the effect of the tax rate applied to a litigation reserve and the recognition of $580 million of tax benefits as a result of the favorable resolution of various tax positions pertaining to prior years, which also decreased the gross amount of unrecognized tax benefits by approximately $1.3 billion. Exclusive of these discrete items, the effective rates are lower than the U.S. federal statutory rate of 35 percent due primarily to the benefit of lower foreign tax rates and tax exemptions that reduced the tax rates by 24.9, 22.9, and 19.4 percentage points in 2012, 2011 and 2010, respectively. The tax rate reductions are primarily derived from operations in Puerto Rico, Switzerland, Ireland and Singapore where Abbott benefits from a combination of favorable statutory tax rules, tax rulings, grants, and exemptions. See Note 5 to the consolidated financial statements for a full reconciliation of the effective tax rate to the U.S. federal statutory rate.

        As a result of the American Taxpayer Relief Act of 2012 signed into law in January 2013, Abbott expects to record a tax benefit of approximately $100 million in the first quarter of 2013 for the retroactive extension of the research tax credit and the look-through rules of section 954(c)(6) of the Internal Revenue Code to the beginning of 2012. Excluding this and any other discrete items, Abbott expects to apply an annual effective rate of approximately 21 percent.

38


        As a result of the Patient Protection and Affordable Care Act and the Health Care and Education Reconciliation Act which were signed into law in 2010, Abbott recorded a charge of approximately $60 million in 2010 to reduce deferred tax assets associated with retiree health care liabilities related to the Medicare Part D retiree drug subsidy.

        In October 2010, Puerto Rico enacted legislation that assesses a tax beginning in 2011 on certain products manufactured in Puerto Rico. The tax is levied on gross intercompany purchases from Puerto Rican entities and is included in inventory costs. In 2012 and 2011, cost of goods sold included $187 million and $111 million, respectively, related to this tax. The tax is creditable for U.S. income tax purposes.

Research and Development Programs

        Abbott currently has numerous pharmaceutical, medical and nutritional products in development.

Research and Development Process

        In the Proprietary Pharmaceuticals segment, the research and development process generally begins with discovery research which focuses on the identification of a molecule that has a desired effect against a given disease. If preclinical testing of an identified compound proves successful, the compound moves into clinical development which generally includes the following phases:

•
Phase I — involves the first human tests in a small number of healthy volunteers to assess tolerability and potential dosing.

•
Phase II — tests the molecule's efficacy against the disease in a small group of patients.

•
Phase III — tests a molecule that demonstrates favorable results in the earlier phases in a significantly larger patient population to further demonstrate efficacy and safety based on regulatory criteria.
        The clinical trials from all of the development phases provide the data required to prepare and submit a New Drug Application (NDA), a Biological License Application (BLA) or other submission for regulatory approval to the U.S. Food and Drug Administration (FDA) or similar government agencies outside the U.S. The specific requirements (e.g., scope of clinical trials) for obtaining regulatory approval vary across different countries and geographic regions.

        The research and development process from discovery through a new drug launch typically takes 8 - 12 years and can be even longer. There is a significant amount of uncertainty inherent in the research and development of new pharmaceutical products and there is no guarantee when, or if, a molecule will receive the regulatory approval required to launch a new drug or indication.

        In addition to the development of new products and new formulations, proprietary pharmaceutical research and development projects also may include Phase IV trials, sometimes called post-marketing studies. For such projects, clinical trials are designed and conducted to collect additional data regarding, among other parameters, the benefits and risks of an approved drug.

        In the Established Pharmaceuticals segment, the development process focuses on the geographic expansion and continuous improvement of the segment's existing products to provide benefits to patients and customers. As Established Pharmaceuticals does not actively pursue primary research, development usually begins with work on existing products or after the acquisition of an advanced stage licensing opportunity.

        Depending upon the product, the phases of development may include:

•
Drug product development
39

•
Phase I bioequivalence studies to compare a future Established Pharmaceutical's brand with an already marketed compound with the same active pharmaceutical ingredient (API).

•
Phase II studies to test the efficacy of benefits in a small group of patients.

•
Phase III studies to broaden the testing to a wider population that reflects the actual medical use.

•
Phase IV and other post-marketing studies to obtain new clinical use data on existing products within approved indications.
        The specific requirements (e.g. scope of clinical trials) for obtaining regulatory approval vary across different countries and geographic regions. The process may range from one year for a bioequivalence study project to 6 or more years for complex formulations, new indications, or geographic expansion in specific countries such as China.

        In the Diagnostics segment, the phases of the research and development process include:

•
Discovery which focuses on identification of a product that will address a specific therapeutic area, platform, or unmet clinical need,

•
Concept/Feasibility during which the materials and manufacturing processes are evaluated, testing may include product characterization and analysis is performed to confirm clinical utility, and

•
Development during which extensive testing is performed to demonstrate that the product meets specified design requirements and that the design specifications conform to user needs and intended uses.
        As with pharmaceutical products, the regulatory requirements for diagnostic products vary across different countries and geographic regions. In the U.S., the FDA classifies diagnostic products into classes (I, II, or III) and the classification determines the regulatory process for approval. While the Diagnostics segment has products in all three classes, the vast majority of its products are categorized as Class I or Class II. Submission of a separate regulatory filing is not required for Class I products. Class II devices typically require pre-market notification to the FDA through a regulatory filing known as a 510(k) submission. Most Class III products are subject to the FDA's Pre-Marketing Approval (PMA) requirements. Other Class III products, such as those used to screen blood, require the submission and approval of a BLA.

        In the EU, diagnostic products are also categorized into different categories and the regulatory process, which is governed by the European In Vitro Diagnostic Medical Device Directive, depends upon the category. Certain product categories require review and approval by an independent company, known as a Notified Body, before the manufacturer can affix a CE mark to the product to show compliance with the Directive. Other products only require a self-certification process.

        In the Vascular segment, the research and development process begins with research on a specific technology that is evaluated for feasibility and commercial viability. If the research program passes that hurdle, it moves forward into development. The development process includes evaluation and selection of a product design, completion of clinical trials to test the product's safety and efficacy, and validation of the manufacturing process to demonstrate its repeatability and ability to consistently meet pre-determined specifications.

        Similar to the diagnostic products discussed above, in the U.S., vascular products are classified as Class I, II, or III. Most of Abbott's vascular products are classified as Class II devices that follow the 510(k) regulatory process or Class III devices that are subject to the PMA process.

        In the EU, vascular products are also categorized into different classes and the regulatory process, which is governed by the European Medical Device Directive, varies by class. Each product must bear a CE mark to show compliance with the Directive. Some products require submission of a design dossier to

40

the appropriate regulatory authority for review and approval prior to CE marking of the device. For other products, the company is required to prepare a technical file which includes testing results and clinical evaluations but can self-certify its ability to apply the CE mark to the product. Outside the U.S. and the EU, the regulatory requirements vary across different countries and regions.

        After approval and commercial launch of some vascular products, post-market trials may be conducted either due to a conditional requirement of the regulatory market approval or with the objective of proving product superiority.

        In the Nutritional segment, the research and development process generally focuses on identifying and developing ingredients and products that address the nutritional needs of particular populations (e.g., infants, athletes) or patients (e.g., people with diabetes). Depending upon the country and/or region, if claims regarding a product's efficacy will be made, clinical studies typically must be conducted. Most other product development, such as a product form change from liquid to powder, generally does not necessitate clinical studies.

        In the U.S., the FDA requires that it be notified of proposed new formulations and formulation or packaging changes related to infant formula products. Prior to the launch of an infant formula or product packaging change, the company is required to obtain the FDA's confirmation that it has no objections to the proposed product or packaging. For other nutrition products, notification or pre-approval from the FDA is not required unless the product includes a new food additive. In some countries, regulatory approval may be required for certain nutritional products, including infant formula and medical nutritional products.

Areas of Focus

        In 2013 and beyond, Abbott's significant areas of therapeutic focus will include the following:

Established Pharmaceuticals — Abbott is actively working on plans for about 20 - 30 key brands. Depending on the product, the development activities focus on new data, markets, formulations, combinations, or indications. Abbott focuses on building country-specific portfolios made up of global and local pharmaceutical brands that best meet each local market's needs. Over the next several years, Established Pharmaceuticals will work to expand the market for many of its products through registrations across multiple geographies, including key emerging markets.

Vascular — Ongoing projects in the pipeline include:

•
Xience Xpedition, our next-generation drug-eluting stent (DES) with enhanced deliverability and an expanded size matrix. It utilizes the Xience PRIME stent, everolimus and biocompatible coating technology but incorporates new catheter technology for improved deliverability. Xience Xpedition received U.S. regulatory approval in December 2012 and is also available in Europe and parts of Asia and Latin America. Abbott expects to launch the product in additional markets in 2013.

•
Absorb, the world's first drug eluting bioresorbable vascular scaffold (BVS) device for the treatment of coronary artery disease that is gradually resorbed into the vessel wall. In January 2013 Abbott initiated the ABSORB III clinical trial which is designed to enroll approximately 2,250 patients of which the majority will be in the U.S. The data from this trial will be used to support the U.S. regulatory filing of Absorb. In 2011 Abbott released five-year data from its ABSORB clinical trial, which showed efficacy and safety results consistent with the four-year data. In 2011 after receiving CE Mark approval for Absorb, Abbott initiated a randomized, controlled clinical trial to further study the device in an expanded population in Europe. In 2010, Abbott initiated the ABSORB EXTEND clinical trial which will enroll up to 1,000 patients with more complex coronary artery disease.
41

•
MitraClip device for the treatment of mitral regurgitation — Abbott's MitraClip system which is on the market in Europe and in parts of Asia and Latin America is currently under review for approval by the FDA. An amended filing to the FDA was submitted in December 2011. A FDA panel is expected to review the filing in the first half of 2013.

•
Coronary and endovascular core product projects, including new coronary and endovascular guide wires. The Absolute Pro and Omnilink Elite stent systems, both for the treatment of iliac artery disease, a form of peripheral artery disease that affects the lower extremities, were launched in the U.S. in 2012.
Medical Optics — Abbott is developing a number of new products for patients undergoing cataract surgery, which are designed to improve physician efficiency and patient outcomes. Abbott has developed advanced intraocular lenses (IOLs) that address astigmatism as well as presbyopia. The Tecnis brand monofocal Toric IOL, which is sold in Europe, is currently undergoing U.S. regulatory review. A multifocal version of the Toric IOL was launched in a number of international markets in 2012. A preloaded IOL insertion system that is designed to improve surgeon efficiency is also currently under regulatory review in the U.S.; the product was launched in Europe in 2012. Abbott is continuing the development activities required to obtain U.S. approval for an enhanced version of the Synchrony IOL which is designed to mimic the eye's natural ability to change focus and deliver improved vision at all distances; this product was launched in Europe in late 2012. Abbott has also developed a new diagnostic instrument and laser treatment planning software which is designed to improve visual outcomes. After the receipt of CE Mark approval in November 2011, this instrument and software were launched in Europe in 2012. A PMA filing for U.S. regulatory approval of this product was submitted in 2012.

Molecular Diagnostics — Various new molecular in vitro diagnostic (IVD) products, including oncology and infectious disease assays and a next generation instrument system are in various stages of development and commercialization. Abbott's companion diagnostic test for an ALK gene rearrangement test for non-small-cell lung cancer has been approved in more than 40 countries around the world. In 2012, companion diagnostic efforts were expanded to include collaborative efforts with multiple major pharmaceutical companies. In the U.S., an assay to genotype HCV-infected patients to aid in the choice of an appropriate therapy was submitted for regulatory approval. Additional assays for infectious diseases including MTb and MTb drug resistance are in development.

Core Laboratory Diagnostics — Abbott is working on the development of assays in various areas including infectious disease, cardiac care, fertility and metabolics, and on next-generation blood screening, hematology, and immunochemistry instrument systems. Abbott is also focusing on near-term launches of automation solutions, such as its next-generation track system, ACCELERATOR a3600 to increase efficiency in laboratories.

Diabetes Care — In the first quarter of 2012, Abbott obtained U.S. regulatory approval for its FreeStyle InsuLinx blood glucose monitoring system that includes a touch-screen interface and other features designed to support the insulin-using patient. After receiving CE Mark for this system in May 2011 and Health Canada approval in October 2011, Abbott is continuing to provide R&D support as the product is launched in additional markets. Development is also continuing on an updated hospital blood glucose monitoring system for which a filing for approval is projected to be submitted in the U.S. during the first half of 2013. Abbott is also developing a next-generation monitoring system under the Precision product platform and for which Abbott anticipates submitting filings for approval in various markets in the second half of 2013.

Nutrition — Abbott is focusing its research and development spend on six benefit platforms that span the pediatric, adult and performance nutrition areas: immunity, cognition, lean body mass, inflammation, metabolism and tolerance. Numerous new products that build on advances in these benefit platforms are currently under development and are expected to be launched over the coming years.

42

        Given the diversity of Abbott's business, its intention to remain a broad-based healthcare company and the numerous sources for potential future growth, no individual project is expected to be material to cash flows or results of operations over the next five years. Factors considered included research and development expenses projected to be incurred for the project over the next year relative to Abbott's total research and development expenses as well as qualitative factors, such as marketplace perceptions and impact of a new product on Abbott's overall market position. There were no delays in Abbott's 2012 research and development activities that are expected to have a material impact on operations.

        While the aggregate cost to complete the numerous projects currently in development is expected to be material, the total cost to complete will depend upon Abbott's ability to successfully complete each project, the rate at which each project advances, and the ultimate timing for completion. Given the potential for significant delays and the high rate of failure inherent in the research and development of new pharmaceutical and medical device products and technologies, it is not possible to accurately estimate the total cost to complete all projects currently in development. After the separation of AbbVie, Abbott plans to manage its portfolio of projects to achieve research and development spend equal to approximately 6 percent to 7 percent of sales each year. Abbott does not regularly accumulate or make management decisions based on the total expenses incurred for a particular development phase in a given period.

Business Combinations, Technology Acquisitions and Related Transactions

        On September 8, 2010, Abbott acquired Piramal Healthcare Limited's Healthcare Solutions business, a leader in the Indian branded generics market, for $2.2 billion, in cash, plus additional payments of $400 million annually in 2011, 2012, 2013 and 2014. Abbott recorded a $1.6 billion liability for the present value of the additional payments at the acquisition date. The acquisition was financed with cash. The allocation of the fair value of the acquisition resulted in the recording of $2.7 billion of deductible acquired intangible assets and $1.0 billion of deductible goodwill. Acquired intangible assets consist primarily of trade names, customer relationships and associated rights and are amortized over an average of 19 years.

        In February 2010, Abbott acquired Solvay's pharmaceuticals business (Solvay Pharmaceuticals) for approximately $6.1 billion, in cash, plus additional payments of up to EUR 100 million per year if certain sales milestones are met in 2011, 2012 and 2013. Contingent consideration of approximately $290 million was recorded. The acquisition of Solvay Pharmaceuticals provided Abbott with a large and complementary portfolio of pharmaceutical products and expands Abbott's presence in key global emerging markets. Abbott acquired control of this business on February 15, 2010 and the financial results of the acquired operations are included in these financial statements beginning on that date. Net sales for the acquired operations for 2010 were approximately $3.1 billion. Pretax loss of the acquired operations, including acquisition, integration and restructuring expenses, for 2010 was approximately $395 million. The acquisition was funded with cash and short-term investments. The allocation of the fair value of the acquisition resulted in the recording of $2.2 billion of non-deductible goodwill, $4.1 billion of non-deductible intangible assets, $500 million of non-deductible acquired in-process research and development assets, net tangible assets of $700 million and deferred income taxes of $1.1 billion. Acquired intangible assets consist primarily of product rights for currently marketed products and are amortized over 2 to 14 years (average of 11 years). Acquired in-process research and development projects are accounted for as indefinite lived intangible assets until regulatory approval or discontinuation. The net tangible assets acquired consist primarily of trade accounts receivable of approximately $675 million, inventory of approximately $390 million, property and equipment of approximately $725 million, net of assumed liabilities, primarily trade accounts payable, accrued compensation and other liabilities.

        Had the acquisition of Solvay Pharmaceuticals taken place on January 1, 2010, unaudited pro forma net sales, net earnings and diluted earnings per share for 2010 would have been $35.8 billion, $4.6 billion and $2.96, respectively. The pro forma information includes adjustments for amortization of intangible assets and fair value adjustments to acquisition-date inventory as well as acquisition, integration and restructuring expenses. The pro forma financial information is not necessarily indicative of the results of operations as they would have been had the transaction been effected on the assumed date.

43


        In March 2010, Abbott acquired STARLIMS Technologies for approximately $100 million, in cash, net of cash held by STARLIMS, providing Abbott with leading products and expertise to build its position in laboratory informatics. A substantial portion of the fair value of the acquisition has been allocated to goodwill and amortizable intangible assets. In April 2010, Abbott acquired the outstanding shares of Facet Biotech Corporation for approximately $430 million, in cash, net of cash held by Facet. The acquisition enhanced Abbott's early- and mid-stage pharmaceutical pipeline, including a biologic for multiple sclerosis and compounds that complement Abbott's oncology program. A substantial portion of the fair value of the acquisition was allocated to acquired in-process research and development that is accounted for as an indefinite-lived intangible asset until regulatory approval or discontinuation.

        Except for the acquisition of Solvay Pharmaceuticals, had the above acquisitions taken place on January 1 of the previous year, consolidated net sales and income would not have been significantly different from reported amounts.

        Abbott's Proprietary Pharmaceutical Products segment has entered into various collaboration research and development agreements. In 2012, Abbott acquired AP214, a drug under development for the prevention of acute kidney injury associated with major cardiac surgery in patients at increased risk, and as a result of this transaction, Abbott recorded a charge to acquired in-process and collaborations research and development of $110 million. In addition, in 2012, Abbott entered into a global collaboration to develop and commercialize an oral, next-generation JAK1 inhibitor in Phase II development with the potential to treat multiple autoimmune diseases, and as a result of this transaction Abbott recorded a charge to acquired in-process and collaborations research and development of $150 million. Additional payments of approximately $1.2 billion could be required for the achievement of certain development, regulatory and commercial milestones under this agreement. Under another collaboration, Abbott was granted the rights in 2012 to utilize up to three antibody-drug conjugate compounds and Abbott recorded a charge to acquired in-process and collaborations research and development of $28 million. Additional payments of approximately $220 million for each licensed compound could be required for the achievement of certain development, regulatory and commercial milestones under this agreement. In connection with the acquisition of Solvay Pharmaceuticals, the achievement of a certain sales milestone resulted in a payment of approximately $134 million in the first quarter of 2012 for which a liability was previously established.

        During 2010 and 2011, Abbott entered into a series of transactions with Reata Pharmaceuticals which included (1) a collaboration agreement for the joint development and commercialization of second generation oral antioxidant inflammation modulators resulting in a charge to acquired in-process and collaborations research and development of $400 million in 2011, (2) an agreement to acquire licensing rights outside the U.S., excluding certain Asian markets, to bardoxolone methyl, a product in development for the treatment of chronic kidney disease resulting in a charge to acquired in-process and collaborations research and development of $238 million in 2010 and (3) the acquisition of equity interests in Reata of $62 million each in 2011 and 2010. In 2011, certain milestones were achieved in the development for the treatment of chronic kidney disease and charges to acquired in-process and collaborations research and development of $188 million were recorded. In the first quarter of 2012, $50 million of research and development expense was recorded related to the achievement of a clinical development milestone under the license agreement. The license agreement requires additional payments of up to $150 million if certain development and regulatory milestones associated with the chronic kidney disease compound are achieved.

        On October 17, 2012 Reata informed Abbott that it is discontinuing the Phase III clinical study for bardoxolone methyl for chronic kidney disease. Reata and Abbott will closely examine the data from this study to determine whether there is an appropriate path forward for the development of bardoxolone methyl in chronic kidney disease or other indications. In the fourth quarter of 2012, Abbott recorded a charge of approximately $50 million for the impairment of the equity investment in Reata.

44

        In 2011, Abbott entered into an agreement with Biotest AG to develop and commercialize a treatment for rheumatoid arthritis and psoriasis resulting in a charge to acquired in-process and collaborations research and development of $85 million. Additional payments totaling up to $395 million based on projected regulatory approval timelines could be required for the achievement of certain development, regulatory and commercial milestones under this agreement. In 2010, Abbott entered into an agreement with Neurocrine Biosciences to develop and commercialize a product for the treatment of endometriosis resulting in a charge to acquired in-process and collaborations research and development of $75 million. Additional payments of approximately $500 million could be required for the achievement of certain development, regulatory and commercial milestones under this agreement.

Goodwill

        At December 31, 2012, goodwill recorded as a result of business combinations totaled $15.8 billion. Goodwill is reviewed for impairment annually or when an event that could result in an impairment occurs. The results of the last impairment test indicated that the fair value of each reporting unit was substantially in excess of its carrying value except for the Medical Optics unit. While the fair value of the Medical Optics business exceeds its carrying value, extended economic pressure on government-reimbursed cataract procedures in Europe and on the global LASIK surgery business as well as longer regulatory approval timelines for products currently under development could result in a valuation in the future where the fair value of the Medical Optics unit has declined below its carrying value, thereby triggering the requirement to estimate the implied fair value of the goodwill and measure for impairment.

Financial Condition

Cash Flow

        Net cash from operating activities amounted to $9.3 billion, $9.0 billion and $8.7 billion in 2012, 2011 and 2010, respectively. Trade accounts payable and other liabilities in Net cash from operating activities in 2012 includes the payment of approximately $1.5 billion related to a litigation accrual recorded in 2011. This was partially offset by increases in other liabilities, primarily restructuring reserves. Income taxes payable in 2012 and 2011 includes $408 million and $580 million, respectively, of tax benefits related to the favorable resolution of various tax positions pertaining to prior years. While substantially all cash and cash equivalents at December 31, 2012 that will be retained by Abbott after the separation and all cash and cash equivalents at December 31, 2011 and 2010 is considered reinvested indefinitely in foreign subsidiaries, Abbott does not expect such reinvestment to affect its liquidity and capital resources. If these funds were needed for operations in the U.S., Abbott would be required to accrue and pay U.S. income taxes to repatriate these funds. Abbott believes that it has sufficient sources of liquidity to support its assumption that the disclosed amount of undistributed earnings at December 31, 2012 can be considered to be reinvested indefinitely. Abbott funded $379 million in 2012, $394 million in 2011 and $525 million in 2010 to defined pension plans. Abbott expects pension funding for its main domestic pension plan of $170 million in 2013; the decrease primarily reflects the separation of AbbVie and the transfer of certain plan assets and liabilities to AbbVie. Abbott expects annual cash flow from operating activities to continue to exceed Abbott's capital expenditures and cash dividends.

        For 2010, the reductions in cash and cash equivalents due to the effect of exchange rate changes was primarily driven by the impact of changes in the value of the U.S. dollar compared to the euro on non-dollar denominated cash and cash equivalents. While future fluctuations in the value of the U.S. dollar against foreign currencies could have a substantial effect on the dollar value of Abbott's cash and cash equivalents, such fluctuations are not expected to materially impact Abbott's liquidity.

45

Debt and Capital

        At December 31, 2012, Abbott's long-term debt rating was A+ by Standard & Poor's Corporation and A1 by Moody's Investors Service. In 2012, Abbott replaced unused lines of credit of $3.0 billion and $3.7 billion that were to expire in October 2012 and in 2013, respectively, with two five-year credit facilities totaling $7.0 billion that support commercial paper borrowing arrangements. One of the credit facilities totaling $2.0 billion will support AbbVie commercial paper borrowings after separation and expired for Abbott at the separation of AbbVie from Abbott on January 1, 2013.

        In 2012, Abbott redeemed $7.7 billion of long-term notes in preparation for the separation of AbbVie from Abbott and repaid $1 billion of long-term notes that were due in 2012. In addition, AbbVie Inc., a wholly owned subsidiary of Abbott, issued $14.7 billion of long-term notes that were guaranteed by Abbott until AbbVie's separation from Abbott on January 1, 2013. In 2011, Abbott repaid $2.0 billion of long-term notes using primarily short-term borrowings. Under a registration statement filed with the Securities and Exchange Commission in February 2009, Abbott issued $3.0 billion of long-term debt in the second quarter of 2010 with maturity dates in 2015, 2020 and 2040 and interest rates of 2.7 percent, 4.125 percent and 5.3 percent, respectively. The debt due in 2015 was extinguished in 2012. Proceeds from this debt were used to pay down short-term borrowings.

        In October 2008, the board of directors authorized the purchase of up to $5 billion of Abbott's common shares from time to time. Under this authorization, 37.0 million and 14.8 million shares were purchased in 2012 and 2010 at a cost of approximately $2.2 billion and $800 million, respectively. No shares were purchased under this authorization in 2011. Abbott plans to purchase shares from time to time in 2013.

        The judgment entered in 2009 by the U.S. District Court for the Eastern District of Texas against Abbott in its litigation with New York University and Centocor, Inc. required Abbott to secure the judgment in the event that its appeal to the Federal Circuit court was unsuccessful in overturning the district court's decision. In the first quarter of 2010, Abbott deposited $1.87 billion with an escrow agent and considered these assets to be restricted. On February 23, 2011, the Federal Circuit reversed the district court's final judgment and found Centocor's patent invalid. On April 25, 2011, Centocor petitioned the Federal Circuit to rehear and reconsider the decision. In June 2011 the Federal Circuit denied Centocor's petition and the restrictions on the funds were lifted.

Working Capital

        Working capital was $18.0 billion at December 31, 2012, $8.3 billion at December 31, 2011 and $5.1 billion at December 31, 2010. The increase in working capital in 2012 was due primarily to higher cash generated from operating activities and higher cash and investments as a result of the net issuance of long-term debt in connection with the separation of AbbVie from Abbott. The increase in working capital in 2011 was due primarily to higher cash generated from operating activities and lower debt levels. The decrease in working capital in 2010 was due primarily to cash and investments used to acquire Solvay's pharmaceuticals business and Piramal Healthcare Limited's Healthcare Solutions business.

        Substantially all of Abbott's trade receivables in Italy, Spain, Portugal, and Greece are with governmental health systems. Given the economic conditions and sovereign debt issues in these countries, the time it takes to collect outstanding receivables increased in 2011. In 2012, collection times improved

46

relative to 2011 with the exception of Greece. Outstanding net governmental receivables in these countries at December 31, 2012 were: (dollars in millions)


 	Net
Receivables	 	Percentage Over
One Year Past Due
Italy

 	$	564	 	 	16.2
Spain

 	 	431	 	 	0.8
Portugal

 	 	121	 	 	28.4
Greece

 	 	88	 	 	29.6
        With the exception of Greece, Abbott historically has collected almost all of the outstanding receivables in these countries. Abbott continues to monitor the credit worthiness of customers located in these and other geographic areas and establishes an allowance against a trade receivable when it is probable that the balance will not be collected. In addition to closely monitoring economic conditions and budgetary and other fiscal developments in these countries, Abbott regularly communicates with its customers regarding the status of receivable balances, including their payment plans and obtains positive confirmation of the validity of the receivables. Abbott also monitors the potential for and periodically has utilized factoring arrangements to mitigate credit risk although the receivables included in such arrangements have historically not been a material amount of total outstanding receivables. If government funding were to become unavailable in these countries or if significant adverse changes in their reimbursement practices were to occur, Abbott may not be able to collect the entire balance.

Capital Expenditures

        Capital expenditures of $1.8 billion in 2012, $1.5 billion in 2011 and $1.0 billion in 2010 were principally for upgrading and expanding manufacturing, research and development, and administrative support facilities in all segments, investments in information technology, and laboratory instruments placed with customers.

Contractual Obligations

        The table below summarizes Abbott's estimated contractual obligations as of December 31, 2012. The amounts do not reflect the separation of AbbVie from Abbott on January 1, 2013. After the separation of AbbVie from Abbott on January 1, 2013, principal payments required on long-term debt outstanding and retained by Abbott are $309 million in 2013 and $3.3 billion in 2019 and thereafter. Payments on long-term debt to be made by AbbVie, including interest, total approximately $19.6 billion.


 	Payment Due By Period

 	Total	 	2013	 	2014-2015	 	2016-2017	 	2018 and
Thereafter

 	(dollars in millions)

Long-term debt, including current maturities and future interest payments

 	$	26,403	 	$	839	 	$	5,039	 	$	4,929	 	$	15,596
Operating lease obligations

 	 	795	 	 	146	 	 	234	 	 	170	 	 	245
Capitalized auto lease obligations

 	 	92	 	 	31	 	 	61	 	 	—	 	 	—
Purchase commitments (a)

 	 	3,154	 	 	3,048	 	 	88	 	 	1	 	 	17
Other long-term liabilities reflected on the consolidated balance sheet —


Benefit plan obligations

 	 	5,126	 	 	—	 	 	1,024	 	 	1,149	 	 	2,953
Other

 	 	3,869	 	 	—	 	 	3,682	 	 	32	 	 	155

Total (b)

 	$	39,439	 	$	4,064	 	$	10,128	 	$	6,281	 	$	18,966

(a)
Purchase commitments are for purchases made in the normal course of business to meet operational and capital expenditure requirements.

(b)
Unrecognized tax benefits totaling $2.0 billion are excluded from the table above as Abbott is unable to reasonably estimate the period of cash settlement with the respective taxing authorities on such items.
47

Contingent Obligations

        Abbott has periodically entered into agreements in the ordinary course of business, such as assignment of product rights, with other companies which has resulted in Abbott becoming secondarily liable for obligations that Abbott was previously primarily liable. Since Abbott no longer maintains a business relationship with the other parties, Abbott is unable to develop an estimate of the maximum potential amount of future payments, if any, under these obligations. Based upon past experience, the likelihood of payments under these agreements is remote. In addition, Abbott periodically acquires a business or product rights in which Abbott agrees to pay contingent consideration based on attaining certain thresholds or based on the occurrence of certain events.

Legislative Issues

        In 2010, the Patient Protection and Affordable Care Act and the Health Care and Education Reconciliation Act (collectively referred to herein as "health care reform legislation") were signed into law in the U.S. Health care reform legislation included an increase in the basic Medicaid rebate rate from 15.1 percent to 23.1 percent and extended the rebate to drugs provided through Medicaid managed care organizations.

        Beginning in 2013, health care reform legislation will eliminate the federal income tax deduction for prescription drug expenses of retirees for which Abbott receives reimbursement under the Medicare Part D retiree drug subsidy program. As a result, Abbott recorded a charge of approximately $60 million in the first quarter 2010 to reduce deferred tax assets associated with retiree health care liabilities.

        In 2011, Abbott began recording the annual fee imposed by health care reform legislation on companies that sell branded prescription drugs to specified government programs. The amount of the annual fee is based on the ratio of certain of Abbott's sales as compared to the total such sales of all covered entities multiplied by a fixed dollar amount specified in the legislation by year. In 2011, additional rebates were incurred related to the Medicare Part D coverage gap "donut hole." Beginning in 2013, Abbott will record the 2.3 percent excise tax imposed by health care reform legislation on the sale of certain medical devices in the U.S.

        Abbott's primary markets are highly competitive and subject to substantial government regulations throughout the world. Abbott expects debate to continue over the availability, method of delivery, and payment for health care products and services. It is not possible to predict the extent to which Abbott or the health care industry in general might be adversely affected by these factors in the future. A more complete discussion of these factors is contained in Item 1, Business, and Item 1A, Risk Factors, to the Annual Report on Form 10-K.

Private Securities Litigation Reform Act of 1995 — A Caution Concerning Forward-Looking Statements

        Under the safe harbor provisions of the Private Securities Litigation Reform Act of 1995, Abbott cautions investors that any forward-looking statements or projections made by Abbott, including those made in this document, are subject to risks and uncertainties that may cause actual results to differ materially from those projected. Economic, competitive, governmental, technological and other factors that may affect Abbott's operations are discussed in Item 1A, Risk Factors, to the Annual Report on Form 10-K.

48


ITEM 7A.    QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK

Financial Instruments and Risk Management

Market Price Sensitive Investments

        Abbott holds available-for-sale equity securities from strategic technology acquisitions. The market value of these investments was approximately $76 million and $93 million as of December 31, 2012 and 2011, respectively. Abbott monitors these investments for other than temporary declines in market value, and charges impairment losses to income when an other than temporary decline in value occurs. A hypothetical 20 percent decrease in the share prices of these investments would decrease their fair value at December 31, 2012 by approximately $15 million. (A 20 percent decrease is believed to be a reasonably possible near-term change in share prices.)

Non-Publicly Traded Equity Securities

        Abbott holds equity securities from strategic technology acquisitions that are not traded on public stock exchanges. The carrying value of these investments was approximately $137 million and $224 million as of December 31, 2012 and 2011, respectively. No individual investment is recorded at a value in excess of $30 million. Abbott monitors these investments for other than temporary declines in market value, and charges impairment losses to income when an other than temporary decline in estimated value occurs.

Interest Rate Sensitive Financial Instruments

        At December 31, 2012 and 2011, Abbott had interest rate hedge contracts totaling $9.5 billion and $6.8 billion, respectively, to manage its exposure to changes in the fair value of debt. The effect of these hedges is to change the fixed interest rate to a variable rate. Abbott does not use derivative financial instruments, such as interest rate swaps, to manage its exposure to changes in interest rates for its investment securities. At December 31, 2012, Abbott had $1.3 billion of domestic commercial paper outstanding with an average annual interest rate of 0.32% with an average remaining life of 17 days. The fair value of long-term debt at December 31, 2012 and 2011 amounted to $19.6 billion and $15.1 billion, respectively (average interest rates of 2.9%) with maturities through 2042. At December 31, 2012 and 2011, the fair value of current and long-term investment securities amounted to approximately $4.6 billion and $1.7 billion, respectively. A hypothetical 100-basis point change in the interest rates would not have a material effect on cash flows, income or market values. (A 100-basis point change is believed to be a reasonably possible near-term change in rates.)

Foreign Currency Sensitive Financial Instruments

        Certain Abbott foreign subsidiaries enter into foreign currency forward exchange contracts to manage exposures to changes in foreign exchange rates for anticipated intercompany purchases by those subsidiaries whose functional currencies are not the U.S. dollar. These contracts are designated as cash flow hedges of the variability of the cash flows due to changes in foreign currency exchange rates and are marked-to-market with the resulting gains or losses reflected in Accumulated other comprehensive income (loss). Gains or losses will be included in Cost of products sold at the time the products are sold, generally within the next twelve months. At December 31, 2012 and 2011, Abbott held $1.6 billion of such contracts, which all mature in the following calendar year.

        Abbott enters into foreign currency forward exchange contracts to manage its exposure to foreign currency denominated intercompany loans and trade payables and third-party trade payables and receivables. The contracts are marked-to-market, and resulting gains or losses are reflected in income and are generally offset by losses or gains on the foreign currency exposure being managed. At December 31, 2012 and 2011, Abbott held $18.2 billion and $15.7 billion, respectively, of such contracts, which mature in the next twelve months.

49

        Abbott has designated foreign denominated short-term debt of approximately $615 million and approximately $680 million as of December 31, 2012 and 2011, respectively, as a hedge of the net investment in a foreign subsidiary. Accordingly, changes in the fair value of this debt due to changes in exchange rates are recorded in Accumulated other comprehensive income (loss), net of tax.

        The following table reflects the total foreign currency forward contracts outstanding at December 31, 2012 and 2011: (dollars in millions)


 	2012	 	2011

 	Contract
Amount	 	Weighted
Average
Exchange
Rate	 	Fair and
Carrying Value
Receivable/
(Payable)	 	Contract
Amount	 	Weighted
Average
Exchange
Rate	 	Fair and
Carrying Value
Receivable/
(Payable)
Receive primarily U.S. Dollars
in exchange for
the following currencies:


Euro

 	$	11,349	 	 	1.317	 	$	(4	)	$	10,526	 	 	1.329	 	$	102
British Pound

 	 	1,318	 	 	1.621	 	 	1	 	 	1,501	 	 	1.571	 	 	3
Japanese Yen

 	 	2,624	 	 	81.2	 	 	9	 	 	2,458	 	 	80.3	 	 	(3	)
Canadian Dollar

 	 	332	 	 	.992	 	 	1	 	 	280	 	 	1.026	 	 	(2	)
All other currencies

 	 	4,169	 	 	N/A	 	 	(33	)	 	2,544	 	 	N/A	 	 	(1	)

Total

 	$	19,792	 	 	 	 	$	(26	)	$	17,309	 	 	 	 	$	99

50


ITEM 8.    FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA



 	Page
Consolidated Statement of Earnings

 	 	52
Consolidated Statement of Comprehensive Income

 	 	53
Consolidated Statement of Cash Flows

 	 	54
Consolidated Balance Sheet

 	 	55
Consolidated Statement of Shareholders' Investment

 	 	57
Notes to Consolidated Financial Statements

 	 	58
Management Report on Internal Control Over Financial Reporting

 	 	87
Reports of Independent Registered Public Accounting Firm

 	 	88
51



Abbott Laboratories and Subsidiaries

Consolidated Statement of Earnings
(dollars and shares in thousands except per share data)


 	Year Ended December 31

 	2012	 	2011	 	2010
Net Sales

 	$	39,873,910	 	$	38,851,259	 	$	35,166,721

Cost of products sold

 	 	15,119,718	 	 	15,540,580	 	 	14,665,192
Research and development

 	 	4,322,182	 	 	4,129,414	 	 	3,724,424
Acquired in-process and collaborations research and development

 	 	288,000	 	 	672,500	 	 	313,200
Selling, general and administrative

 	 	12,059,495	 	 	12,756,817	 	 	10,376,324

Total Operating Cost and Expenses

 	 	31,789,395	 	 	33,099,311	 	 	29,079,140

Operating Earnings

 	 	8,084,515	 	 	5,751,948	 	 	6,087,581
Interest expense

 	 	592,403	 	 	530,141	 	 	553,135
Interest (income)

 	 	(79,225	)	 	(85,196	)	 	(105,453	)
Net loss on extinguishment of debt

 	 	1,350,973	 	 	—	 	 	—
Net foreign exchange (gain) loss

 	 	(8,044	)	 	(50,271	)	 	(10,924	)
Other (income) expense, net

 	 	(34,206	)	 	158,632	 	 	(62,011	)

Earnings Before Taxes

 	 	6,262,614	 	 	5,198,642	 	 	5,712,834
Taxes on Earnings

 	 	299,694	 	 	470,193	 	 	1,086,662

Net Earnings

 	$	5,962,920	 	$	4,728,449	 	$	4,626,172

Basic Earnings Per Common Share

 	$	3.76	 	$	3.03	 	$	2.98

Diluted Earnings Per Common Share

 	$	3.72	 	$	3.01	 	$	2.96

Average Number of Common Shares Outstanding Used for Basic Earnings Per Common Share

 	 	1,575,378	 	 	1,557,643	 	 	1,546,400
Dilutive Common Stock Options and Awards

 	 	16,460	 	 	9,746	 	 	9,622

Average Number of Common Shares Outstanding Plus Dilutive Common Stock Options and Awards

 	 	1,591,838	 	 	1,567,389	 	 	1,556,022

Outstanding Common Stock Options Having No Dilutive Effect

 	 	1,166	 	 	26,789	 	 	29,403

The accompanying notes to consolidated financial statements are an integral part of this statement.

52



Abbott Laboratories and Subsidiaries

Consolidated Statement of Comprehensive Income
(dollars in thousands)


 	Year Ended December 31

 	2012	 	2011	 	2010
Net Earnings

 	$	5,962,920	 	$	4,728,449	 	$	4,626,172

Foreign currency translation (loss) adjustments

 	 	(6,826	)	 	(817,539	)	 	(2,290,256	)
Net actuarial (losses) and prior service cost and credits and amortization of net actuarial losses and prior service cost and credits, net of taxes of $(276,076) in 2012, $(391,528) in 2011 and $(70,389) in 2010

 	 	(864,935	)	 	(510,444	)	 	(59,447	)
Unrealized (losses) gains on marketable equity securities, net of taxes of $(4,079) in 2012, $8,338 in 2011 and $61 in 2010

 	 	(7,066	)	 	14,442	 	 	106
Net adjustments for derivative instruments designated as cash flow hedges, net of taxes of $(29,417) in 2012, $19,857 in 2011 and $20,567 in 2010

 	 	(117,666	)	 	83,202	 	 	128,677

Other Comprehensive (loss)

 	 	(996,493	)	 	(1,230,339	)	 	(2,220,920	)

Comprehensive Income

 	$	4,966,427	 	$	3,498,110	 	$	2,405,252

Supplemental Accumulated Other Comprehensive Income Information, net of tax as of December 31:


Cumulative foreign currency translation loss (gain) adjustments

 	$	79,353	 	$	72,527	 	$	(745,012	)
Net actuarial losses and prior service cost and credits

 	 	3,595,554	 	 	2,730,619	 	 	2,220,175
Cumulative unrealized (gains) on marketable equity securities

 	 	(31,363	)	 	(38,429	)	 	(23,987	)
Cumulative (gains) on derivative instruments designated as cash flow hedges

 	 	(49,866	)	 	(167,532	)	 	(84,330	)
The accompanying notes to consolidated financial statements are an integral part of this statement.

53



Abbott Laboratories and Subsidiaries

Consolidated Statement of Cash Flows
(dollars in thousands)


 	Year Ended December 31

 	2012	 	2011	 	2010
Cash Flow From (Used in) Operating Activities:


Net earnings

 	$	5,962,920	 	$	4,728,449	 	$	4,626,172
Adjustments to reconcile earnings to net cash from operating activities —


Depreciation

 	 	1,363,673	 	 	1,395,371	 	 	1,207,450
Amortization of intangible assets

 	 	1,419,534	 	 	1,648,523	 	 	1,416,855
Share-based compensation

 	 	433,114	 	 	382,602	 	 	387,183
Acquired in-process and collaborations research and development

 	 	288,000	 	 	672,500	 	 	313,200
Investing and financing (gains) losses, net

 	 	356,020	 	 	141,565	 	 	126,337
Net loss on extinguishment of debt

 	 	1,350,973	 	 	—	 	 	—
Trade receivables

 	 	35,996	 	 	(670,152	)	 	(394,665	)
Inventories

 	 	(417,053	)	 	(129,621	)	 	139,857
Prepaid expenses and other assets

 	 	(35,298	)	 	413,266	 	 	553,145
Trade accounts payable and other liabilities

 	 	(134,209	)	 	1,789,652	 	 	572,533
Income taxes

 	 	(1,309,269	)	 	(1,402,078	)	 	(212,086	)

Net Cash From Operating Activities

 	 	9,314,401	 	 	8,970,077	 	 	8,735,981

Cash Flow From (Used in) Investing Activities:


Acquisitions of businesses and technologies, net of cash acquired

 	 	(1,227,473	)	 	(672,500	)	 	(9,433,243	)
Acquisitions of property and equipment

 	 	(1,795,289	)	 	(1,491,500	)	 	(1,015,075	)
Purchases of investment securities

 	 	(11,997,654	)	 	(5,109,987	)	 	(805,932	)
Proceeds from sales of investment securities

 	 	8,936,406	 	 	5,648,720	 	 	954,361
Release of (deposit of) restricted funds

 	 	—	 	 	1,870,000	 	 	(1,870,000	)
Other

 	 	2,722	 	 	16,099	 	 	(18,426	)

Net Cash (Used in) From Investing Activities

 	 	(6,081,288	)	 	260,832	 	 	(12,188,315	)

Cash Flow From (Used in) Financing Activities:


Proceeds from issuance of (repayments of) short-term debt and other

 	 	783,868	 	 	(1,964,685	)	 	(203,854	)
Proceeds from issuance of long-term debt and debt with maturities over 3 months

 	 	14,700,000	 	 	1,000,000	 	 	4,000,000
Repayments of long-term debt and debt with maturities over 3 months

 	 	(11,071,178	)	 	(3,012,426	)	 	(1,673,998	)
Purchases of common shares

 	 	(2,364,240	)	 	(77,007	)	 	(866,825	)
Proceeds from stock options exercised, including income tax benefit

 	 	1,850,454	 	 	968,759	 	 	328,411
Dividends paid

 	 	(3,182,811	)	 	(2,938,096	)	 	(2,671,475	)

Net Cash From (Used in) Financing Activities

 	 	716,093	 	 	(6,023,455	)	 	(1,087,741	)

Effect of exchange rate changes on cash and cash equivalents

 	 	40,137	 	 	(43,005	)	 	(620,893	)

Net Increase (Decrease) in Cash and Cash Equivalents

 	 	3,989,343	 	 	3,164,449	 	 	(5,160,968	)
Cash and Cash Equivalents, Beginning of Year

 	 	6,812,820	 	 	3,648,371	 	 	8,809,339

Cash and Cash Equivalents, End of Year

 	$	10,802,163	 	$	6,812,820	 	$	3,648,371

Supplemental Cash Flow Information:


Income taxes paid

 	$	1,366,581	 	$	1,781,602	 	$	809,710
Interest paid

 	 	575,895	 	 	544,559	 	 	580,168
The accompanying notes to consolidated financial statements are an integral part of this statement.

54




Abbott Laboratories and Subsidiaries

Consolidated Balance Sheet
(dollars in thousands)


 	December 31

 	2012	 	2011	 	2010
Current Assets:


Cash and cash equivalents

 	$	10,802,163	 	$	6,812,820	 	$	3,648,371
Investments, primarily bank time deposits and U.S. treasury bills

 	 	4,371,821	 	 	1,284,539	 	 	1,803,079
Restricted funds, primarily U.S. treasury bills

 	 	—	 	 	—	 	 	1,872,490
Trade receivables, less allowances of — 2012: $405,921; 2011: $420,579; 2010: $388,564

 	 	7,612,860	 	 	7,683,920	 	 	7,184,034
Inventories:


Finished products

 	 	2,345,455	 	 	2,220,527	 	 	2,058,735
Work in process

 	 	628,874	 	 	432,358	 	 	383,580
Materials

 	 	817,984	 	 	631,364	 	 	746,419

Total inventories

 	 	3,792,313	 	 	3,284,249	 	 	3,188,734
Deferred income taxes

 	 	2,986,216	 	 	2,700,540	 	 	3,076,051
Other prepaid expenses and receivables

 	 	1,757,210	 	 	2,002,706	 	 	1,544,770

Total Current Assets

 	 	31,322,583	 	 	23,768,774	 	 	22,317,529

Investments

 	 	273,595	 	 	378,225	 	 	302,049

Property and Equipment, at Cost:


Land

 	 	604,462	 	 	633,917	 	 	648,988
Buildings

 	 	4,259,240	 	 	4,467,387	 	 	4,334,236
Equipment

 	 	13,110,833	 	 	12,216,388	 	 	11,813,618
Construction in progress

 	 	954,352	 	 	698,873	 	 	577,460


 	 	18,928,887	 	 	18,016,565	 	 	17,374,302
Less: accumulated depreciation and amortization

 	 	10,865,840	 	 	10,142,610	 	 	9,403,346

Net Property and Equipment

 	 	8,063,047	 	 	7,873,955	 	 	7,970,956

Intangible Assets, net of amortization

 	 	8,588,285	 	 	9,989,636	 	 	12,151,628
Goodwill

 	 	15,774,127	 	 	15,705,380	 	 	15,930,077
Deferred Income Taxes and Other Assets

 	 	3,213,307	 	 	2,560,923	 	 	1,901,613


 	$	67,234,944	 	$	60,276,893	 	$	60,573,852

55


Abbott Laboratories and Subsidiaries

Consolidated Balance Sheet
(dollars in thousands)


 	December 31

 	2012	 	2011	 	2010
Liabilities and Shareholders' Investment


Current Liabilities:


Short-term borrowings

 	$	2,081,839	 	$	2,347,859	 	$	4,349,796
Trade accounts payable

 	 	1,796,990	 	 	1,721,127	 	 	1,535,759
Salaries, wages and commissions

 	 	1,427,765	 	 	1,260,121	 	 	1,328,665
Other accrued liabilities

 	 	6,787,995	 	 	7,854,994	 	 	6,014,772
Dividends payable

 	 	221,340	 	 	754,284	 	 	680,749
Income taxes payable

 	 	655,424	 	 	514,947	 	 	1,307,723
Current portion of long-term debt

 	 	308,823	 	 	1,026,896	 	 	2,044,970

Total Current Liabilities

 	 	13,280,176	 	 	15,480,228	 	 	17,262,434

Long-term Debt

 	 	18,085,302	 	 	12,039,822	 	 	12,523,517

Post-employment Obligations and Other Long-term Liabilities

 	 	9,056,234	 	 	8,230,698	 	 	8,022,770

Commitments and Contingencies


Shareholders' Investment:


Preferred shares, one dollar par value Authorized — 1,000,000 shares, none issued

 	 	—	 	 	—	 	 	—
Common shares, without par value Authorized — 2,400,000,000 shares Issued at stated capital amount — Shares: 2012: 1,675,930,484; 2011: 1,638,870,201; 2010: 1,619,689,876

 	 	11,754,552	 	 	9,817,134	 	 	8,744,703
Common shares held in treasury, at cost — Shares: 2012: 99,262,992; 2011: 68,491,382; 2010: 72,705,928

 	 	(5,590,909	)	 	(3,687,478	)	 	(3,916,823	)
Earnings employed in the business

 	 	24,150,996	 	 	20,907,362	 	 	19,215,768
Accumulated other comprehensive income (loss)

 	 	(3,593,678	)	 	(2,597,185	)	 	(1,366,846	)

Total Abbott Shareholders' Investment

 	 	26,720,961	 	 	24,439,833	 	 	22,676,802
Noncontrolling Interests in Subsidiaries

 	 	92,271	 	 	86,312	 	 	88,329

Total Shareholders' Investment

 	 	26,813,232	 	 	24,526,145	 	 	22,765,131


 	$	67,234,944	 	$	60,276,893	 	$	60,573,852

The accompanying notes to consolidated financial statements are an integral part of this statement.

56



Abbott Laboratories and Subsidiaries

Consolidated Statement of Shareholders' Investment
(dollars in thousands except per share data)


 	Year Ended December 31

 	2012	 	2011	 	2010
Common Shares:


Beginning of Year


Shares: 2012: 1,638,870,201; 2011: 1,619,689,876; 2010: 1,612,683,987

 	$	9,817,134	 	$	8,744,703	 	$	8,257,873
Issued under incentive stock programs


Shares: 2012: 37,060,283; 2011: 19,180,325; 2010: 7,005,889

 	 	1,853,574	 	 	954,148	 	 	316,071
Share-based compensation

 	 	434,601	 	 	382,326	 	 	388,493
Issuance of restricted stock awards

 	 	(350,757	)	 	(264,043	)	 	(217,734	)

End of Year


Shares: 2012: 1,675,930,484; 2011: 1,638,870,201; 2010: 1,619,689,876

 	$	11,754,552	 	$	9,817,134	 	$	8,744,703

Common Shares Held in Treasury:


Beginning of Year


Shares: 2012: 68,491,382; 2011: 72,705,928; 2010: 61,516,398

 	$	(3,687,478	)	$	(3,916,823	)	$	(3,310,347	)
Issued under incentive stock programs


Shares: 2012: 6,691,748; 2011: 4,638,841; 2010: 4,166,200

 	 	362,764	 	 	249,876	 	 	224,237
Purchased


Shares: 2012: 37,463,358; 2011: 424,295; 2010: 15,355,730

 	 	(2,266,195	)	 	(20,531	)	 	(830,713	)

End of Year


Shares: 2012: 99,262,992; 2011: 68,491,382; 2010: 72,705,928

 	$	(5,590,909	)	$	(3,687,478	)	$	(3,916,823	)

Earnings Employed in the Business:


Beginning of Year

 	$	20,907,362	 	$	19,215,768	 	$	17,342,694
Net earnings

 	 	5,962,920	 	 	4,728,449	 	 	4,626,172
Cash dividends declared on common shares (per share — 2012: $1.67; 2011: $1.92; 2010: $1.76)

 	 	(2,649,866	)	 	(3,011,631	)	 	(2,731,584	)
Effect of common and treasury share transactions

 	 	(69,420	)	 	(25,224	)	 	(21,514	)

End of Year

 	$	24,150,996	 	$	20,907,362	 	$	19,215,768

Accumulated Other Comprehensive Income (Loss):


Beginning of Year

 	$	(2,597,185	)	$	(1,366,846	)	$	854,074
Other comprehensive income (loss)

 	 	(996,493	)	 	(1,230,339	)	 	(2,220,920	)

End of Year

 	$	(3,593,678	)	$	(2,597,185	)	$	(1,366,846	)

Noncontrolling Interests in Subsidiaries:


Beginning of Year

 	$	86,312	 	$	88,329	 	$	43,102
Noncontrolling Interests' share of income, business combinations, net of distributions and share repurchases

 	 	5,959	 	 	(2,017	)	 	45,227

End of Year

 	$	92,271	 	$	86,312	 	$	88,329

The accompanying notes to consolidated financial statements are an integral part of this statement.

57



Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements

Note 1 — Summary of Significant Accounting Policies

        NATURE OF BUSINESS — Abbott's principal business is the discovery, development, manufacture and sale of a broad line of health care products.

        In October 2011, Abbott announced a plan to separate into two publicly traded companies, one in diversified medical products and the other in research-based pharmaceuticals. To accomplish the separation, Abbott created a new company, AbbVie Inc. for its research-based pharmaceuticals business which consists primarily of Abbott's Proprietary Pharmaceutical Products segment. On January 1, 2013, Abbott distributed all of the outstanding shares of AbbVie Inc. to Abbott's shareholders. As a result of the distribution, AbbVie is now an independent company trading under the symbol "ABBV". Beginning in the first quarter of 2013, the historical results of the research-based pharmaceuticals business will be reflected in Abbott's consolidated financial statements as discontinued operations.

        CONCENTRATION OF RISK AND GUARANTEES — Due to the nature of its operations, Abbott is not subject to significant concentration risks relating to customers, products or geographic locations, except that three U.S. wholesalers accounted for 23 percent of trade receivables as of December 31, 2012 and 2010 and 22 percent of trade receivables as of December 31, 2011. In addition, governmental accounts in Italy, Spain, Greece and Portugal accounted for 16 percent, 23 percent, and 21 percent of total net trade receivables as of December 31, 2012, 2011, and 2010, respectively. Product warranties are not significant.

        Abbott has no material exposures to off-balance sheet arrangements; no special purpose entities; nor activities that include non-exchange-traded contracts accounted for at fair value. Abbott has periodically entered into agreements in the ordinary course of business, such as assignment of product rights, with other companies which has resulted in Abbott becoming secondarily liable for obligations that Abbott was previously primarily liable. Since Abbott no longer maintains a business relationship with the other parties, Abbott is unable to develop an estimate of the maximum potential amount of future payments, if any, under these obligations. Based upon past experience, the likelihood of payments under these agreements is remote. Abbott periodically acquires a business or product rights in which Abbott agrees to pay contingent consideration based on attaining certain thresholds or based on the occurrence of certain events.

        BASIS OF CONSOLIDATION AND CHANGE IN ACCOUNTING PRINCIPLE — Prior to January 1, 2011, the accounts of foreign subsidiaries were consolidated based on a fiscal year ended November 30 due to the time needed to consolidate these subsidiaries. Effective January 1, 2011, the one month lag in the consolidation of the accounts of foreign subsidiaries was eliminated and the year-end of foreign subsidiaries was changed to December 31. Abbott believes that the change in accounting principle related to the elimination of the one month reporting lag is preferable because it results in more contemporaneous reporting of the results of foreign subsidiaries. In accordance with applicable accounting literature, a change in subsidiaries' year-end is treated as a change in accounting principle and requires retrospective application. The cumulative effect of the change was an increase in retained earnings of $289 million as of January 1, 2009 and a corresponding decrease in other long-term liabilities. The impact of the change was not material to the results of operations for the previously reported annual and interim periods after January 1, 2009, and thus, those results have not been revised. A charge of $137 million was recorded to Other (income) expense, net in 2011 to recognize the cumulative immaterial impacts to 2009 and 2010. Had the financial statements been revised, net sales, operating earnings and net earnings in 2010 would have decreased by $21 million, $195 million and $175 million, respectively.

        USE OF ESTIMATES — The financial statements have been prepared in accordance with generally accepted accounting principles in the United States and necessarily include amounts based on estimates and assumptions by management. Actual results could differ from those amounts. Significant estimates

58


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 1 — Summary of Significant Accounting Policies (Continued)

include amounts for sales rebates, income taxes, pension and other post-employment benefits, valuation of intangible assets, litigation, derivative financial instruments, and inventory and accounts receivable exposures.

        REVENUE RECOGNITION — Revenue from product sales is recognized upon passage of title and risk of loss to customers. Provisions for discounts, rebates and sales incentives to customers, and returns and other adjustments are provided for in the period the related sales are recorded. Sales incentives to customers are not material. Historical data is readily available and reliable, and is used for estimating the amount of the reduction in gross sales. Revenue from the launch of a new product, from an improved version of an existing product, or for shipments in excess of a customer's normal requirements are recorded when the conditions noted above are met. In those situations, management records a returns reserve for such revenue, if necessary. In certain of Abbott's businesses, primarily within diagnostics and medical optics, Abbott participates in selling arrangements that include multiple deliverables (e.g., instruments, reagents, procedures, and service agreements). Under these arrangements, Abbott recognizes revenue upon delivery of the product or performance of the service and allocates the revenue based on the relative selling price of each deliverable, which is based primarily on vendor specific objective evidence. Sales of product rights for marketable products are recorded as revenue upon disposition of the rights. Revenue from license of product rights, or for performance of research or selling activities, is recorded over the periods earned.

        INCOME TAXES — Deferred income taxes are provided for the tax effect of differences between the tax bases of assets and liabilities and their reported amounts in the financial statements at the enacted statutory rate to be in effect when the taxes are paid. U.S. income taxes are provided on those earnings of foreign subsidiaries which are intended to be remitted to the parent company. Deferred income taxes are not provided on undistributed earnings reinvested indefinitely in foreign subsidiaries as working capital and plant and equipment. Interest and penalties on income tax obligations are included in taxes on income.

        EARNINGS PER SHARE — Unvested restricted stock that contain non-forfeitable rights to dividends are treated as participating securities and are included in the computation of earnings per share under the two-class method. Under the two-class method, net earnings are allocated between common shares and participating securities. Net earnings allocated to common shares in 2012, 2011 and 2010 were $5.917 billion, $4.714 billion and $4.613 billion, respectively.

        PENSION AND POST-EMPLOYMENT BENEFITS — Abbott accrues for the actuarially determined cost of pension and post-employment benefits over the service attribution periods of the employees. Abbott must develop long-term assumptions, the most significant of which are the health care cost trend rates, discount rates and the expected return on plan assets. Differences between the expected long-term return on plan assets and the actual return are amortized over a five-year period. Actuarial losses and gains are amortized over the remaining service attribution periods of the employees under the corridor method.

        FAIR VALUE MEASUREMENTS — For assets and liabilities that are measured using quoted prices in active markets, total fair value is the published market price per unit multiplied by the number of units held without consideration of transaction costs. Assets and liabilities that are measured using significant other observable inputs are valued by reference to similar assets or liabilities, adjusted for contract restrictions and other terms specific to that asset or liability. For these items, a significant portion of fair value is derived by reference to quoted prices of similar assets or liabilities in active markets. For all remaining assets and liabilities, fair value is derived using a fair value model, such as a discounted cash flow

59


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 1 — Summary of Significant Accounting Policies (Continued)

model or Black-Scholes model. Purchased intangible assets are recorded at fair value. The fair value of significant purchased intangible assets is based on independent appraisals. Abbott uses a discounted cash flow model to value intangible assets. The discounted cash flow model requires assumptions about the timing and amount of future net cash flows, risk, the cost of capital, terminal values and market participants. Intangible assets, goodwill and indefinite-lived intangible assets are reviewed for impairment at least on a quarterly and annual basis, respectively.

        SHARE-BASED COMPENSATION — The value of stock options and restricted stock awards and units are amortized over their service period, which could be shorter than the vesting period if an employee is retirement eligible, with a charge to compensation expense.

        LITIGATION — Abbott accounts for litigation losses in accordance with FASB ASC No. 450, "Contingencies." Under ASC No. 450, loss contingency provisions are recorded for probable losses at management's best estimate of a loss, or when a best estimate cannot be made, a minimum loss contingency amount is recorded. Legal fees are recorded as incurred.

        CASH, CASH EQUIVALENTS AND INVESTMENTS — Cash equivalents consist of bank time deposits and U.S. treasury bills with original maturities of three months or less. Investments in marketable equity securities and certain investments in debt securities are classified as available-for-sale and are recorded at fair value with any unrealized holding gains or losses, net of tax, included in Accumulated other comprehensive income (loss). Investments in equity securities that are not traded on public stock exchanges are recorded at cost. Investments in other debt securities are classified as held-to-maturity, as management has both the intent and ability to hold these securities to maturity, and are reported at cost, net of any unamortized premium or discount. Income relating to these securities is reported as interest income.

        Abbott reviews the carrying value of investments each quarter to determine whether an other than temporary decline in market value exists. Abbott considers factors affecting the investee, factors affecting the industry the investee operates in and general equity market trends. Abbott considers the length of time an investment's market value has been below carrying value and the near-term prospects for recovery to carrying value. When Abbott determines that an other than temporary decline has occurred, the investment is written down with a charge to Other (income) expense, net.

        TRADE RECEIVABLE VALUATIONS — Accounts receivable are stated at their net realizable value. The allowance against gross trade receivables reflects the best estimate of probable losses inherent in the receivables portfolio determined on the basis of historical experience, specific allowances for known troubled accounts and other currently available information. Accounts receivable are charged off after all reasonable means to collect the full amount (including litigation, where appropriate) have been exhausted.

        INVENTORIES — Inventories are stated at the lower of cost (first-in, first-out basis) or market. Cost includes material and conversion costs.

60


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 1 — Summary of Significant Accounting Policies (Continued)

        PROPERTY AND EQUIPMENT — Depreciation and amortization are provided on a straight-line basis over the estimated useful lives of the assets. The following table shows estimated useful lives of property and equipment:

Classification
 	Estimated Useful Lives
Buildings

 	10 to 50 years (average 27 years)
Equipment

 	3 to 20 years (average 11 years)
        PRODUCT LIABILITY — Abbott accrues for product liability claims, on an undiscounted basis, when it is probable that a liability has been incurred and the amount of the liability can be reasonably estimated based on existing information. The liabilities are adjusted quarterly as additional information becomes available. Receivables for insurance recoveries for product liability claims are recorded as assets, on an undiscounted basis, when it is probable that a recovery will be realized. Product liability losses are self-insured.

        RESEARCH AND DEVELOPMENT COSTS — Internal research and development costs are expensed as incurred. Clinical trial costs incurred by third parties are expensed as the contracted work is performed. Where contingent milestone payments are due to third parties under research and development arrangements, the milestone payment obligations are expensed when the milestone results are achieved.

        ACQUIRED IN-PROCESS AND COLLABORATIONS RESEARCH AND DEVELOPMENT (IPR&D) — The initial costs of rights to IPR&D projects obtained in an asset acquisition are expensed as IPR&D unless the project has an alternative future use. These costs include initial payments incurred prior to regulatory approval in connection with research and development collaboration agreements that provide rights to develop, manufacture, market and/or sell pharmaceutical products. The fair value of IPR&D projects acquired in a business combination are capitalized and accounted for as indefinite-lived intangible assets.

Note 2 — Supplemental Financial Information


 	2012	 	2011	 	2010

 	(dollars in millions)

Long-term Investments:


Equity securities

 	$	213	 	$	317	 	$	240
Other

 	 	61	 	 	61	 	 	62

Total

 	$	274	 	$	378	 	$	302

        The loss on the extinguishment of debt of $1.35 billion relates to the early redemption of $7.7 billion of long-term notes. The loss consists of the premium paid on the notes and the write off of deferred financing costs totaling $1.83 billion and was partially offset by a gain of $479 million related to the unwinding of interest rate swaps related to a portion of the debt. Other (income) expense, net, for 2012 includes income of approximately $60 million from the resolution of a contractual agreement and a loss of approximately $62 million for the impairment of certain equity securities. As discussed in Note 1, Other (income) expense, net, for 2011 includes a charge of $137 million to recognize the cumulative immaterial impacts to 2009 and 2010 relating to the change in year end for foreign subsidiaries. In addition, Other

61


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 2 — Supplemental Financial Information (Continued)

(income) expense, net, for 2011 includes $56 million of fair value adjustments and accretion in the contingent consideration related to the acquisition of Solvay's pharmaceutical business. Other (income) expense, net, for 2012, 2011 and 2010 also includes ongoing contractual payments from Takeda associated with the conclusion of the TAP joint venture.


 	2012	 	2011	 	2010

 	(dollars in millions)

Other Accrued Liabilities:


Accrued rebates payable to government agencies

 	$	1,020	 	$	1,049	 	$	900
Accrued other rebates (a)

 	 	1,079	 	 	1,030	 	 	862
All other (b)

 	 	4,689	 	 	5,776	 	 	4,253

Total

 	$	6,788	 	$	7,855	 	$	6,015

(a)
Accrued wholesaler chargeback rebates of $300, $239 and $216 at December 31, 2012, 2011 and 2010, respectively, are netted in trade receivables because Abbott's customers are invoiced at a higher catalog price but only remit to Abbott their contract price for the products. The 2011 balances have been revised to reflect a reclassification of certain amounts from Accrued other rebates to All other.

(b)
2011 includes $1,509 related to a previously disclosed government investigation and $400 for acquired in-process research and development. 2012, 2011 and 2010 includes acquisition consideration payable of $400 related to the acquisition of Piramal Healthcare Limited's Healthcare Solutions business.

 	2012	 	2011	 	2010

 	(dollars in millions)

Post-employment Obligations and Other Long-term Liabilities:


Defined benefit pension plans and post-employment medical and dental plans for significant plans

 	$	4,557	 	$	3,301	 	$	2,425
Deferred income taxes

 	 	710	 	 	703	 	 	1,112
All other (c)

 	 	3,789	 	 	4,227	 	 	4,486

Total

 	$	9,056	 	$	8,231	 	$	8,023

(c)
2012, 2011 and 2010 includes acquisition consideration payable of $385, $770 and $1,150, respectively, related to the acquisition of Piramal Healthcare Limited's Healthcare Solutions business.
        The judgment entered by the U.S. District Court for the Eastern District of Texas against Abbott in its litigation with New York University and Centocor, Inc. required Abbott to secure the judgment in the event that its appeal to the Federal Circuit court was unsuccessful in overturning the district court's decision. In the first quarter of 2010, Abbott deposited $1.87 billion with an escrow agent and considered these assets to be restricted. On February 23, 2011, the Federal Circuit reversed the district court's final judgment and found Centocor's patent invalid. On April 25, 2011 Centocor petitioned the Federal Circuit to rehear and reconsider the decision. In June 2011 the Federal Circuit denied Centocor's petition and the restrictions on the funds were lifted.

62


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 3 — Financial Instruments, Derivatives and Fair Value Measures

        Certain Abbott foreign subsidiaries enter into foreign currency forward exchange contracts to manage exposures to changes in foreign exchange rates for anticipated intercompany purchases by those subsidiaries whose functional currencies are not the U.S. dollar. These contracts, totaling $1.6 billion at December 31, 2012 and 2011 and $1.3 billion at December 31, 2010 are designated as cash flow hedges of the variability of the cash flows due to changes in foreign exchange rates and are recorded at fair value. Accumulated gains and losses as of December 31, 2012 will be included in Cost of products sold at the time the products are sold, generally through the next twelve months. The amount of hedge ineffectiveness was not significant in 2012, 2011 and 2010.

        Abbott enters into foreign currency forward exchange contracts to manage currency exposures for foreign currency denominated third-party trade payables and receivables, and for intercompany loans and trade accounts payable where the receivable or payable is denominated in a currency other than the functional currency of the entity. For intercompany loans, the contracts require Abbott to sell or buy foreign currencies, primarily European currencies and Japanese yen, in exchange for primarily U.S. dollars and other European currencies. For intercompany and trade payables and receivables, the currency exposures are primarily the U.S. dollar, European currencies and Japanese yen. At December 31, 2012, 2011 and 2010, Abbott held $18.2 billion, $15.7 billion and $10.8 billion, respectively, of such foreign currency forward exchange contracts.

        Abbott has designated foreign denominated short-term debt as a hedge of the net investment in a foreign subsidiary of approximately $615 million, $680 million and $650 million as of December 31, 2012, 2011 and 2010, respectively. Accordingly, changes in the fair value of this debt due to changes in exchange rates are recorded in Accumulated other comprehensive income (loss), net of tax.

        Abbott is a party to interest rate hedge contracts totaling $9.5 billion, $6.8 billion and $7.3 billion at December 31, 2012, 2011 and 2010, respectively, to manage its exposure to changes in the fair value of fixed-rate debt. These contracts are designated as fair value hedges of the variability of the fair value of fixed-rate debt due to changes in the long-term benchmark interest rates. The effect of the hedge is to change a fixed-rate interest obligation to a variable rate for that portion of the debt. Abbott records the contracts at fair value and adjusts the carrying amount of the fixed-rate debt by an offsetting amount. No hedge ineffectiveness was recorded in income in 2012, 2011 and 2010 for these hedges.

        Gross unrealized holding gains on available-for-sale equity securities totaled $51 million, $64 million and $40 million at December 31, 2012, 2011 and 2010, respectively.

63



Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 3 — Financial Instruments, Derivatives and Fair Value Measures (Continued)

        The following table summarizes the amounts and location of certain derivative financial instruments as of December 31:


 	Fair Value — Assets	 	Fair Value — Liabilities

 	2012	 	2011	 	2010	 	Balance Sheet Caption	 	2012	 	2011	 	2010	 	Balance Sheet Caption

 	(dollars in millions)
Interest rate swaps designated as fair value hedges

 	$


  185
  	 	$


  598
  	 	$


  138
  	 	Deferred income taxes and other assets	 	$


  80
  	 	$


  —
  	 	$


  36
  	 	Post-employment obligations and other long-term liabilities
Foreign currency forward exchange contracts —


Hedging instruments

 	 	22
  	 	 	115
  	 	 	16
  	 	Other prepaid expenses and receivables	 	 	11
  	 	 	2
  	 	 	10
  	 	Other accrued liabilities
Others not designated as hedges

 	 	98	 	 	165	 	 	109	 	 	 	 	135	 	 	179	 	 	120
Debt designated as a hedge of net investment in a foreign subsidiary

 	 	—	 	 	—	 	 	—	 	n/a	 	 	615	 	 	680	 	 	650	 	Short-term borrowings


 	$	305	 	$	878	 	$	263	 	 	 	$	841	 	$	861	 	$	816

        The following table summarizes the activity for foreign currency forward exchange contracts designated as cash flow hedges, debt designated as a hedge of net investment in a foreign subsidiary and the amounts and location of income (expense) and gain (loss) reclassified into income and for certain other derivative financial instruments. The amount of hedge ineffectiveness was not significant in 2012, 2011 and 2010 for these hedges.


 	Gain (loss) Recognized in
Other Comprehensive
Income (loss)	 	Income (expense) and
Gain (loss) Reclassified
into Income

 	2012	 	2011	 	2010	 	2012	 	2011	 	2010	 	Income Statement Caption

 	(dollars in millions)

Foreign currency forward exchange contracts designated as cash flow hedges

 	$	2	 	$	65	 	$	170	 	$	138	 	$	(26	)	$	63	 	Cost of products sold
Debt designated as a hedge of net investment in a foreign subsidiary

 	 	65	 	 	(30	)	 	(75	)	 	—	 	 	—	 	 	—	 	n/a
Interest rate swaps designated as fair value hedges

 	 	n/a	 	 	n/a	 	 	n/a	 	 	62	 	 	488	 	 	248	 	Interest expense
Foreign currency forward exchange contracts not designated as hedges

 	 	n/a	 	 	n/a	 	 	n/a	 	 	108	 	 	(11	)	 	155	 	Net foreign exchange (gain) loss
        The interest rate swaps are designated as fair value hedges of the variability of the fair value of fixed-rate debt due to changes in the long-term benchmark interest rates. The hedged debt is marked to market, offsetting the effect of marking the interest rate swaps to market.

        The carrying values and fair values of certain financial instruments as of December 31 are shown in the table below. The carrying values of all other financial instruments approximate their estimated fair

64


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 3 — Financial Instruments, Derivatives and Fair Value Measures (Continued)

values. The counterparties to financial instruments consist of select major international financial institutions. Abbott does not expect any losses from nonperformance by these counterparties.


 	2012	 	2011	 	2010

 	Carrying
Value	 	Fair
Value	 	Carrying
Value	 	Fair
Value	 	Carrying
Value	 	Fair
Value

 	(dollars in millions)

Long-term Investment Securities:


Equity securities

 	$	213	 	$	213	 	$	317	 	$	317	 	$	240	 	$	240
Other

 	 	61	 	 	56	 	 	61	 	 	42	 	 	62	 	 	43
Total Long-term Debt

 	 	(18,394	)	 	(19,588	)	 	(13,067	)	 	(15,129	)	 	(14,568	)	 	(15,723	)
Foreign Currency Forward Exchange Contracts:


Receivable position

 	 	120	 	 	120	 	 	280	 	 	280	 	 	125	 	 	125
(Payable) position

 	 	(146	)	 	(146	)	 	(181	)	 	(181	)	 	(130	)	 	(130	)
Interest Rate Hedge Contracts:


Receivable position

 	 	185	 	 	185	 	 	598	 	 	598	 	 	146	 	 	146
(Payable) position

 	 	(80	)	 	(80	)	 	—	 	 	—	 	 	(36	)	 	(36	)
65


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 3 — Financial Instruments, Derivatives and Fair Value Measures (Continued)

        The following table summarizes the bases used to measure certain assets and liabilities at fair value on a recurring basis in the balance sheet:



 	Basis of Fair Value Measurement

 	Outstanding
Balances	 	Quoted
Prices in
Active Markets	 	Significant Other
Observable
Inputs	 	Significant
Unobservable
Inputs

 	(dollars in millions)

December 31, 2012:


Equity securities

 	$	76	 	$	76	 	$	—	 	$	—
Interest rate swap financial instruments

 	 	185	 	 	—	 	 	185	 	 	—
Foreign currency forward exchange contracts

 	 	120	 	 	—	 	 	120	 	 	—

Total Assets

 	$	381	 	$	76	 	$	305	 	$	—

Fair value of hedged long-term debt

 	$	9,632	 	$	—	 	$	9,632	 	$	—
Interest rate swap financial instruments

 	 	80	 	 	—	 	 	80	 	 	—
Foreign currency forward exchange contracts

 	 	146	 	 	—	 	 	146	 	 	—
Contingent consideration related to business combinations

 	 	323	 	 	—	 	 	—	 	 	323

Total Liabilities

 	$	10,181	 	$	—	 	$	9,858	 	$	323

December 31, 2011


Equity securities

 	$	93	 	$	93	 	$	—	 	$	—
Interest rate swap financial instruments

 	 	598	 	 	—	 	 	598	 	 	—
Foreign currency forward exchange contracts

 	 	280	 	 	—	 	 	280	 	 	—

Total Assets

 	$	971	 	$	93	 	$	878	 	$	—

Fair value of hedged long-term debt

 	$	7,427	 	$	—	 	$	7,427	 	$	—
Foreign currency forward exchange contracts

 	 	181	 	 	—	 	 	181	 	 	—
Contingent consideration related to business combinations

 	 	423	 	 	—	 	 	—	 	 	423

Total Liabilities

 	$	8,031	 	$	—	 	$	7,608	 	$	423

December 31, 2010:


Equity securities

 	$	75	 	$	75	 	$	—	 	$	—
Interest rate swap financial instruments

 	 	146	 	 	—	 	 	146	 	 	—
Foreign currency forward exchange contracts

 	 	125	 	 	—	 	 	125	 	 	—

Total Assets

 	$	346	 	$	75	 	$	271	 	$	—

Fair value of hedged long-term debt

 	$	7,444	 	$	—	 	$	7,444	 	$	—
Interest rate swap financial instruments

 	 	36	 	 	—	 	 	36	 	 	—
Foreign currency forward exchange contracts

 	 	130	 	 	—	 	 	130	 	 	—
Contingent consideration related to business combinations

 	 	365	 	 	—	 	 	—	 	 	365

Total Liabilities

 	$	7,975	 	$	—	 	$	7,610	 	$	365

        The fair value of the debt was determined based on the face value of the debt adjusted for the fair value of the interest rate swaps, which is based on a discounted cash flow analysis. The fair value of the contingent consideration was determined based on an independent appraisal adjusted for the time value of money, exchange and other changes in fair value.

66



Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 4 — Post-Employment Benefits

        Retirement plans consist of defined benefit, defined contribution and medical and dental plans. Information for Abbott's major defined benefit plans and post-employment medical and dental benefit plans is as follows: (dollars in millions)


 	Defined Benefit Plans	 	Medical and Dental Plans

 	2012	 	2011	 	2010	 	2012	 	2011	 	2010
Projected benefit obligations, January 1

 	$	8,963	 	$	8,606	 	$	6,852	 	$	1,657	 	$	1,673	 	$	1,705
Service cost — benefits earned during the year

 	 	376	 	 	332	 	 	288	 	 	61	 	 	55	 	 	60
Interest cost on projected benefit obligations

 	 	447	 	 	446	 	 	421	 	 	81	 	 	88	 	 	101
Losses (gains), primarily changes in discount rates, plan design changes, law changes and differences between actual and estimated health care costs

 	 	1,412	 	 	608	 	 	565	 	 	148	 	 	(104	)	 	(153	)
Benefits paid

 	 	(302	)	 	(294	)	 	(289	)	 	(63	)	 	(62	)	 	(74	)
Acquisition of Solvay's pharmaceuticals business

 	 	—	 	 	—	 	 	1,045	 	 	—	 	 	—	 	 	28
Settlement

 	 	—	 	 	(776	)	 	—	 	 	—	 	 	—	 	 	—
Other, primarily foreign currency translation

 	 	108	 	 	41	 	 	(276	)	 	5	 	 	7	 	 	6

Projected benefit obligations, December 31

 	$	11,004	 	$	8,963	 	$	8,606	 	$	1,889	 	$	1,657	 	$	1,673

Plans' assets at fair value, January 1

 	$	6,961	 	$	7,451	 	$	5,812	 	$	389	 	$	396	 	$	341
Actual return on plans' assets

 	 	878	 	 	29	 	 	782	 	 	48	 	 	5	 	 	55
Company contributions

 	 	379	 	 	394	 	 	525	 	 	40	 	 	40	 	 	74
Benefits paid

 	 	(302	)	 	(294	)	 	(289	)	 	(60	)	 	(52	)	 	(74	)
Acquisition of Solvay's pharmaceuticals business

 	 	—	 	 	—	 	 	763	 	 	—	 	 	—	 	 	—
Settlement

 	 	—	 	 	(776	)	 	—	 	 	—	 	 	—	 	 	—
Other, primarily foreign currency translation

 	 	33	 	 	157	 	 	(142	)	 	—	 	 	—	 	 	—

Plans' assets at fair value, December 31

 	$	7,949	 	$	6,961	 	$	7,451	 	$	417	 	$	389	 	$	396

Projected benefit obligations greater than plans' assets, December 31

 	$	(3,055	)	$	(2,002	)	$	(1,155	)	$	(1,472	)	$	(1,268	)	$	(1,277	)

Long-term assets

 	$	69	 	$	66	 	$	27	 	$	—	 	$	—	 	$	—
Short-term liabilities

 	 	(39	)	 	(35	)	 	(34	)	 	—	 	 	—	 	 	—
Long-term liabilities

 	 	(3,085	)	 	(2,033	)	 	(1,148	)	 	(1,472	)	 	(1,268	)	 	(1,277	)

Net liability

 	$	(3,055	)	$	(2,002	)	$	(1,155	)	$	(1,472	)	$	(1,268	)	$	(1,277	)

Amounts Recognized in Accumulated Other Comprehensive Income (loss):


Actuarial losses, net

 	$	4,742	 	$	3,822	 	$	2,879	 	$	701	 	$	601	 	$	713
Prior service cost (credits)

 	 	71	 	 	25	 	 	30	 	 	(322	)	 	(364	)	 	(406	)

Total

 	$	4,813	 	$	3,847	 	$	2,909	 	$	379	 	$	237	 	$	307

        The projected benefit obligations for non-U.S. defined benefit plans was $3.1 billion, $2.3 billion and $3.0 billion at December 31, 2012, 2011 and 2010, respectively. The accumulated benefit obligations for all defined benefit plans was $9.4 billion, $7.7 billion and $7.5 billion at December 31, 2012, 2011 and 2010, respectively. For plans where the accumulated benefit obligations exceeded plan assets at December 31, 2012, 2011 and 2010, the aggregate accumulated benefit obligations were $7.9 billion, $6.7 billion and $2.0 billion, respectively; the projected benefit obligations were $9.3 billion, $7.9 billion and $2.2 billion, respectively; and the aggregate plan assets were $6.2 billion, $5.8 billion and $1.1 billion, respectively.

        In connection with the separation of AbbVie from Abbott on January 1, 2013, Abbott will transfer certain liabilities and assets of both defined benefit pension plans and medical and dental plans. The

67


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 4 — Post-Employment Benefits (Continued)

estimated amount of the accumulated benefit obligations, projected benefit obligations, fair value of assets and deferred gains and losses to be assumed by AbbVie are $3.9 billion, $4.5 billion, $3.1 billion and $1.9 billion, respectively, for defined benefit plans. The estimated amount of the accumulated benefit obligations and deferred gains and losses to be assumed by AbbVie are $501 million and $114 million, respectively, for medical and dental plans.

        During 2011, $776 million of assets and liabilities of a plan sponsored by Abbott Healthcare BV, a Dutch subsidiary of Abbott Laboratories, were irrevocably transferred to a Dutch insurance company in full settlement of that plan. The assets were used to purchase an annuity contract to fulfill the plan's obligations.


 	Defined Benefit Plans	 	Medical and
Dental Plans

 	2012	 	2011	 	2010	 	2012	 	2011	 	2010

 	(dollars in millions)

Service cost — benefits earned during the year

 	$	376	 	$	332	 	$	288	 	$	61	 	$	55	 	$	60
Interest cost on projected benefit obligations

 	 	447	 	 	446	 	 	421	 	 	81	 	 	88	 	 	101
Expected return on plans' assets

 	 	(611	)	 	(608	)	 	(571	)	 	(33	)	 	(34	)	 	(31	)
Settlement

 	 	—	 	 	40	 	 	—	 	 	—	 	 	—	 	 	—
Amortization of actuarial losses

 	 	235	 	 	163	 	 	136	 	 	34	 	 	38	 	 	38
Amortization of prior service cost (credits)

 	 	4	 	 	4	 	 	4	 	 	(42	)	 	(42	)	 	(22	)

Total cost

 	$	451	 	$	377	 	$	278	 	$	101	 	$	105	 	$	146

        Other comprehensive income (loss) for 2012 includes amortization of actuarial losses and prior service cost of $235 million and $4 million, respectively, and net actuarial losses of $1.2 billion for defined benefit plans and amortization of actuarial losses and prior service credits of $34 million and $42 million, respectively, and net actuarial losses of $134 million for medical and dental plans. Other comprehensive income (loss) for 2011 includes amortization of actuarial losses and prior service cost of $163 million and $4 million, respectively, and net actuarial losses of $1.1 billion for defined benefit plans and amortization of actuarial losses and prior service credits of $38 million and $42 million, respectively, and net actuarial gains of $66 million for medical and dental plans. Other comprehensive income (loss) for 2010 includes amortization of actuarial losses and prior service cost of $136 million and $4 million, respectively, and net actuarial losses of $305 million for defined benefit plans and amortization of actuarial losses and prior service credits of $38 million and $22 million, respectively, and net actuarial gains of $177 million for medical and dental plans. The pretax amount of actuarial losses and prior service cost (credits) included in Accumulated other comprehensive income (loss) at December 31, 2012 that is expected to be recognized in the net periodic benefit cost in 2013 is $175 million and $5 million, respectively, for defined benefit pension plans and $25 million and $(23) million, respectively, for medical and dental plans.

        The weighted average assumptions used to determine benefit obligations for defined benefit plans and medical and dental plans are as follows:


 	2012	 	2011	 	2010
Discount rate

 	 	4.3	%	 	5.0	%	 	5.4	%
Expected aggregate average long-term change in compensation

 	 	5.3	%	 	5.3	%	 	5.1	%
68


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 4 — Post-Employment Benefits (Continued)

        The weighted average assumptions used to determine the net cost for defined benefit plans and medical and dental plans are as follows:


 	2012	 	2011	 	2010
Discount rate

 	 	5.0	%	 	5.4	%	 	5.8	%
Expected return on plan assets

 	 	8.0	%	 	7.8	%	 	7.8	%
Expected aggregate average long-term change in compensation

 	 	5.3	%	 	5.1	%	 	4.9	%
        The assumed health care cost trend rates for medical and dental plans at December 31 were as follows:


 	2012	 	2011	 	2010
Health care cost trend rate assumed for the next year

 	 	7	%	 	7	%	 	7	%
Rate that the cost trend rate gradually declines to

 	 	5	%	 	5	%	 	5	%
Year that rate reaches the assumed ultimate rate

 	 	2019	 	 	2019	 	 	2016
        The discount rates used to measure liabilities were determined based on high-quality fixed income securities that match the duration of the expected retiree benefits. The health care cost trend rates represent Abbott's expected annual rates of change in the cost of health care benefits and is a forward projection of health care costs as of the measurement date. A one-percentage point increase/(decrease) in the assumed health care cost trend rate would increase/(decrease) the accumulated post-employment benefit obligations as of December 31, 2012, by $274 million /$(222) million, and the total of the service and interest cost components of net post-employment health care cost for the year then ended by approximately $24 million /$(19) million.

69


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 4 — Post-Employment Benefits (Continued)

        The following table summarizes the bases used to measure defined benefit plans' assets at fair value:



 	Basis of Fair Value Measurement

 	Outstanding
Balances	 	Quoted
Prices in
Active Markets	 	Significant Other
Observable
Inputs	 	Significant
Unobservable
Inputs

 	(dollars in millions)

December 31, 2012:


Equities:


U.S. large cap (a)

 	$	1,731	 	$	1,731	 	$	—	 	$	—
U.S. mid cap (b)

 	 	461	 	 	140	 	 	321	 	 	—
International (c)

 	 	1,558	 	 	677	 	 	881	 	 	—
Fixed income securities:


U.S. government securities (d)

 	 	843	 	 	545	 	 	298	 	 	—
Corporate debt instruments (e)

 	 	704	 	 	427	 	 	277	 	 	—
Non-U.S. government securities (f)

 	 	373	 	 	101	 	 	272	 	 	—
Other (g)

 	 	23	 	 	18	 	 	5	 	 	—
Absolute return funds (h)

 	 	1,941	 	 	425	 	 	825	 	 	691
Commodities (i)

 	 	208	 	 	9	 	 	161	 	 	38
Other (j)

 	 	107	 	 	104	 	 	—	 	 	3


 	$	7,949	 	$	4,177	 	$	3,040	 	$	732

December 31, 2011:


Equities:


U.S. large cap (a)

 	$	1,470	 	$	1,449	 	$	21	 	$	—
U.S. mid cap (b)

 	 	423	 	 	152	 	 	271	 	 	—
International (c)

 	 	1,217	 	 	485	 	 	732	 	 	—
Fixed income securities:


U.S. government securities (d)

 	 	857	 	 	370	 	 	487	 	 	—
Corporate debt instruments (e)

 	 	527	 	 	223	 	 	304	 	 	—
Non-U.S. government securities (f)

 	 	450	 	 	228	 	 	222	 	 	—
Other (g)

 	 	45	 	 	21	 	 	24	 	 	—
Absolute return funds (h)

 	 	1,709	 	 	334	 	 	751	 	 	624
Commodities (i)

 	 	183	 	 	8	 	 	165	 	 	10
Other (j)

 	 	80	 	 	78	 	 	—	 	 	2


 	$	6,961	 	$	3,348	 	$	2,977	 	$	636

December 31, 2010:


Equities:


U.S. large cap (a)

 	$	1,523	 	$	1,499	 	$	24	 	$	—
U.S. mid cap (b)

 	 	437	 	 	162	 	 	275	 	 	—
International (c)

 	 	1,552	 	 	758	 	 	794	 	 	—
Fixed income securities:


U.S. government securities (d)

 	 	793	 	 	355	 	 	438	 	 	—
Corporate debt instruments (e)

 	 	524	 	 	237	 	 	286	 	 	1
Non-U.S. government securities (f)

 	 	758	 	 	172	 	 	586	 	 	—
Other (g)

 	 	40	 	 	20	 	 	19	 	 	1
Absolute return funds (h)

 	 	1,426	 	 	258	 	 	582	 	 	586
Commodities (i)

 	 	242	 	 	5	 	 	234	 	 	3
Other (j)

 	 	156	 	 	156	 	 	—	 	 	—


 	$	7,451	 	$	3,622	 	$	3,238	 	$	591

(a)
A mix of index funds that track the S&P 500 (50 percent in 2012 and 45 percent in 2011 and 2010) and separate actively managed equity accounts that are benchmarked to the Russell 1000 (50 percent in 2012 and 55 percent in 2011 and 2010).
70


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 4 — Post-Employment Benefits (Continued)

(b)
A mix of index funds (75 percent) and separate actively managed equity accounts (25 percent) that track or are benchmarked to the S&P 400 midcap index.

(c)
Primarily separate actively managed pooled investment accounts that are benchmarked to the MSCI and MSCI emerging market indices.

(d)
Index funds not actively managed (50 percent in 2012 and 45 percent in 2011 and 2010) and separate actively managed accounts (50 percent in 2012 and 55 percent in 2011 and 2010).

(e)
Index funds not actively managed (20 percent in 2012, 40 percent in 2011 and 15 percent in 2010) and separate actively managed accounts (80 percent in 2012, 60 percent in 2011 and 85 percent in 2010).

(f)
Primarily United Kingdom, Japan and Irish government-issued bonds.

(g)
Primarily mortgage backed securities.

(h)
Primarily funds invested by managers that have a global mandate with the flexibility to allocate capital broadly across a wide range of asset classes and strategies including, but not limited to equities, fixed income, commodities, interest rate futures, currencies and other securities to outperform an agreed upon benchmark with specific return and volatility targets.

(i)
Primarily investments in liquid commodity future contracts and private energy funds.

(j)
Primarily cash and cash equivalents.
        Equities that are valued using quoted prices are valued at the published market prices. Equities in a common collective trust or a registered investment company that are valued using significant other observable inputs are valued at the net asset value (NAV) provided by the fund administrator. The NAV is based on the value of the underlying assets owned by the fund minus its liabilities. Fixed income securities that are valued using significant other observable inputs are valued at prices obtained from independent financial service industry-recognized vendors. Absolute return funds and commodities are valued at the NAV provided by the fund administrator. Private energy funds are valued at the NAV provided by the partnership on a one-quarter lag adjusted for known cash flows and significant events through the reporting date.

        The following table summarizes the change in the value of assets that are measured using significant unobservable inputs:


 	2012	 	2011	 	2010

 	(dollars in millions)

January 1

 	$	636	 	$	591	 	$	530
Transfers in (out of) from other categories

 	 	2	 	 	(1	)	 	(37	)
Actual return on plan assets:


Assets on hand at year end

 	 	59	 	 	(14	)	 	41
Assets sold during the year

 	 	(4	)	 	(1	)	 	(2	)
Purchases, sales and settlements, net

 	 	39	 	 	61	 	 	59

December 31

 	$	732	 	$	636	 	$	591

        The investment mix of equity securities, fixed income and other asset allocation strategies is based upon achieving a desired return, balancing higher return, more volatile equity securities, and lower return, less volatile fixed income securities. Investment allocations are made across a range of markets, industry

71


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 4 — Post-Employment Benefits (Continued)

sectors, capitalization sizes, and in the case of fixed income securities, maturities and credit quality. The plans do not directly hold any securities of Abbott. There are no known significant concentrations of risk in the plans' assets. Abbott's medical and dental plans' assets are invested in a similar mix as the pension plan assets.

        The plans' expected return on assets, as shown above is based on management's expectations of long-term average rates of return to be achieved by the underlying investment portfolios. In establishing this assumption, management considers historical and expected returns for the asset classes in which the plans are invested, as well as current economic and capital market conditions.

        Abbott funds its domestic pension plans according to IRS funding limitations. International pension plans are funded according to similar regulations. Abbott funded $379 million in 2012, $394 million in 2011 and $525 million in 2010 to defined pension plans. Abbott expects pension funding for its main domestic pension plan of $170 million in 2013. The projected decrease reflects the separation of AbbVie from Abbott and the transfer of certain assets and liabilities to AbbVie.

        Total benefit payments expected to be paid to participants, giving effect to the separation of AbbVie from Abbott, which includes payments funded from company assets as well as paid from the plans, are as follows: (dollars in millions)


 	Defined
Benefit Plans	 	Medical and
Dental Plans
2013

 	$	173	 	$	78
2014

 	 	183	 	 	80
2015

 	 	197	 	 	83
2016

 	 	211	 	 	87
2017

 	 	224	 	 	90
2018 to 2022

 	 	1,367	 	 	510
        The Abbott Stock Retirement Plan is the principal defined contribution plan. Abbott's contributions to this plan were $150 million in 2012, $151 million in 2011 and $147 million in 2010.

        Abbott provides certain other post-employment benefits, primarily salary continuation plans, to qualifying domestic employees, and accrues for the related cost over the service lives of the employees.

Note 5 — Taxes on Earnings

        Taxes on earnings reflect the annual effective rates, including charges for interest and penalties. Deferred income taxes reflect the tax consequences on future years of differences between the tax bases of assets and liabilities and their financial reporting amounts. The $620 million domestic loss before taxes in 2012 includes Abbott's $1.35 billion net loss on the early extinguishment of debt and approximately $395 million of separation related expenses. U.S. income taxes are provided on those earnings of foreign subsidiaries which are intended to be remitted to the parent company. Abbott does not record deferred income taxes on earnings reinvested indefinitely in foreign subsidiaries. Undistributed earnings reinvested indefinitely in foreign subsidiaries as working capital and plant and equipment aggregated $40.0 billion at December 31, 2012. It is not practicable to determine the amount of deferred income taxes not provided on these earnings. In the U.S., Abbott's federal income tax returns through 2009 are settled except for one item, and the income tax returns for years after 2009 are open. There are numerous other income tax jurisdictions for which tax returns are not yet settled, none of which are individually significant. Reserves for interest and penalties are not significant.

72



Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 5 — Taxes on Earnings (Continued)

        Earnings before taxes, and the related provisions for taxes on earnings, were as follows: (dollars in millions)


 	2012	 	2011	 	2010
Earnings Before Taxes:


Domestic

 	$	(620	)	$	364	 	$	(275	)
Foreign

 	 	6,883	 	 	4,835	 	 	5,988

Total

 	$	6,263	 	$	5,199	 	$	5,713




 	2012	 	2011	 	2010
Taxes on Earnings:


Current:


Domestic

 	$	198	 	$	(586	)	$	1,462
Foreign

 	 	1,230	 	 	1,187	 	 	835

Total current

 	 	1,428	 	 	601	 	 	2,297

Deferred:


Domestic

 	 	(483	)	 	162	 	 	(1,068	)
Foreign

 	 	(645	)	 	(293	)	 	(142	)

Total deferred

 	 	(1,128	)	 	(131	)	 	(1,210	)

Total

 	$	300	 	$	470	 	$	1,087

        Differences between the effective income tax rate and the U.S. statutory tax rate were as follows:


 	2012	 	2011	 	2010
Statutory tax rate on earnings

 	 	35.0	%	 	35.0	%	 	35.0	%
Benefit of lower foreign tax rates and tax exemptions

 	 	(24.9	)	 	(22.9	)	 	(19.4	)
Resolution of certain tax positions pertaining to prior years

 	 	(6.5	)	 	(11.2	)	 	—
Effect of non-deductible litigation reserve

 	 	0.6	 	 	9.1	 	 	—
State taxes, net of federal benefit

 	 	0.1	 	 	(0.4	)	 	0.4
All other, net

 	 	0.5	 	 	(0.6	)	 	3.0

Effective tax rate on earnings

 	 	4.8	%	 	9.0	%	 	19.0	%

        As of December 31, 2012, 2011 and 2010, total deferred tax assets were $7.4 billion, $6.3 billion and $6.1 billion, respectively, and total deferred tax liabilities were $2.6 billion, $2.9 billion and $3.0 billion, respectively. Abbott has incurred losses in a foreign jurisdiction where realization of the future economic benefit is so remote that the benefit is not reflected as a deferred tax asset. Valuation allowances for

73


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 5 — Taxes on Earnings (Continued)

recorded deferred tax assets were not significant. The tax effect of the differences that give rise to deferred tax assets and liabilities were as follows: (dollars in millions)


 	2012	 	2011	 	2010
Compensation and employee benefits

 	$	1,936	 	$	1,658	 	$	1,327
Trade receivable reserves

 	 	557	 	 	492	 	 	525
Inventory reserves

 	 	211	 	 	212	 	 	293
Deferred intercompany profit

 	 	1,095	 	 	711	 	 	255
State income taxes

 	 	197	 	 	227	 	 	233
Depreciation

 	 	(75	)	 	(164	)	 	(64	)
Acquired in-process research and development and other accruals and reserves not currently deductible

 	 	3,278	 	 	2,886	 	 	3,401
Other, primarily the excess of book basis over tax basis of intangible assets

 	 	(2,447	)	 	(2,636	)	 	(2,905	)

Total

 	$	4,752	 	$	3,386	 	$	3,065

        The following table summarizes the gross amounts of unrecognized tax benefits without regard to reduction in tax liabilities or additions to deferred tax assets and liabilities if such unrecognized tax benefits were settled. (dollars in millions)


 	2012	 	2011	 	2010
January 1

 	$	2,123	 	$	2,724	 	$	2,172
Increase due to current year tax positions

 	 	673	 	 	588	 	 	635
Increase due to prior year tax positions

 	 	62	 	 	282	 	 	171
Decrease due to prior year tax positions

 	 	(438	)	 	(824	)	 	(94	)
Settlements

 	 	(163	)	 	(647	)	 	(160	)

December 31

 	$	2,257	 	$	2,123	 	$	2,724

        The total amount of unrecognized tax benefits that, if recognized, would impact the effective tax rate is approximately $2.0 billion. Abbott believes that it is reasonably possible that the recorded amount of gross unrecognized tax benefits may decrease by $550 million to $650 million, including cash adjustments, within the next twelve months as a result of concluding various domestic and international tax matters.

Note 6 — Segment and Geographic Area Information

        Abbott's principal business is the discovery, development, manufacture and sale of a broad line of health care products. Abbott's products are generally sold directly to retailers, wholesalers, hospitals, health care facilities, laboratories, physicians' offices and government agencies throughout the world. Effective January 1, 2012, certain international operations were transferred from the Established Pharmaceutical Products segment to the Proprietary Pharmaceutical Products segment. The segment information below has been adjusted to reflect this reorganization. Abbott's reportable segments are as follows:

        Proprietary Pharmaceutical Products —  Worldwide sales of a broad line of proprietary pharmaceutical products.

74


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 6 — Segment and Geographic Area Information (Continued)

        Established Pharmaceutical Products —  International sales of a broad line of branded generic pharmaceutical products.

        Nutritional Products —  Worldwide sales of a broad line of adult and pediatric nutritional products.

        Diagnostic Products —  Worldwide sales of diagnostic systems and tests for blood banks, hospitals, commercial laboratories and alternate-care testing sites. For segment reporting purposes, the Core Laboratories Diagnostics, Molecular Diagnostics, Point of Care and Ibis diagnostic divisions are aggregated and reported as the Diagnostic Products segment.

        Vascular Products —  Worldwide sales of coronary, endovascular, structural heart, vessel closure and other medical device products.

        Non-reportable segments include the Diabetes Care and Medical Optics segments.

        Abbott's underlying accounting records are maintained on a legal entity basis for government and public reporting requirements. Segment disclosures are on a performance basis consistent with internal management reporting. Intersegment transfers of inventory are recorded at standard cost and are not a measure of segment operating earnings. The cost of some corporate functions and the cost of certain employee benefits are charged to segments at predetermined rates that approximate cost. Remaining costs, if any, are not allocated to segments. For acquisitions prior to 2006, substantially all intangible assets and related amortization are not allocated to segments. In addition, no intangible assets or related amortization are allocated to the Established Pharmaceutical Products segment. The following segment information has been prepared in accordance with the internal accounting policies of Abbott, as described above, and are not presented in accordance with generally accepted accounting principles applied to the consolidated financial statements. (dollars in millions)


 	Net Sales to External Customers (a)	 	Operating
Earnings (a)	 	Depreciation
and Amortization	 	Additions to
Long-term Assets	 	Total Assets

 	2012	 	2011	 	2010	 	2012	 	2011	 	2010	 	2012	 	2011	 	2010	 	2012	 	2011	 	2010	 	2012	 	2011	 	2010
Proprietary Pharmaceuticals

 	$	18,012	 	$	17,080	 	$	15,389	 	$	7,948	 	$	7,202	 	$	6,592	 	$	622	 	$	639	 	$	553	 	$	256	 	$	168	 	$	2,779	 	$	12,026	 	$	10,974	 	$	11,421
Established Pharmaceuticals (b)

 	 	5,121	 	 	5,355	 	 	4,461	 	 	1,237	 	 	1,254	 	 	938	 	 	156	 	 	169	 	 	148	 	 	237	 	 	183	 	 	2,804	 	 	5,704	 	 	6,986	 	 	6,730
Nutritionals

 	 	6,471	 	 	6,006	 	 	5,532	 	 	1,019	 	 	797	 	 	777	 	 	191	 	 	183	 	 	177	 	 	458	 	 	205	 	 	163	 	 	3,583	 	 	3,241	 	 	3,244
Diagnostics

 	 	4,292	 	 	4,126	 	 	3,794	 	 	804	 	 	766	 	 	559	 	 	315	 	 	339	 	 	244	 	 	349	 	 	409	 	 	319	 	 	3,907	 	 	3,429	 	 	3,462
Vascular

 	 	3,071	 	 	3,333	 	 	3,194	 	 	902	 	 	980	 	 	910	 	 	195	 	 	233	 	 	252	 	 	69	 	 	148	 	 	528	 	 	5,301	 	 	5,272	 	 	5,390

Total Reportable Segments

 	 	36,967	 	 	35,900	 	 	32,370	 	$	11,910	 	$	10,999	 	$	9,776	 	$	1,479	 	$	1,563	 	$	1,374	 	$	1,369	 	$	1,113	 	$	6,593	 	$	30,521	 	$	29,902	 	$	30,247

Other

 	 	2,907	 	 	2,951	 	 	2,797

Net Sales

 	$	39,874	 	$	38,851	 	$	35,167

(a)
Net sales and operating earnings were unfavorably affected by the relatively stronger U.S. dollar in 2012 and were favorably affected by the relatively weaker U.S. dollar in 2011 and 2010.
75


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 6 — Segment and Geographic Area Information (Continued)

(b)
Additions to long-term assets in 2010 for the Established Pharmaceutical Products segment include goodwill of $2,797.

 	2012	 	2011	 	2010

 	(dollars in millions)


Total Reportable Segment Operating Earnings

 	$	11,910	 	$	10,999	 	$	9,776

Corporate functions and benefit plans costs

 	 	(651	)	 	(529	)	 	(558	)

Non-reportable segments

 	 	335	 	 	276	 	 	139

Net interest expense

 	 	(513	)	 	(445	)	 	(448	)

Net loss on extinguishment of debt

 	 	(1,351	)	 	—	 	 	—

Acquired in-process and collaborations research and development

 	 	(288	)	 	(673	)	 	(313	)

Share-based compensation

 	 	(433	)	 	(383	)	 	(387	)

Other, net (c)

 	 	(2,746	)	 	(4,046	)	 	(2,496	)


Consolidated Earnings Before Taxes

 	$	6,263	 	$	5,199	 	$	5,713

(c)
Other, net, for 2011 includes a charge of $1,509 related to a previously disclosed government investigation. Other, net, for 2012, 2011 and 2010 includes charges of $1,309, $402 and $881, respectively, for separation related costs in 2012 and for cost reduction initiatives and integration.

 	2012	 	2011	 	2010

 	(dollars in millions)


Total Reportable Segment Assets

 	$	30,521	 	$	29,902	 	$	30,247

Cash, investments and restricted funds

 	 	15,448	 	 	8,476	 	 	7,626

Current deferred income taxes

 	 	2,986	 	 	2,701	 	 	3,076

Non-reportable segments

 	 	4,413	 	 	4,173	 	 	5,385

All other, net, primarily goodwill and intangible assets not allocated to reportable segments

 	 	13,867	 	 	15,025	 	 	14,240


Total Assets

 	$	67,235	 	$	60,277	 	$	60,574

76


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 6 — Segment and Geographic Area Information (Continued)




 	Net Sales to External
Customers (d)	 	Long-term Assets

 	2012	 	2011	 	2010	 	2012	 	2011	 	2010

 	(dollars in millions)


United States

 	$	16,784	 	$	16,014	 	$	15,194	 	$	15,244	 	$	15,867	 	$	16,769

Japan

 	 	2,441	 	 	2,342	 	 	2,025	 	 	1,169	 	 	1,225	 	 	1,172

Germany

 	 	1,740	 	 	1,759	 	 	1,846	 	 	6,173	 	 	5,909	 	 	5,950

The Netherlands

 	 	1,883	 	 	2,108	 	 	2,001	 	 	532	 	 	462	 	 	312

Italy

 	 	1,127	 	 	1,189	 	 	1,144	 	 	222	 	 	229	 	 	242

Canada

 	 	1,253	 	 	1,098	 	 	1,036	 	 	352	 	 	237	 	 	224

France

 	 	1,167	 	 	1,297	 	 	1,216	 	 	220	 	 	214	 	 	87

Spain

 	 	942	 	 	1,063	 	 	1,066	 	 	314	 	 	293	 	 	291

United Kingdom

 	 	1,049	 	 	971	 	 	888	 	 	1,345	 	 	1,273	 	 	1,272

India

 	 	933	 	 	931	 	 	501	 	 	3,467	 	 	3,160	 	 	3,791

All Other Countries

 	 	10,555	 	 	10,079	 	 	8,250	 	 	6,874	 	 	7,639	 	 	8,146


Consolidated

 	$	39,874	 	$	38,851	 	$	35,167	 	$	35,912	 	$	36,508	 	$	38,256

(d)
Sales by country are based on the country that sold the product.
Note 7 — Litigation and Environmental Matters

        Abbott has been identified as a potentially responsible party for investigation and cleanup costs at a number of locations in the United States and Puerto Rico under federal and state remediation laws and is investigating potential contamination at a number of company-owned locations. Abbott has recorded an estimated cleanup cost for each site for which management believes Abbott has a probable loss exposure. No individual site cleanup exposure is expected to exceed $4 million, and the aggregate cleanup exposure is not expected to exceed $15 million.

        There are a number of patent disputes with third parties who claim Abbott's products infringe their patents. On February 21, 2012, the United States Supreme Court denied Centocor Inc.'s and New York University's petition to review a February 2011 Federal Circuit Court of Appeals decision reversing a $1.67 billion judgment in favor of Centocor and New York University on a patent they claimed Abbott's HUMIRA infringed. This decision concludes the case.

        The United States Department of Justice, through the United States Attorney for the Western District of Virginia, and various state Attorneys General investigated Abbott's sales and marketing activities for Depakote. The government sought to determine whether any of these activities violated civil and/or criminal laws, including the Federal False Claims Act, the Food, Drug and Cosmetic Act, and the Anti-Kickback Statute in connection with Medicare and/or Medicaid reimbursement to third parties. The state Attorneys General offices sought to determine whether any of these activities violated various state laws, including state consumer fraud/protection statutes. Abbott recorded charges of $1.5 billion in the third quarter of 2011 and $100 million in the first quarter of 2012 related to civil and criminal claims arising from this matter. In May 2012, Abbott reached resolution of all Depakote-related federal claims, Medicaid-related claims with 49 states and the District of Columbia, and consumer protection claims with 45 states and the District of Columbia. In 2012, Abbott paid approximately $1.6 billion for the settlement. The payments were material to Abbott's cash flows in 2012.

77


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 7 — Litigation and Environmental Matters (Continued)

        Abbott estimates the range of possible loss for its legal proceedings and environmental exposures to be from approximately $70 million to $100 million. The recorded accrual balance at December 31, 2012 for these proceedings and exposures was approximately $80 million. This accrual represents management's best estimate of probable loss, as defined by FASB ASC No. 450, "Contingencies." Within the next year, legal proceedings may occur that may result in a change in the estimated loss accrued by Abbott. While it is not feasible to predict the outcome of all such proceedings and exposures with certainty, management believes that their ultimate disposition should not have a material adverse effect on Abbott's financial position, cash flows, or results of operations.

Note 8 — Incentive Stock Program

        The 2009 Incentive Stock Program authorizes the granting of nonqualified stock options, replacement stock options, restricted stock awards, restricted stock units, performance awards, foreign benefits and other share-based awards. Stock options, replacement stock options and restricted stock awards and units comprise the majority of benefits that have been granted and are currently outstanding under this program and a prior program. In 2012, Abbott granted 1,931,213 stock options, 2,124,743 replacement stock options, 1,134,062 restricted stock awards and 7,056,609 restricted stock units under this program. The purchase price of shares under option must be at least equal to the fair market value of the common stock on the date of grant, and the maximum term of an option is 10 years. Options vest equally over three years except for replacement options, which vest in six months. Options granted before January 1, 2005 included a replacement feature. Except for options outstanding that have a replacement feature, options granted after December 31, 2004 do not include a replacement feature. When an employee tenders mature shares to Abbott upon exercise of a stock option, a replacement stock option may be granted equal to the amount of shares tendered. Replacement options are granted at the then current market price for a term that expires on the date of the underlying option grant.

        Upon a change in control of Abbott, all outstanding stock options become fully exercisable, and all terms and conditions of all restricted stock awards and units are deemed satisfied. Restricted stock awards generally vest between 3 and 5 years and for restricted stock awards that vest over 5 years, no more than one-third of the award vests in any one year upon Abbott reaching a minimum return on equity target. Restricted stock units vest over three years and upon vesting, the recipient receives one share of Abbott stock for each vested restricted stock unit. The aggregate fair market value of restricted stock awards and units is recognized as expense over the service period. Restricted stock awards and settlement of vested restricted stock units are issued out of treasury shares. Abbott generally issues new shares for exercises of stock options. Abbott does not have a policy of purchasing its shares relating to its share-based programs.

        At December 31, 2012, approximately 155 million shares were reserved for future grants. Subsequent to year-end, the reserve was reduced by approximately 25 million shares for stock options and restricted stock awards and units granted by the Board of Directors. In connection with the separation of AbbVie from Abbott on January 1, 2013, Abbott employees, including those employees transferring to AbbVie, holding stock options or restricted stock awards or units as of December 31, 2012 generally received one AbbVie stock option for each Abbott stock option held and one AbbVie restricted stock award or unit for each Abbott award or unit held. For Abbott stock options, the exercise price of an Abbott option was adjusted to reflect the effect of the separation. The per share data presented below has not been adjusted to reflect this adjustment on the per share amounts.

78


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 8 — Incentive Stock Program (Continued)

        The number of restricted stock awards and units outstanding and the weighted-average grant-date fair value at December 31, 2011 and December 31, 2012 was 14,698,595 and $50.29 and 15,506,416 and $53.17, respectively. The number of restricted stock awards and units, and the weighted-average grant-date fair value, that were granted, vested and lapsed during 2012 were 8,190,671 and $56.74, 6,774,145 and $51.32 and 608,705 and $52.32, respectively. The fair market value of restricted stock awards and units vested in 2012, 2011 and 2010 was $385 million, $237 million and $203 million, respectively.


 	Options Outstanding	 	Exercisable Options

 	Shares	 	Weighted
Average
Exercise
Price	 	Weighted
Average
Remaining
Life (Years)	 	Shares	 	Weighted
Average
Exercise
Price	 	Weighted
Average
Remaining
Life (Years)
December 31, 2011

 	 	85,439,279	 	$	50.52	 	 	4.7	 	 	81,734,460	 	$	50.51	 	 	4.5

Granted

 	 	4,055,956	 	 	60.91
Exercised

 	 	(40,923,624	)	 	49.73
Lapsed

 	 	(380,717	)	 	60.63

December 31, 2012

 	 	48,190,894	 	$	51.98	 	 	4.0	 	 	43,052,057	 	$	51.36	 	 	3.7

        The aggregate intrinsic value of options outstanding and exercisable at December 31, 2012 was $679 million and $633 million, respectively. The total intrinsic value of options exercised in 2012, 2011 and 2010 was $528 million, $94 million and $77 million, respectively. The total unrecognized compensation cost related to all share-based compensation plans at December 31, 2012 amounted to approximately $174 million, giving effect to the separation of AbbVie from Abbott, which is expected to be recognized over the next three years.

        Total non-cash compensation expense charged against income in 2012, 2011 and 2010 for share-based plans totaled approximately $433 million, $383 million and $385 million, respectively, and the tax benefit recognized was approximately $132 million, $116 million and $119 million, respectively. Compensation cost capitalized as part of inventory is not significant.

        The fair value of an option granted in 2012, 2011 and 2010 was $6.80, $6.23, and $9.24, respectively. The fair value of an option grant was estimated using the Black-Scholes option-pricing model with the following assumptions:


 	2012	 	2011	 	2010
Risk-free interest rate

 	 	1.2	%	 	2.7	%	 	2.9	%
Average life of options (years)

 	 	6.0	 	 	6.0	 	 	6.0
Volatility

 	 	21.0	%	 	21.0	%	 	22.0	%
Dividend yield

 	 	3.6	%	 	4.1	%	 	3.2	%
        The risk-free interest rate is based on the rates available at the time of the grant for zero-coupon U.S. government issues with a remaining term equal to the option's expected life. The average life of an option is based on both historical and projected exercise and lapsing data. Expected volatility is based on implied volatilities from traded options on Abbott's stock and historical volatility of Abbott's stock over the expected life of the option. Dividend yield is based on the option's exercise price and annual dividend rate at the time of grant.

79


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 9 — Debt and Lines of Credit

        The following is a summary of long-term debt at December 31: (dollars in millions)


 	2012	 	2011	 	2010
5.15% Notes, due 2012

 	$	—	 	$	—	 	$	1,000
1.95% Yen Notes, due 2013

 	 	—	 	 	321	 	 	299
4.35% Notes, due 2014

 	 	—	 	 	500	 	 	500
1.2% Notes, due 2015 (1)

 	 	3,500	 	 	—	 	 	—
Variable Rate Notes, due 2015 (1)

 	 	500	 	 	—	 	 	—
2.7% Notes, due 2015

 	 	—	 	 	750	 	 	750
5.875% Notes, due 2016

 	 	—	 	 	2,000	 	 	2,000
1.75% Notes, due 2017 (1)

 	 	4,000	 	 	—	 	 	—
5.6% Notes, due 2017

 	 	—	 	 	1,500	 	 	1,500
2.0% Notes, due 2018 (1)

 	 	1,000	 	 	—	 	 	—
5.125% Notes, due 2019

 	 	947	 	 	2,000	 	 	2,000
4.125% Notes, due 2020

 	 	597	 	 	1,000	 	 	1,000
2.9% Notes, due 2022 (1)

 	 	3,100	 	 	—	 	 	—
6.15% Notes, due 2037

 	 	547	 	 	1,000	 	 	1,000
6.0% Notes, due 2039

 	 	515	 	 	1,000	 	 	1,000
5.3% Notes, due 2040

 	 	694	 	 	1,250	 	 	1,250
4.4% Notes, due 2042 (1)

 	 	2,600	 	 	—	 	 	—
Other, including fair value adjustments relating to interest rate hedge contracts designated as fair value hedges

 	 	85	 	 	719	 	 	225

Total, net of current maturities

 	 	18,085	 	 	12,040	 	 	12,524
Current maturities of long-term debt

 	 	309	 	 	1,027	 	 	2,045

Total carrying amount

 	$	18,394	 	$	13,067	 	$	14,569

(1)
These notes were issued by AbbVie Inc. in November 2012. With the separation of AbbVie on January 1, 2013, Abbott no longer has any obligations related to this debt.
        In 2012, Abbott redeemed $7.7 billion of its outstanding notes. Abbott incurred a cost of $1.35 billion to extinguish this debt, net of gains from the unwinding of interest rate swaps related to the debt. In 2012, AbbVie Inc., a wholly owned subsidiary of Abbott, issued $14.7 billion of long-term debt with maturities ranging from 3 to 30 years. The debt issued by AbbVie Inc. was guaranteed by Abbott with the guarantee expiring when AbbVie Inc. separated from Abbott on January 1, 2013.

        After the separation of AbbVie from Abbott on January 1, 2013, principal payments required on long-term debt outstanding and retained by Abbott are $309 million in 2013 and $3.3 billion in 2019 and thereafter.

        At December 31, 2012, Abbott's long-term debt rating was A+ by Standard & Poor's Corporation and A1 by Moody's Investors Service. In the third quarter 2012, Abbott replaced unused lines of credit of $3.0 billion and $3.7 billion that were to expire in October 2012 and in 2013, respectively, with two five-year credit facilities totaling $7.0 billion that support commercial paper borrowing arrangements. One of the credit facilities totaling $2.0 billion will support AbbVie commercial paper borrowings after separation and expired for Abbott at the separation of AbbVie from Abbott on January 1, 2013. Abbott's weighted-average interest rate on short-term borrowings was 0.4% at December 31, 2012, 2011 and 2010.

80



Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 10 — Business Combinations, Technology Acquisitions and Related Transactions

        On September 8, 2010, Abbott acquired Piramal Healthcare Limited's Healthcare Solutions business, a leader in the Indian branded generics market, for $2.2 billion, in cash, plus additional payments of $400 million annually in 2011, 2012, 2013 and 2014. Abbott recorded a $1.6 billion liability for the present value of the additional payments at the acquisition date. The acquisition was financed with cash. The allocation of the fair value of the acquisition resulted in the recording of $2.7 billion of deductible acquired intangible assets and $1.0 billion of deductible goodwill. Acquired intangible assets consist primarily of trade names, customer relationships and associated rights and are amortized over an average of 19 years.

        In February 2010, Abbott acquired Solvay's pharmaceuticals business (Solvay Pharmaceuticals) for approximately $6.1 billion, in cash, plus additional payments of up to EUR 100 million per year if certain sales milestones are met in 2011, 2012 and 2013. Contingent consideration of approximately $290 million was recorded. The acquisition of Solvay Pharmaceuticals provided Abbott with a large and complementary portfolio of pharmaceutical products and expands Abbott's presence in key global emerging markets. Abbott acquired control of this business on February 15, 2010 and the financial results of the acquired operations are included in these financial statements beginning on that date. Net sales for the acquired operations for 2010 were approximately $3.1 billion. Pretax loss of the acquired operations, including acquisition, integration and restructuring expenses, for 2010 was approximately $395 million. The acquisition was funded with cash and short-term investments. The allocation of the fair value of the acquisition resulted in the recording of $2.2 billion of non-deductible goodwill, $4.1 billion of non-deductible intangible assets, $500 million of non-deductible acquired in-process research and development assets, net tangible assets of $700 million and deferred income taxes of $1.1 billion. Acquired intangible assets consist primarily of product rights for currently marketed products and are amortized over 2 to 14 years (average of 11 years). Acquired in-process research and development projects are accounted for as indefinite lived intangible assets until regulatory approval or discontinuation. The net tangible assets acquired consist primarily of trade accounts receivable of approximately $675 million, inventory of approximately $390 million, property and equipment of approximately $725 million, net of assumed liabilities, primarily trade accounts payable, accrued compensation and other liabilities.

        Had the acquisition of Solvay Pharmaceuticals taken place on January 1, 2010, unaudited pro forma net sales, net earnings and diluted earnings per share for 2010 would have been $35.8 billion, $4.6 billion and $2.96, respectively. The pro forma information includes adjustments for amortization of intangible assets and fair value adjustments to acquisition-date inventory as well as acquisition, integration and restructuring expenses. The pro forma financial information is not necessarily indicative of the results of operations as they would have been had the transaction been effected on the assumed date.

        In March 2010, Abbott acquired STARLIMS Technologies for approximately $100 million, in cash, net of cash held by STARLIMS, providing Abbott with leading products and expertise to build its position in laboratory informatics. A substantial portion of the fair value of the acquisition has been allocated to goodwill and amortizable intangible assets. In April 2010, Abbott acquired the outstanding shares of Facet Biotech Corporation for approximately $430 million, in cash, net of cash held by Facet. The acquisition enhanced Abbott's early- and mid-stage pharmaceutical pipeline, including a biologic for multiple sclerosis and compounds that complement Abbott's oncology program. A substantial portion of the fair value of the acquisition was allocated to acquired in-process research and development that is accounted for as an indefinite-lived intangible asset until regulatory approval or discontinuation.

81


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 10 — Business Combinations, Technology Acquisitions and Related Transactions (Continued)

        Except for the acquisition of Solvay Pharmaceuticals, had the above acquisitions taken place on January 1 of the previous year, consolidated net sales and income would not have been significantly different from reported amounts.

        Abbott's Proprietary Pharmaceutical Products segment has entered into various collaboration research and development agreements. In 2012, Abbott acquired AP214, a drug under development for the prevention of acute kidney injury associated with major cardiac surgery in patients at increased risk, and as a result of this transaction, Abbott recorded a charge to acquired in-process and collaborations research and development of $110 million. In addition, in 2012, Abbott entered into a global collaboration to develop and commercialize an oral, next-generation JAK1 inhibitor in Phase II development with the potential to treat multiple autoimmune diseases, and as a result of this transaction Abbott recorded a charge to acquired in-process and collaborations research and development of $150 million. Additional payments of approximately $1.2 billion could be required for the achievement of certain development, regulatory and commercial milestones under this agreement. Under another collaboration, Abbott was granted the rights in 2012 to utilize up to three antibody-drug conjugate compounds and Abbott recorded a charge to acquired in-process and collaborations research and development of $28 million. Additional payments of approximately $220 million for each licensed compound could be required for the achievement of certain development, regulatory and commercial milestones under this agreement. In connection with the acquisition of Solvay Pharmaceuticals, the achievement of a certain sales milestone resulted in a payment of approximately $134 million in the first quarter of 2012 for which a liability was previously established.

        During 2010 and 2011, Abbott entered into a series of transactions with Reata Pharmaceuticals which included (1) a collaboration agreement for the joint development and commercialization of second generation oral antioxidant inflammation modulators resulting in a charge to acquired in-process and collaborations research and development of $400 million in 2011, (2) an agreement to acquire licensing rights outside the U.S., excluding certain Asian markets, to bardoxolone methyl, a product in development for the treatment of chronic kidney disease resulting in a charge to acquired in-process and collaborations research and development of $238 million in 2010 and (3) the acquisition of equity interests in Reata of $62 million each in 2011 and 2010. In 2011, certain milestones were achieved in the development for the treatment of chronic kidney disease and charges to acquired in-process and collaborations research and development of $188 million were recorded. In the first quarter of 2012, $50 million of research and development expense was recorded related to the achievement of a clinical development milestone under the license agreement. The license agreement requires additional payments of up to $150 million if certain development and regulatory milestones associated with the chronic kidney disease compound are achieved.

        On October 17, 2012 Reata informed Abbott that it is discontinuing the Phase III clinical study for bardoxolone methyl for chronic kidney disease. Reata and Abbott will closely examine the data from this study to determine whether there is an appropriate path forward for the development of bardoxolone methyl in chronic kidney disease or other indications. In the fourth quarter of 2012, Abbott recorded a charge of approximately $50 million for the impairment of the equity investment in Reata.

        In 2011, Abbott entered into an agreement with Biotest AG to develop and commercialize a treatment for rheumatoid arthritis and psoriasis resulting in a charge to acquired in-process and collaborations research and development of $85 million. Additional payments totaling up to $395 million based on projected regulatory approval timelines could be required for the achievement of certain development, regulatory and commercial milestones under this agreement. In 2010, Abbott entered into an

82


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 10 — Business Combinations, Technology Acquisitions and Related Transactions (Continued)

agreement with Neurocrine Biosciences to develop and commercialize a product for the treatment of endometriosis resulting in a charge to acquired in-process and collaborations research and development of $75 million. Additional payments of approximately $500 million could be required for the achievement of certain development, regulatory and commercial milestones under this agreement.

Note 11 — Goodwill and Intangible Assets

        Abbott recorded goodwill of approximately $3.4 billion in 2010 related to the acquisitions of Solvay's pharmaceuticals business, Piramal Healthcare Limited's Healthcare Solutions business, Facet Biotech and STARLIMS Technologies. Goodwill related to the Solvay, Piramal and Facet acquisitions was allocated to the pharmaceutical products segments. In addition, in 2010, Abbott paid $250 million to Boston Scientific as a result of the approval to market the Xience V drug-eluting stent in Japan, resulting in an increase in goodwill in the Vascular Products segment. Foreign currency translation and other adjustments increased (decreased) goodwill in 2012, 2011 and 2010 by $69 million, $(225) million and $(879) million, respectively. The amount of goodwill related to reportable segments at December 31, 2012 was $6.3 billion for the Proprietary Pharmaceutical Products segment, $3.0 billion for the Established Pharmaceutical Products segment, $209 million for the Nutritional Products segment, $385 million for the Diagnostic Products segment, and $2.7 billion for the Vascular Products segment. There were no significant reductions of goodwill relating to impairments or disposal of all or a portion of a business.

        The gross amount of amortizable intangible assets, primarily product rights and technology was $17.6 billion, $17.5 billion and $17.3 billion as of December 31, 2012, 2011 and 2010, respectively, and accumulated amortization was $9.7 billion, $8.3 billion and $6.5 billion as of December 31, 2012, 2011 and 2010, respectively. Indefinite-lived intangible assets, which relate to in-process research and development acquired in a business combination, were approximately $691 million, $814 million and $1.4 billion at December 31, 2012, 2011 and 2010, respectively. In 2012 and 2011, Abbott recorded impairment charges of $82 million and $174 million, respectively, for certain research and development assets due to changes in the projected development and regulatory timelines for the projects. The 2012 charge relates to a non-reportable segment and in 2011, $125 million related to a non-reportable segment and $49 million related to the Other category in Abbott's segment reporting. Discounted cash flow analysis was used to analyze fair value and the charges are included in research and development expenses. The estimated annual amortization expense for intangible assets recorded at December 31, 2012, adjusted for the separation of AbbVie from Abbott, is approximately $800 million in 2013, $675 million in 2014, $590 million in 2015, $605 million in 2016 and $565 million in 2017. Intangible asset amortization is included in Cost of products sold in the consolidated statement of earnings. Amortizable intangible assets are amortized over 2 to 30 years (average 11 years).

Note 12 — Restructuring Plans

        In 2012, Abbott management approved plans to streamline various commercial operations in order to reduce costs and improve efficiencies in Abbott's core diagnostics, established pharmaceutical and nutritionals businesses. Abbott recorded employee related severance charges of approximately $167 million in 2012. Additional charges of approximately $22 million were also recorded in 2012, primarily for asset impairments. Approximately $70 million is recorded in Cost of products sold and approximately $119 million as Selling, general and administrative expense. As of December 31, 2012, no significant cash payments have been made relating to these actions.

83


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 12 — Restructuring Plans (Continued)

        In 2011 and prior years, Abbott management approved plans to realign its worldwide pharmaceutical and vascular manufacturing operations and selected domestic and international commercial and research and development operations in order to reduce costs. In 2011 and 2010, Abbott recorded charges of approximately $194 million and $56 million, respectively, reflecting the impairment of manufacturing facilities and other assets, employee severance and other related charges. Approximately $76 million in 2011 is classified as Cost of products sold, $69 million as Research and development and $49 million as Selling, general and administrative. Approximately $56 million in 2010 is classified as Cost of products sold. The following summarizes the activity for these restructurings: (dollars in millions)

Accrued balance at January 1, 2010

 	$	145
2010 restructuring charges

 	 	56
Payments, impairments and other adjustments

 	 	(124	)

Accrued balance at December 31, 2010

 	 	77
2011 restructuring charges

 	 	194
Payments, impairments and other adjustments

 	 	(94	)

Accrued balance at December 31, 2011

 	 	177
Payments and other adjustments

 	 	(48	)

Accrued balance at December 31, 2012

 	$	129

        An additional $110 million, $25 million and $13 million were recorded in 2012, 2011 and 2010, respectively, relating to these restructurings, primarily for accelerated depreciation.

        In 2012 and 2010, Abbott management approved restructuring plans primarily related to the acquisition of Solvay's pharmaceuticals business. These plans streamline operations, improve efficiencies and reduce costs in certain Solvay sites and functions as well as in certain Abbott and Solvay commercial organizations in various countries. In 2012, Abbott recorded a charge of approximately $150 million for employee severance and contractual obligations, primarily related to the exit from a research and development facility. Approximately $142 million is recorded as Research and development and $8 million as Selling, general and administrative. In 2010, Abbott recorded charges to Cost of products sold, Research and development and Selling, general and administrative of approximately $99 million, $152 million and $272 million, respectively. The following summarizes the activity for these restructurings: (dollars in millions)

2010 restructuring charge

 	$	523
Payments, impairments and other adjustments

 	 	(113	)

Accrued balance at December 31, 2010

 	 	410
Payments and other adjustments

 	 	(302	)

Accrued balance at December 31, 2011

 	 	108
Restructuring charges

 	 	150
Payments and other adjustments

 	 	(143	)

Accrued balance at December 31, 2012

 	$	115

84


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 12 — Restructuring Plans (Continued)

        An additional $38 million, $102 million and $12 million were recorded in 2012, 2011 and 2010, respectively, relating to these restructurings, primarily for additional employee severance and accelerated depreciation.

        In 2011 and 2008, Abbott management approved plans to streamline global manufacturing operations, reduce overall costs, and improve efficiencies in Abbott's core diagnostic business. In 2011, a charge of $28 million was recorded in Cost of products sold. The following summarizes the activity for these restructurings: (dollars in millions)

Accrued balance at January 1, 2010

 	$	98
Payments and other adjustments

 	 	(10	)

Accrued balance at December 31, 2010

 	 	88
2011 restructuring charge

 	 	28
Payments and other adjustments

 	 	(37	)

Accrued balance at December 31, 2011

 	 	79
Payments and other adjustments

 	 	(23	)

Accrued balance at December 31, 2012

 	$	56

        In addition, charges of approximately $16 million, $42 million and $60 million were recorded in 2012, 2011 and 2010, primarily for accelerated depreciation and product transfer costs.

85


Abbott Laboratories and Subsidiaries

Notes to Consolidated Financial Statements (Continued)

Note 13 — Quarterly Results (Unaudited)

(dollars in millions except per share data)


 	2012	 	2011	 	2010
First Quarter


Net Sales

 	$	9,456.6	 	$	9,040.9	 	$	7,698.4
Gross Profit

 	 	5,731.7	 	 	5,181.9	 	 	4,363.2
Net Earnings

 	 	1,242.1	 	 	863.8	 	 	1,003.0
Basic Earnings Per Common Share (a)

 	 	.79	 	 	.56	 	 	.65
Diluted Earnings Per Common Share (a)

 	 	.78	 	 	.55	 	 	.64
Market Price Per Share-High

 	 	61.49	 	 	49.45	 	 	56.79
Market Price Per Share-Low

 	 	53.96	 	 	45.07	 	 	52.21
Second Quarter


Net Sales

 	$	9,807.1	 	$	9,616.3	 	$	8,826.0
Gross Profit

 	 	6,169.8	 	 	5,745.8	 	 	5,282.1
Net Earnings

 	 	1,724.6	 	 	1,942.8	 	 	1,291.7
Basic Earnings Per Common Share (a)

 	 	1.09	 	 	1.24	 	 	.83
Diluted Earnings Per Common Share (a)

 	 	1.08	 	 	1.23	 	 	.83
Market Price Per Share-High

 	 	64.47	 	 	54.24	 	 	53.25
Market Price Per Share-Low

 	 	59.04	 	 	49.05	 	 	45.26
Third Quarter


Net Sales

 	$	9,773.3	 	$	9,816.7	 	$	8,674.5
Gross Profit

 	 	6,075.2	 	 	5,843.4	 	 	4,933.4
Net Earnings

 	 	1,942.8	 	 	303.2	 	 	890.7
Basic Earnings Per Common Share (a)

 	 	1.22	 	 	.19	 	 	.58
Diluted Earnings Per Common Share (a)

 	 	1.21	 	 	.19	 	 	.57
Market Price Per Share-High

 	 	70.41	 	 	53.60	 	 	52.86
Market Price Per Share-Low

 	 	63.51	 	 	46.29	 	 	44.59
Fourth Quarter


Net Sales

 	$	10,836.9	 	$	10,377.4	 	$	9,967.8
Gross Profit

 	 	6,777.5	 	 	6,539.6	 	 	5,922.8
Net Earnings

 	 	1,053.4	 	 	1,618.7	 	 	1,440.8
Basic Earnings Per Common Share (a)

 	 	.66	 	 	1.03	 	 	.93
Diluted Earnings Per Common Share (a)

 	 	.66	 	 	1.02	 	 	.92
Market Price Per Share-High

 	 	72.47	 	 	56.44	 	 	53.75
Market Price Per Share-Low

 	 	62.62	 	 	48.96	 	 	46.03
(a)
The sum of the quarters' basic earnings per share for 2011 and 2010 and diluted earnings per share for 2012 and 2011 do not add to the full year earnings per share amounts due to rounding.
86



Management Report on Internal Control
Over Financial Reporting

        The management of Abbott Laboratories is responsible for establishing and maintaining adequate internal control over financial reporting. Abbott's internal control system was designed to provide reasonable assurance to the company's management and board of directors regarding the preparation and fair presentation of published financial statements.

        All internal control systems, no matter how well designed, have inherent limitations. Therefore, even those systems determined to be effective can provide only reasonable assurance with respect to financial statement preparation and presentation.

        Abbott's management assessed the effectiveness of the company's internal control over financial reporting as of December 31, 2012. In making this assessment, it used the criteria set forth in Internal Control — Integrated Framework issued by the Committee of Sponsoring Organizations of the Treadway Commission. Based on our assessment, we believe that, as of December 31, 2012, the company's internal control over financial reporting was effective based on those criteria.

        Abbott's independent registered public accounting firm has issued an audit report on their assessment of the effectiveness of the company's internal control over financial reporting. This report appears on page 89.

Miles D. White
CHAIRMAN OF THE BOARD AND CHIEF EXECUTIVE OFFICER

Thomas C. Freyman
EXECUTIVE VICE PRESIDENT, FINANCE AND CHIEF FINANCIAL OFFICER

Greg W. Linder
VICE PRESIDENT AND CONTROLLER

February 15, 2013

87



Reports of Independent Registered Public Accounting Firm

To the Board of Directors and Shareholders of Abbott Laboratories:

        We have audited the accompanying consolidated balance sheets of Abbott Laboratories and subsidiaries (the "Company") as of December 31, 2012, 2011, and 2010, and the related consolidated statements of earnings, comprehensive income, shareholders' investment, and cash flows for the years then ended. These financial statements are the responsibility of the Company's management. Our responsibility is to express an opinion on these financial statements based on our audits.

        We conducted our audits in accordance with the standards of the Public Company Accounting Oversight Board (United States). Those standards require that we plan and perform the audit to obtain reasonable assurance about whether the financial statements are free of material misstatement. An audit includes examining, on a test basis, evidence supporting the amounts and disclosures in the financial statements. An audit also includes assessing the accounting principles used and significant estimates made by management, as well as evaluating the overall financial statement presentation. We believe that our audits provide a reasonable basis for our opinion.

        In our opinion, such consolidated financial statements present fairly, in all material respects, the financial position of the Company as of December 31, 2012, 2011, and 2010, and the results of its operations and its cash flows for the years then ended, in conformity with accounting principles generally accepted in the United States of America.

        As discussed in Note 1 to the consolidated financial statements, on January 1, 2013, the Company distributed all of the outstanding shares of AbbVie Inc., which encompasses the Company's research-based pharmaceuticals business, to the Company's shareholders. As also discussed in Note 1, in 2011 the Company changed the year end of its foreign subsidiaries from a November 30 fiscal year end to a December 31 calendar year end.

        We have also audited, in accordance with the standards of the Public Company Accounting Oversight Board (United States), the Company's internal control over financial reporting as of December 31, 2012, based on the criteria established in Internal Control — Integrated Framework issued by the Committee of Sponsoring Organizations of the Treadway Commission and our report dated February 15, 2013 expressed an unqualified opinion on the Company's internal control over financial reporting.

/s/ Deloitte & Touche LLP

Chicago, Illinois
February 15, 2013

88


To the Board of Directors and Shareholders of Abbott Laboratories:

        We have audited the internal control over financial reporting of Abbott Laboratories and subsidiaries (the "Company") as of December 31, 2012, based on criteria established in Internal Control — Integrated Framework issued by the Committee of Sponsoring Organizations of the Treadway Commission. The Company's management is responsible for maintaining effective internal control over financial reporting and for its assessment of the effectiveness of internal control over financial reporting, included in the accompanying Management Report on Internal Control over Financial Reporting. Our responsibility is to express an opinion on the Company's internal control over financial reporting based on our audit.

        We conducted our audit in accordance with the standards of the Public Company Accounting Oversight Board (United States). Those standards require that we plan and perform the audit to obtain reasonable assurance about whether effective internal control over financial reporting was maintained in all material respects. Our audit included obtaining an understanding of internal control over financial reporting, assessing the risk that a material weakness exists, testing and evaluating the design and operating effectiveness of internal control based on the assessed risk, and performing such other procedures as we considered necessary in the circumstances. We believe that our audit provides a reasonable basis for our opinion.

        A company's internal control over financial reporting is a process designed by, or under the supervision of, the company's principal executive and principal financial officers, or persons performing similar functions, and effected by the company's board of directors, management, and other personnel to provide reasonable assurance regarding the reliability of financial reporting and the preparation of financial statements for external purposes in accordance with generally accepted accounting principles. A company's internal control over financial reporting includes those policies and procedures that (1) pertain to the maintenance of records that, in reasonable detail, accurately and fairly reflect the transactions and dispositions of the assets of the company; (2) provide reasonable assurance that transactions are recorded as necessary to permit preparation of financial statements in accordance with generally accepted accounting principles, and that receipts and expenditures of the company are being made only in accordance with authorizations of management and directors of the company; and (3) provide reasonable assurance regarding prevention or timely detection of unauthorized acquisition, use, or disposition of the company's assets that could have a material effect on the financial statements.

        Because of the inherent limitations of internal control over financial reporting, including the possibility of collusion or improper management override of controls, material misstatements due to error or fraud may not be prevented or detected on a timely basis. Also, projections of any evaluation of the effectiveness of the internal control over financial reporting to future periods are subject to the risk that the controls may become inadequate because of changes in conditions, or that the degree of compliance with the policies or procedures may deteriorate.

        In our opinion, the Company maintained, in all material respects, effective internal control over financial reporting as of December 31, 2012, based on the criteria established in Internal Control — Integrated Framework issued by the Committee of Sponsoring Organizations of the Treadway Commission.

        We have also audited, in accordance with the standards of the Public Company Accounting Oversight Board (United States), the consolidated financial statements of the Company as of and for the year ended December 31, 2012 and our report dated February 15, 2013 expresses an unqualified opinion on those financial statements and includes an explanatory paragraph regarding the distribution of the shares of AbbVie Inc. to the Company's shareholders and the Company's change to the year end of its foreign subsidiaries.

/s/ Deloitte & Touche LLP

Chicago, Illinois
February 15, 2013

89


ITEM 9.    CHANGES IN AND DISAGREEMENTS WITH ACCOUNTANTS ON ACCOUNTING AND FINANCIAL DISCLOSURE

        As previously reported on Abbott's Current Report on Form 8-K, dated December 14, 2012, the Audit Committee of Abbott's Board of Directors approved the dismissal of Deloitte & Touche LLP (Deloitte) as Abbott's independent registered public accountant, effective as of the date of Deloitte's completion of the audit services for the fiscal year ending December 31, 2013 and the filing of Abbott's 2013 Annual Report on Securities and Exchange Commission Form 10-K, and approved the appointment of Ernst & Young LLP as Abbott's independent registered public accounting firm to perform independent audit services beginning with the fiscal year ending December 31, 2014.

        During the fiscal years ended December 31, 2012, 2011 and 2010, and through February 15, 2013, (i) there were no disagreements (as that term is defined in Item 304(a)(1)(iv) of Regulation S-K and the related instructions) between Abbott and Deloitte on any matter of accounting principles or practices, financial statement disclosure, or auditing scope or procedure, which, if not resolved to the satisfaction of Deloitte, would have caused Deloitte to make reference to the subject matter of the disagreement in connection with its reports on Abbott's consolidated financial statements for such years, and (ii) there were no "reportable events" (as that term is defined in Item 304(a)(1)(v) of Regulation S-K).

ITEM 9A.    CONTROLS AND PROCEDURES

Disclosure Controls and Procedures

        Evaluation of disclosure controls and procedures.    The Chief Executive Officer, Miles D. White, and the Chief Financial Officer, Thomas C. Freyman, evaluated the effectiveness of Abbott Laboratories' disclosure controls and procedures as of the end of the period covered by this report, and concluded that Abbott Laboratories' disclosure controls and procedures were effective to ensure that information Abbott is required to disclose in the reports that it files or submits with the Securities and Exchange Commission under the Securities Exchange Act of 1934 (the Exchange Act) is recorded, processed, summarized and reported, within the time periods specified in the Commission's rules and forms, and to ensure that information required to be disclosed by Abbott in the reports that it files or submits under the Exchange Act is accumulated and communicated to Abbott's management, including its principal executive officer and principal financial officer, as appropriate to allow timely decisions regarding required disclosure.

Internal Control Over Financial Reporting

        Management's annual report on internal control over financial reporting.    Management's report on Abbott's internal control over financial reporting is included on page 87 hereof. The report of Abbott's independent registered public accounting firm related to their assessment of the effectiveness of internal control over financial reporting is included on page 89 hereof.

        Changes in internal control over financial reporting.    During the quarter ended December 31, 2012, there were no changes in Abbott's internal control over financial reporting (as defined in Rule 13a-15(f) under the Exchange Act) that have materially affected, or are reasonably likely to materially affect, Abbott's internal control over financial reporting.

ITEM 9B.    OTHER INFORMATION

        None.

90



PART III

ITEM 10.    DIRECTORS, EXECUTIVE OFFICERS AND CORPORATE GOVERNANCE

        Incorporated herein by reference are "Information Concerning Nominees for Directors," "Committees of the Board of Directors," "Section 16(a) Beneficial Ownership Reporting Compliance," and "Procedure for Recommendation and Nomination of Directors and Transaction of Business at Annual Meeting" to be included in the 2013 Abbott Laboratories Proxy Statement. The 2013 Proxy Statement will be filed on or about March 15, 2013. Also incorporated herein by reference is the text found under the caption, "Executive Officers of the Registrant" on pages 21 through 23 hereof.

        Abbott has adopted a code of ethics that applies to its principal executive officer, principal financial officer, and principal accounting officer and controller. That code is part of Abbott's code of business conduct which is available free of charge through Abbott's investor relations website (www.abbottinvestor.com). Abbott intends to include on its website (www.abbott.com) any amendment to, or waiver from, a provision of its code of ethics that applies to Abbott's principal executive officer, principal financial officer, and principal accounting officer and controller that relates to any element of the code of ethics definition enumerated in Item 406(b) of Regulation S-K.

ITEM 11.    EXECUTIVE COMPENSATION

        The material to be included in the 2013 Proxy Statement under the headings "Director Compensation," and "Executive Compensation," and "Compensation Committee Report" is incorporated herein by reference. The 2013 Proxy Statement will be filed on or about March 15, 2013.

ITEM 12.    SECURITY OWNERSHIP OF CERTAIN BENEFICIAL OWNERS AND MANAGEMENT AND RELATED STOCKHOLDER MATTERS

(a)
Equity Compensation Plan Information.
        The following table presents information as of December 31, 2012 about our compensation plans under which Abbott common shares have been authorized for issuance.

Plan Category
 	(a)
Number of
securities to be
issued upon
exercise of
outstanding
options,
warrants and
rights	 	(b)
Weighted-
average exercise
price of
outstanding
options,
warrants and
rights1	 	(c)
Number of
securities
remaining
available for
future issuance
under equity
compensation
plans (excluding
securities
reflected in
column (a))
Equity compensation plans approved by security holders2

 	 	48,190,894	 	$	51.98	 	 	162,410,835
Equity compensation plans not approved by security holders

 	 	0	 	$	—	 	 	0
Total2

 	 	48,190,894	 	$	51.98	 	 	162,410,835
1.
Weighted-average exercise prices have not been adjusted to reflect the separation of AbbVie on January 1, 2013.

2.
(i) Abbott Laboratories 1996 Incentive Stock Program. Benefits under the 1996 Program include stock options intended to qualify for special tax treatment under Section 422 of the Internal Revenue Code ("incentive stock options"), stock options that do not qualify for that special tax treatment ("non-qualified stock options"), restricted stock, restricted stock units, stock appreciation rights, performance awards, and foreign qualified benefits. The shares that remain available for issuance under the 1996 Program may be issued in connection with any one of these benefits and may be either
91

authorized but unissued shares or treasury shares (except that restricted stock awards may be satisfied only from treasury shares).


If there is a lapse, expiration, termination, forfeiture or cancellation of any benefit granted under the 1996 Program without the issuance of shares or payment of cash thereunder, the shares subject to or reserved for that benefit, or so reacquired, may again be used for new stock options, rights, or awards of any type authorized under the 1996 Program. If shares are issued under any benefit under the 1996 Program and thereafter are reacquired by Abbott pursuant to rights reserved upon their issuance, or pursuant to the payment of the purchase price of shares under stock options by delivery of other common shares of Abbott, the shares subject to or reserved for that benefit, or so reacquired, may not again be used for new stock options, rights, or awards of any type authorized under the 1996 Program.



In April 2009, the 1996 Program was replaced by the Abbott Laboratories 2009 Incentive Stock Program. No further awards will be granted under the 1996 Program.


(ii) Abbott Laboratories 2009 Incentive Stock Program. Benefits under the 2009 Program include stock options that do not qualify for special tax treatment under Section 422 of the Internal Revenue Code ("non-qualified stock options"), restricted stock, restricted stock units, performance awards, other share-based awards (including stock appreciation rights, dividend equivalents and recognition awards), awards to non-employee directors, and foreign benefits. The shares that remain available for issuance under the 2009 Program may be issued in connection with any one of these benefits and may be either authorized but unissued shares or treasury shares (except that restricted stock awards may be satisfied only from treasury shares).


If there is a lapse, expiration, termination, forfeiture or cancellation of any benefit granted under the 2009 Program without the issuance of shares or payment of cash thereunder, the shares subject to or reserved for that benefit, or so reacquired, may again be used for new stock options, rights, or awards of any type authorized under the 2009 Program. If shares are issued under any benefit under the 2009 Program and thereafter are reacquired by Abbott pursuant to rights reserved upon their issuance, or pursuant to the payment of the purchase price of shares under stock options by delivery of other common shares of Abbott, the shares subject to or reserved for that benefit, or so reacquired, may not again be used for new stock options, rights, or awards of any type authorized under the 2009 Program.


(iii) Abbott Laboratories 2009 Employee Stock Purchase Plan for Non-U.S. Employees. Eligible employees of participating non-U.S. affiliates of Abbott may participate in this plan. An eligible employee may authorize payroll deductions at the rate of 1% to 10% of eligible compensation (in multiples of one percent) subject to a limit of US $12,500 during any purchase cycle.


Purchase cycles are generally six months long and usually begin on August 1 and February 1. On the last day of each purchase cycle, Abbott uses participant contributions to acquire Abbott common shares. The shares acquired come from treasury shares. The purchase price is 85% of the lower of the fair market value of the shares on that date or on the first day of that purchase cycle.


(iv) Advanced Medical Optics, Inc. Plans. In 2009, in connection with its acquisition of Advanced Medical Optics, Inc., Abbott assumed options outstanding under the Amended and Restated Advanced Medical Optics, Inc. 2002 Incentive Compensation Plan, as amended; AMO's 2004 Stock Incentive Plan, as amended and restated; the Advanced Medical Optics, Inc. 2005 Incentive Compensation Plan; the VISX, Incorporated 1995 Director Option and Stock Deferral Plan, as amended and restated; the VISX, Incorporated 1995 Stock Plan, as amended; the VISX, Incorporated 2000 Stock Plan; and the VISX, Incorporated 2001 Nonstatutory Stock Option Plan. As of December 31, 2012, 1,385,403 options remained outstanding under the plans. These options have a weighted average purchase price of $79.84. No further awards will be granted under the plans.
        For additional information concerning the Abbott Laboratories 1996 Incentive Stock Program, the Abbott Laboratories 2009 Incentive Stock Program, and the Abbott Laboratories 2009 Employee Stock

92

Purchase Plan for Non-U.S. Employees, see the discussion in Note 8 entitled "Incentive Stock Program" of the Notes to Consolidated Financial Statements included under Item 8, "Financial Statements and Supplementary Data."

(b)
Information Concerning Security Ownership.    Incorporated herein by reference is the material under the heading "Security Ownership of Executive Officers and Directors" in the 2013 Proxy Statement. The 2013 Proxy Statement will be filed on or about March 15, 2013.
ITEM 13.    CERTAIN RELATIONSHIPS AND RELATED TRANSACTIONS, AND DIRECTOR INDEPENDENCE

        The material to be included in the 2013 Proxy Statement under the headings "The Board of Directors," "Committees of the Board of Directors," "Corporate Governance Materials," and "Approval Process for Related Person Transactions" is incorporated herein by reference. The 2013 Proxy Statement will be filed on or about March 15, 2013.

ITEM 14.    PRINCIPAL ACCOUNTING FEES AND SERVICES

        The material to be included in the 2013 Proxy Statement under the headings "Audit Fees and Non-Audit Fees" and "Policy on Audit Committee Pre-Approval of Audit and Permissible Non-Audit Services of the Independent Auditor" is incorporated herein by reference. The 2013 Proxy Statement will be filed on or about March 15, 2013.

93



PART IV

ITEM 15.    EXHIBITS, FINANCIAL STATEMENT SCHEDULES

(a)
Documents filed as part of this Form 10-K.

(1)
Financial Statements:    See Item 8, "Financial Statements and Supplementary Data," on page 51 hereof, for a list of financial statements.

(2)
Financial Statement Schedules:    The required financial statement schedules are found on the pages indicated below. These schedules should be read in conjunction with the Consolidated Financial Statements of Abbott Laboratories:

Abbott Laboratories Financial Statement Schedules
 	Page No.

Valuation and Qualifying Accounts (Schedule II)

 	97

Schedules I, III, IV, and V are not submitted because they are not applicable or not required



Report of Independent Registered Public Accounting Firm

 	98

Individual Financial Statements of businesses acquired by the registrant have been omitted pursuant to Rule 3.05 of Regulation S-X


(3)
Exhibits Required by Item 601 of Regulation S-K:    The information called for by this paragraph is incorporated herein by reference to the Exhibit Index on pages 99 through 105 of this Form 10-K.

(b)
Exhibits filed (see Exhibit Index on pages 99 through 105).

(c)
Financial Statement Schedule filed (page 97).
94



SIGNATURES

        Pursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, Abbott Laboratories has duly caused this report to be signed on its behalf by the undersigned, thereunto duly authorized.

 	 	ABBOTT LABORATORIES



By

/s/ THOMAS C. FREYMAN
Thomas C. Freyman,
Executive Vice President, Finance
and Chief Financial Officer



Date: March 27, 2013
95

(This page has been left blank intentionally.)

96



ABBOTT LABORATORIES AND SUBSIDIARIES
SCHEDULE II VALUATION AND QUALIFYING ACCOUNTS
FOR THE YEARS ENDED DECEMBER 31, 2012, 2011, AND 2010
(in thousands of dollars)

Allowances for Doubtful
Accounts and Product Returns
 	Balance
at Beginning
of Year	 	Provisions/
Charges
to Income	 	Amounts
Charged Off
and Other
Deductions	 	Balance at
End of Year
2012

 	$	420,579	 	$	343,154	 	$	(357,812	)	$	405,921
2011

 	 	388,564	 	 	429,794	 	 	(397,779	)	 	420,579
2010

 	 	311,546	 	 	401,818	 	 	(324,800	)	 	388,564
97



REPORT OF INDEPENDENT REGISTERED PUBLIC ACCOUNTING FIRM

To the Board of Directors and Shareholders of Abbott Laboratories:

We have audited the consolidated financial statements of Abbott Laboratories and subsidiaries (the "Company") as of and for the years ended December 31, 2012, 2011, and 2010, and the Company's internal control over financial reporting as of December 31, 2012, and have issued our reports thereon dated February 15, 2013, which report relating to the consolidated financial statements expresses an unqualified opinion and includes an explanatory paragraph regarding the distribution of the shares of AbbVie Inc. to the Company's shareholders and the Company's change to the year end of its foreign subsidiaries; such reports are included elsewhere in this Form 10-K. Our audits also included the consolidated financial statement schedule of the Company listed in Item 15. This consolidated financial statement schedule is the responsibility of the Company's management. Our responsibility is to express an opinion based on our audits. In our opinion, such consolidated financial statement schedule, when considered in relation to the basic consolidated financial statements taken as a whole, presents fairly, in all material respects, the information set forth therein.

/s/ DELOITTE & TOUCHE LLP

Chicago, Illinois
February 15, 2013

98



EXHIBIT INDEX
ABBOTT LABORATORIES
ANNUAL REPORT
FORM 10-K
2012

        Exhibits 32.1 and 32.2 are furnished herewith and should not be deemed to be "filed under the Securities Exchange Act of 1934."

10-K
Exhibit
Table
Item No.

2.1


*Amendment No. 2 to Business Transfer Agreement dated January 29, 2011, by and among Abbott Healthcare Private Limited, Abbott Laboratories, Piramal Healthcare Limited ("Piramal") and certain shareholders of Piramal, filed as Exhibit 2 to the Abbott Laboratories Quarterly Report on Form 10-Q for the quarter ended March 31, 2011.




2.2




*Separation and Distribution Agreement, dated as of November 28, 2012, by and between Abbott Laboratories and AbbVie Inc.




Certain schedules and exhibits have been omitted from this filing pursuant to Item 601(b)(2) of Regulation S-K. Abbott will furnish supplemental copies of any such schedules or exhibits to the U.S. Securities and Exchange Commission upon request.




3.1




*Articles of Incorporation, Abbott Laboratories, filed as Exhibit 3.1 to the Abbott Laboratories Quarterly Report on Form 10-Q for the quarter ended March 31, 1998.




3.2




*By-Laws of Abbott Laboratories, as amended and restated effective April 27, 2012, filed as Exhibit 3.1 to the Abbott Laboratories Current Report on Form 8-K dated February 17, 2012.




4.1




*Indenture between AbbVie Inc. and U.S. Bank National Association, as Trustee, dated as of November 8, 2012, filed as Exhibit 4.1 to the Abbott Laboratories Current Report on Form 8-K dated November 8, 2012.




4.2




*Supplemental Indenture No. 1, dated as of November 8, 2012, to the Indenture dated as of November 8, 2012, between AbbVie Inc. and U.S. Bank National Association, as Trustee, dated as of November 8, 2012, filed as Exhibit 4.2 to the Abbott Laboratories Current Report on Form 8-K dated November 8, 2012.




4.3




*Guarantee by Abbott Laboratories in favor of U.S. Bank National Association as Trustee for the Holders of Certain Securities specified therein of AbbVie Inc., dated as of November 8, 2012, filed as Exhibit 4.3 to the Abbott Laboratories Current Report on Form 8-K dated November 8, 2012.




4.4




*Registration Rights Agreement, dated as of November 8, 2012, by and among AbbVie Inc., Abbott Laboratories, Morgan Stanley & Co. LLC, Barclays Capital Inc., J.P. Morgan Securities LLC and Merrill Lynch, Pierce, Fenner & Smith Incorporated as representatives of the Initial Purchasers, filed as Exhibit 4.4 to the Abbott Laboratories Current Report on Form 8-K dated November 8, 2012.




4.5




*Indenture dated as of February 9, 2001, between Abbott Laboratories and The Bank of New York Mellon Trust Company, N.A. (as successor to J.P. Morgan Trust Company, National Association, successor to Bank One Trust Company, N.A.) (including form of Security), filed as Exhibit 4.1 to the Abbott Laboratories Registration Statement on Form S-3 dated February 12, 2001.

99

10-K
Exhibit
Table
Item No.

4.6


*Supplemental Indenture dated as of February 27, 2006, between Abbott Laboratories and The Bank of New York Mellon Trust Company, N.A. (as successor to J.P. Morgan Trust Company, National Association), filed as Exhibit 4.2 to the Abbott Laboratories Registration Statement on Form S-3 dated February 28, 2006.




4.7




*Form of $1,000,000,000 6.150% Note due 2037, filed as Exhibit 99.6 to the Abbott Laboratories Current Report on Form 8-K dated November 6, 2007.




4.8




*Actions of the Authorized Officers with respect to Abbott's 5.150% Notes due 2012, 5.600% Notes due 2017 and 6.150% Notes due 2037, filed as Exhibit 99.3 to the Abbott Laboratories Current Report on Form 8-K dated November 6, 2007.




4.9




*Form of $2,000,000,000 5.125% Note due 2019, filed as Exhibit 99.4 to the Abbott Laboratories Current Report on Form 8-K dated February 26, 2009.




4.10




*Form of $1,000,000,000 6.000% Note due 2039, filed as Exhibit 99.5 to the Abbott Laboratories Current Report on Form 8-K dated February 26, 2009.




4.11




*Actions of the Authorized Officers with respect to Abbott's 5.125% Note due 2019 and 6.000% Note due 2039, filed as Exhibit 99.3 to the Abbott Laboratories Current Report on Form 8-K dated February 26, 2009.




4.12




*Form of 2020 Note, filed as Exhibit 99.5 to the Abbott Laboratories Current Report on Form 8-K dated May 27, 2010.




4.13




*Form of 2040 Note, filed as Exhibit 99.6 to the Abbott Laboratories Current Report on Form 8-K dated May 27, 2010.




4.14




*Actions of the Authorized Officers with respect to Abbott's 2.70% Notes, 4.125% Notes and 5.30% Notes, filed as Exhibit 99.3 to the Abbott Laboratories Current Report on Form 8-K dated May 27, 2010.




Other debt instruments are omitted in accordance with Item 601(b)(4)(iii)(A) of Regulation S-K. Copies of such agreements will be furnished to the Securities and Exchange Commission upon request.




10.1




*Supplemental Plan Abbott Laboratories Extended Disability Plan, filed as an exhibit (pages 50-51) to the 1992 Abbott Laboratories Annual Report on Form 10-K.**




10.2




*Abbott Laboratories Deferred Compensation Plan, as amended effective January 1, 2008, filed as Exhibit 4.1 to the 2008 Abbott Laboratories Annual Report on Form 10-K.**




10.3




†Abbott Laboratories 401(k) Supplemental Plan, as amended and restated.**




10.4




†Abbott Laboratories Supplemental Pension Plan, as amended and restated.**




10.5




†The 1986 Abbott Laboratories Management Incentive Plan, as amended and restated.**




10.6




*1998 Abbott Laboratories Performance Incentive Plan, as amended effective January 1, 2008, filed as Exhibit 10.7 to the 2008 Abbott Laboratories Annual Report on Form 10-K.**




10.7




†Rules for the 1998 Abbott Laboratories Performance Incentive Plan, as amended and restated.**

100

10-K
Exhibit
Table
Item No.

10.8


*The Abbott Laboratories 1996 Incentive Stock Program, as amended and restated through the 6th Amendment February 20, 2009, filed as Exhibit 10.11 to the Abbott Laboratories Quarterly Report on Form 10-Q for the quarter ended March 31, 2009.**




10.9




†Abbott Laboratories 2009 Incentive Stock Program, as amended and restated.**




10.10




†Abbott Laboratories Non-Employee Directors' Fee Plan, as amended and restated.**




10.11




*Form of Non-Employee Director Stock Option Agreement under the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 18, 2005, filed as Exhibit 10.7 to the Abbott Laboratories Current Report on Form 8-K dated February 18, 2005.**




10.12




*Form of Employee Stock Option Agreement for a Non-Qualified Stock Option granted with an Incentive Stock Option under the Abbott Laboratories 1996 Incentive Stock Program, filed as Exhibit 10.1 to the Abbott Laboratories Current Report on Form 8-K dated August 20, 2004.**




10.13




*Form of Employee Stock Option Agreement for a Non-Qualified Stock Option under the Abbott Laboratories 1996 Incentive Stock Program, filed as Exhibit 10.2 to the Abbott Laboratories Current Report on Form 8-K dated August 20, 2004.**




10.14




*Form of Employee Stock Option Agreement for an Incentive Stock Option granted with a Non-Qualified Stock Option under the Abbott Laboratories 1996 Incentive Stock Program, filed as Exhibit 10.3 to the Abbott Laboratories Current Report on Form 8-K dated August 20, 2004.**




10.15




*Form of Employee Stock Option Agreement for an Incentive Stock Option under the Abbott Laboratories 1996 Incentive Stock Program, filed as Exhibit 10.4 to the Abbott Laboratories Current Report on Form 8-K dated August 20, 2004.**




10.16




*Form of Employee Stock Option Agreement for a Replacement Stock Option under the Abbott Laboratories 1996 Incentive Stock Program, filed as Exhibit 10.5 to the Abbott Laboratories Current Report on Form 8-K dated August 20, 2004.**




10.17




*Form of Employee Restricted Stock Agreement under the Abbott Laboratories 1996 Incentive Stock Program, filed as Exhibit 10.6 to the Abbott Laboratories Current Report on Form 8-K dated August 20, 2004.**




10.18




*Form of Employee Restricted Stock Unit Agreement under the Abbott Laboratories 1996 Incentive Stock Program, filed as Exhibit 10.7 to the Abbott Laboratories Current Report on Form 8-K dated August 20, 2004.**




10.19




*Form of Non-Employee Director Stock Option Agreement under the Abbott Laboratories 1996 Incentive Stock Program, filed as Exhibit 10.8 to the Abbott Laboratories Current Report on Form 8-K dated August 20, 2004.**




10.20




*Form of Non-Employee Director Restricted Stock Unit Agreement under Abbott Laboratories 1996 Incentive Stock Program, filed as Exhibit 10.2 to the Abbott Laboratories Current Report on Form 8-K dated December 10, 2004.**




10.21




*Form of Employee Stock Option Agreement for a Non-Qualified Stock Option granted with an Incentive Stock Option under the Abbott Laboratories 1996 Incentive Stock Program on or after February 18, 2005, filed as Exhibit 10.1 to the Abbott Laboratories Current Report on Form 8-K dated February 18, 2005.**

101

10-K
Exhibit
Table
Item No.

10.22


*Form of Employee Stock Option Agreement for a new Non-Qualified Stock Option under the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 18, 2005, filed as Exhibit 10.2 to the Abbott Laboratories Current Report on Form 8-K dated February 18, 2005.**




10.23




*Form of Employee Stock Option Agreement for an Incentive Stock Option granted with a Non-Qualified Stock Option under the Abbott Laboratories 1996 Incentive Stock Program on or after February 18, 2005, filed as Exhibit 10.3 to the Abbott Laboratories Current Report on Form 8-K dated February 18, 2005.**




10.24




*Form of Employee Stock Option Agreement for an Incentive Stock Option under the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 18, 2005, filed as Exhibit 10.4 to the Abbott Laboratories Current Report on Form 8-K dated February 18, 2005.**




10.25




*Form of Employee Restricted Stock Agreement under the Abbott Laboratories 1996 Incentive Stock Program, filed as Exhibit 10.5 to the Abbott Laboratories Current Report on Form 8-K dated February 18, 2005.**




10.26




*Form of Employee Restricted Stock Unit Agreement under the Abbott Laboratories 1996 Incentive Stock Program, filed as Exhibit 10.6 to the Abbott Laboratories Current Report on Form 8-K dated February 18, 2005.**




10.27




*Form of Performance Restricted Stock Agreement for an award of performance restricted stock under Section 10 of the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 17, 2006, filed as Exhibit 10.1 to the Abbott Laboratories Current Report on Form 8-K dated February 16, 2006.**




10.28




*Form of Performance Restricted Stock Agreement for an award of performance restricted stock under Section 11 of the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 17, 2006, filed as Exhibit 10.2 to the Abbott Laboratories Current Report on Form 8-K dated February 16, 2006.**




10.29




*Form of Performance Restricted Stock Unit Agreement for an award of performance restricted stock units under the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 17, 2006, filed as Exhibit 10.3 to the Abbott Laboratories Current Report on Form 8-K dated February 16, 2006.**




10.30




*Form of Non-Qualified Stock Option Agreement for an award of non-qualified stock options under the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 17, 2006, filed as Exhibit 10.4 to the Abbott Laboratories Current Report on Form 8-K dated February 16, 2006.**




10.31




*Form of Restricted Stock Unit Agreement for an award of restricted stock units under the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 17, 2006, filed as Exhibit 10.5 to the Abbott Laboratories Current Report on Form 8-K dated February 16, 2006.**




10.32




*Form of Performance Restricted Stock Agreement for an award of performance restricted stock under Section 10 of the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 16, 2007, filed as Exhibit 10.49 to the 2006 Abbott Laboratories Report on Form 10-K.**

102

10-K
Exhibit
Table
Item No.

10.33


*Form of Performance Restricted Stock Agreement for an award of performance restricted stock under Section 11 of the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 16, 2007, filed as Exhibit 10.50 to the 2006 Abbott Laboratories Report on Form 10-K.**




10.34




*Form of Performance Restricted Stock Unit Agreement for an award of performance restricted stock units under the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 16, 2007, filed as Exhibit 10.51 to the 2006 Abbott Laboratories Report on Form 10-K.**




10.35




*Form of Restricted Stock Unit Agreement for an award of restricted stock units under the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 16, 2007, filed as Exhibit 10.52 to the 2006 Abbott Laboratories Report on Form 10-K.**




10.36




*Form of Performance Restricted Stock Agreement for an award of performance restricted stock under Section 10 of the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 20, 2009, filed as Exhibit 10.1 to the Abbott Laboratories Current Report on 8-K dated February 20, 2009.**




10.37




*Form of Performance Restricted Stock Agreement for an award of performance restricted stock under the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 20, 2009, filed as Exhibit 10.2 to the Abbott Laboratories Current Report on 8-K dated February 20, 2009.**




10.38




*Form of Non-Qualified Stock Option Agreement for an award of non-qualified stock options under the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 20, 2009, filed as Exhibit 10.3 to the Abbott Laboratories Current Report on 8-K dated February 20, 2009.**




10.39




*Form of Non-Qualified Replacement Stock Option Agreement for an award of non-qualified replacement stock options under the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 20, 2009, filed as Exhibit 10.4 to the Abbott Laboratories Current Report on 8-K dated February 20, 2009.**




10.40




*Form of Restricted Stock Agreement for an award of restricted stock under the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 20, 2009 (ratable vesting), filed as Exhibit 10.5 to the Abbott Laboratories Current Report on 8-K dated February 20, 2009.**




10.41




*Form of Restricted Stock Agreement for an award of restricted stock under the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 20, 2009 (cliff vesting), filed as Exhibit 10.6 to the Abbott Laboratories Current Report on 8-K dated February 20, 2009.**




10.42




*Form of Restricted Stock Unit Agreement for an award of restricted stock units under the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 20, 2009, filed as Exhibit 10.7 to the Abbott Laboratories Current Report on 8-K dated February 20, 2009.**




10.43




*Form of Non-Employee Director Non-Qualified Replacement Stock Option Agreement for an award of non-qualified replacement stock options under the Abbott Laboratories 1996 Incentive Stock Program granted on or after February 20, 2009, filed as Exhibit 10.8 to the Abbott Laboratories Current Report on 8-K dated February 20, 2009.**

103

10-K
Exhibit
Table
Item No.

10.44


*Form of Non-Employee Director Non-Qualified Stock Option Agreement, filed as Exhibit 10.2 to the Abbott Laboratories Current Report on Form 8-K dated April 24, 2009.**




10.45




*Form of Non-Employee Director Restricted Stock Unit Agreement, filed as Exhibit 10.3 to the Abbott Laboratories Current Report on Form 8-K dated April 24, 2009.**




10.46




*Form of Non-Employee Director Non-Qualified Replacement Stock Option Agreement, filed as Exhibit 10.4 to the Abbott Laboratories Current Report on Form 8-K dated April 24, 2009.**




10.47




*Form of Non-Qualified Stock Option Agreement (ratably vested), filed as Exhibit 10.5 to the Abbott Laboratories Current Report on Form 8-K dated April 24, 2009.**




10.48




*Form of Non-Qualified Replacement Stock Option Agreement, filed as Exhibit 10.6 to the Abbott Laboratories Current Report on Form 8-K dated April 24, 2009.**




10.49




*Form of Performance Restricted Stock Agreement, filed as Exhibit 10.7 to the Abbott Laboratories Current Report on Form 8-K dated April 24, 2009.**




10.50




*Form of Restricted Stock Agreement (ratably vested), filed as Exhibit 10.8 to the Abbott Laboratories Current Report on Form 8-K dated April 24, 2009.**




10.51




*Form of Restricted Stock Agreement (cliff vested), filed as Exhibit 10.9 to the Abbott Laboratories Current Report on Form 8-K dated April 24, 2009.**




10.52




*Form of Performance Restricted Stock Agreement (annual performance based), filed as Exhibit 10.10 to the Abbott Laboratories Current Report on Form 8-K dated April 24, 2009.**




10.53




*Form of Performance Restricted Stock Agreement (interim performance based), filed as Exhibit 10.11 to the Abbott Laboratories Current Report on Form 8-K dated April 24, 2009.**




10.54




*Form of Restricted Stock Unit Agreement (cliff vested), filed as Exhibit 10.12 to the Abbott Laboratories Current Report on Form 8-K dated April 24, 2009.**




10.55




*Form of Restricted Stock Unit Agreement (ratably vested), filed as Exhibit 10.13 to the Abbott Laboratories Current Report on Form 8-K dated April 24, 2009.**




10.56




*Form of Agreement Regarding Change in Control by and between Abbott Laboratories and certain named executive officers (other than Mr. White), filed as Exhibit 10.1 to the Abbott Laboratories Current Report on Form 8-K dated November 30, 2012.**




10.57




*Form of Time Sharing Agreement between Abbott Laboratories, Inc. and M.D. White and T.C. Freyman, filed as Exhibit 10.6 to the Abbott Laboratories Quarterly Report on Form 10-Q for the quarter ended June 30, 2006.**




10.58




*Amended and Restated Advanced Medical Optics, Inc. 2002 Incentive Compensation Plan, as amended, filed as Exhibit 4.3 to the Abbott Laboratories Registration Statement on Form S-8 dated March 20, 2009.**




10.59




*First Amendment to Amended and Restated Advanced Medical Optics, Inc. 2002 Incentive Compensation Plan, filed as Exhibit 4.4 to the Abbott Laboratories Registration Statement on Form S-8 dated March 20, 2009.**




10.60




*2004 Stock Incentive Plan, as amended and restated, filed as Exhibit 4.5 to the Abbott Laboratories Registration Statement on Form S-8 dated March 20, 2009.**

104

10-K
Exhibit
Table
Item No.

10.61


*Advanced Medical Optics, Inc. 2005 Incentive Compensation Plan, filed as Exhibit 4.6 to the Abbott Laboratories Registration Statement on Form S-8 dated March 20, 2009.**




10.62




*VISX, Incorporated 2001 Nonstatutory Stock Option Plan, filed as Exhibit 4.7 to the Abbott Laboratories Registration Statement on Form S-8 dated March 20, 2009.**




10.63




*VISX, Incorporated 2000 Stock Plan, filed as Exhibit 4.8 to the Abbott Laboratories Registration Statement on Form S-8 dated March 20, 2009.**




10.64




*VISX, Incorporated 1995 Director Option and Stock Deferral Plan, as amended and restated, filed as Exhibit 4.9 to the Abbott Laboratories Registration Statement on Form S-8 dated March 20, 2009.**




10.65




*VISX, Incorporated 1995 Stock Plan, as amended, filed as Exhibit 4.10 to the Abbott Laboratories Registration Statement on Form S-8 dated March 20, 2009.**




12




†Computation of Ratio of Earnings to Fixed Charges.




21




†Subsidiaries of Abbott Laboratories.




23.1




Consent of Independent Registered Public Accounting Firm.




31.1




Certification of Chief Executive Officer Required by Rule 13a-14(a) (17 CFR 240.13a-14(a)).




31.2




Certification of Chief Financial Officer Required by Rule 13a-14(a) (17 CFR 240.13a-14(a)).




32.1




Certification of Chief Executive Officer Pursuant to 18 U.S.C. Section 1350, as adopted pursuant to Section 906 of the Sarbanes-Oxley Act of 2002.




32.2




Certification of Chief Financial Officer Pursuant to 18 U.S.C. Section 1350, as adopted pursuant to Section 906 of the Sarbanes-Oxley Act of 2002.




101




†The following financial statements and notes from the Abbott Laboratories Annual Report on Form 10-K for the year ended December 31, 2012 filed on February 15, 2013, formatted in XBRL: (i) Consolidated Statement of Earnings; (ii) Consolidated Statement of Cash Flows; (iii) Consolidated Balance Sheet; (iv) Consolidated Statement of Shareholders' Investment; and (v) the notes to the consolidated financial statements.

        The 2013 Abbott Laboratories Proxy Statement will be filed with the Securities and Exchange Commission under separate cover on or about March 15, 2013.

*
Incorporated herein by reference. Commission file number 1-2189.

**
Denotes management contract or compensatory plan or arrangement required to be filed as an exhibit hereto.

†
Filed as an exhibit to the 2012 Abbott Laboratories Report on Form 10-K filed on February 15, 2013.
        Abbott will furnish copies of any of the above exhibits to a shareholder upon written request to the Secretary, Abbott Laboratories, 100 Abbott Park Road, Abbott Park, Illinois 60064-6400.

105
"""



In [ ]:
text_sample

'10-K/A 1 a2213952z10-ka.htm 10-K/A\n\n\nUNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWASHINGTON, D. C. 20549\n\n\n\nFORM 10-K/A\nAmendment No. 1\n\n(MARK ONE)\t \t \ný\t \tANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF\nTHE SECURITIES EXCHANGE ACT OF 1934\nOR\no\t \tTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF\nTHE SECURITIES EXCHANGE ACT OF 1934\n\n\nFor the fiscal year ended December 31, 2012\t \tCommission file number 1-2189\nAbbott Laboratories\n\nAn Illinois Corporation\t \t36-0698440\n100 Abbott Park Road\nAbbott Park, Illinois 60064-6400\t \t(I.R.S. employer identification number)\n(847) 937-6100\n(telephone number)\nSecurities Registered Pursuant to Section 12(b) of the Act:\n\n \nTitle of Each Class\n \tName of Each Exchange on Which Registered\n \nCommon Shares, Without Par Value\n  \t \tNew York Stock Exchange\nChicago Stock Exchange\n \nIndicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act.\n\

In [ ]:
# rmv new lines, rmv numbers...

In [ ]:
print(data[data['category'].notna()]['url'])

5      https://www.sec.gov/Archives/edgar/data/1800/0...
8      https://www.sec.gov/Archives/edgar/data/1923/0...
9      https://www.sec.gov/Archives/edgar/data/1923/0...
10     https://www.sec.gov/Archives/edgar/data/1923/0...
11     https://www.sec.gov/Archives/edgar/data/1923/0...
                             ...                        
336    https://www.sec.gov/Archives/edgar/data/4127/0...
337    https://www.sec.gov/Archives/edgar/data/4127/0...
338    https://www.sec.gov/Archives/edgar/data/4127/0...
339    https://www.sec.gov/Archives/edgar/data/4127/0...
359    https://www.sec.gov/Archives/edgar/data/4164/0...
Name: url, Length: 119, dtype: object


In [ ]:
https://colab.research.google.com/github/bemarina/text-clf-deep-learning/blob/main/data_preparation.ipynb#scrollTo=NQBbaInqrWFE&line=1&uniqifier=1

In [ ]:
# # split the dataset into training and validation datasets
# train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])
# print(type(train_y))

# # label encode the target variable (in both the training and the validation sets)
# encoder = preprocessing.LabelEncoder()
# train_y = encoder.fit_transform(train_y)
# valid_y = encoder.fit_transform(valid_y)

# print(type(train_y))